## Document Comparison

In this notebook we will look at how MaterialX Elements can be compared with one another.

The example case chosen is to determine what has changed between 2 versions of the standard
libraries. In this case version 1.38 versus the current version used to test with.

The followin aspects will be covered:
1. Loading in different versions into separate Documents.
2. Looking at what has been added or removed.
3. Performing a detailed comparison between Elements.

### Library Loading

We will first load in two versions of the standard libraries.
- One is the "current" one found in the 'libraries' folder of the MaterialX
Python distribition. 
- The other is a downloaded version of the same libraries copied into a renamed folder. 

In [49]:
import MaterialX as mx
from mtlxutils.mxfile import *
from IPython.display import display_markdown

# Load the current standard libraries
currLibrary, currlibFiles, status = MtlxFile.createLibraryDocument()
print(status)

# Load in the old standard libraries
otherVersion = ''
libraryFolders = [ 'libraries1.38' ]
otherLibrary = mx.createDocument()
searchPath = mx.FileSearchPath('../resources/')
# Attempt to preserve the original version of the library
options = mx.XmlReadOptions()
options.upgradeVersion = False

libFiles = mx.loadLibraries(libraryFolders, searchPath, otherLibrary)
print('- Loaded %d old library definitions from %d files' % (len(otherLibrary.getNodeDefs()), len(libFiles)))

- Loaded 730 library definitions from 47 files
- Loaded 663 old library definitions from 26 files


It is possible to selectively filter out which files by providing additional arguments to `loadLibraries` or to perform the equivalent traversal explicitly as shown below.

In [50]:
otherLibrary2 = mx.createDocument()
otherVersion2 = ''
loadedLibraries = set()
readOptions = mx.XmlReadOptions()
readOptions.upgradeVersion = False
for libraryName in libraryFolders:
    libraryPath = searchPath.find(libraryName)
    for path in libraryPath.getSubDirectories():
          for filename in path.getFilesInDirectory('mtlx'):
                file = path / filename;
                if file.asString() not in loadedLibraries:
                    mx.loadLibrary(file, otherLibrary2, searchPath, readOptions)                    
                    loadedLibraries.add(file.asString())
                    otherVersion2 = otherLibrary2.getVersionString()  

print('Read in %d definitions from %d files. Version: %s' % 
      (len(otherLibrary2.getNodeDefs()), len(loadedLibraries), otherVersion2))

Read in 663 definitions from 26 files. Version: 1.38


Note that both libraries will return same version when querying using `getVersionString()` on the document.
At the time of writing we are using a patch and thus grab the version from the Python package instead.

It would be useful if there was a way to find patch versions on documents but this is currently
not supported. Integrations could write additional `patch` meta-data atttibutes but this would not
be standard.

In [51]:
print('- Current library / package version: %s' % mx.getVersionString())
print('- Old library version: %s' % otherLibrary.getVersionString())

- Current library / package version: 1.38.8
- Old library version: 1.38


### Testing for Definition Additions and Removals 

The first test is just a check on the number of definitions in each file. The result found is that there are more definitions, and there appear to be no definitions which have been removed.

In [52]:
# Compare number of nodedefs
currNodeDefs = currLibrary.getNodeDefs()
currNodeDefsCount = len(currNodeDefs)
otherNodeDefs = otherLibrary.getNodeDefs()
otherNodeDefsCount = len(otherNodeDefs) # There seems to be no way to get the previous version?
print('There are %d new definitions.' % (currNodeDefsCount - otherNodeDefsCount))
print('- Current library has %d nodedefs' % currNodeDefsCount)
print('- Other library has %d nodedefs' % otherNodeDefsCount)

There are 67 new definitions.
- Current library has 730 nodedefs
- Other library has 663 nodedefs


To setup for comparision testing, we build up a dictionary of node definitions keyed by the name of the definition.

At time of writing the convention is that definitions cannot be defined at any location other that the child of root Document. Thus only the name is used as the key. The path could also be used for long term robustness if this convention ever changes.

In [53]:
# Create dictionary of current node definitions
currNodeDefSet = {}
for nodeDef in currNodeDefs:
    currNodeDefSet[nodeDef.getName()] = nodeDef

# Create dictionary of old node definitions
otherNodeDefSet = {}
for nodeDef in otherNodeDefs:
    otherNodeDefSet[nodeDef.getName()] = nodeDef


The first check is to see if any definitions have been removed. 

If a definition has been remapped then the hope is that the update process performed on load will handle this. For example when material nodes replaced material elements as part of 1.38.7 the update process performed the appropriate remapping.  

If the definition no longer exists then the integration will need to find out how to handle this. In general, this should hopefully never occur.

In [54]:
    
# Find out if any definitions were removed
removedNodeDefs = []
for nodeDef in otherNodeDefs:
    if nodeDef.getName() not in currNodeDefSet:
        removedNodeDefs.append(nodeDef)

text = ''  
if removedNodeDefs:
    removedNodeDefs = sorted(removedNodeDefs, key=lambda x: x.getName())
    text = text + '| Name | Category |\n'
    text = text + '| :--- | :--- |\n'
    for nd in removedNodeDefs:
        text = text + '| %s | %s |\n' % (nd.getName(), nd.getNodeString())

    delta = '%d node definitions removed' % len(removedNodeDefs)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

<details open><summary>9 node definitions removed</summary>

| Name | Category |
| :--- | :--- |
| ND_add_displacementshader | add |
| ND_add_surfaceshader | add |
| ND_add_volumeshader | add |
| ND_multiply_displacementshaderF | multiply |
| ND_multiply_displacementshaderV | multiply |
| ND_multiply_surfaceshaderC | multiply |
| ND_multiply_surfaceshaderF | multiply |
| ND_multiply_volumeshaderC | multiply |
| ND_multiply_volumeshaderF | multiply |

</details>


The next check is to see if any definitions have been added and list them out.

At the same time we cache a list of definitions which exist in both the old and new library versions for later
comparison.

In [55]:
# Find out what new definitions have been added.
# Cache common definitions in `compareDetails` for further comparison.
compareDetails = []
newNodeDefs = []
for nodeDef in currNodeDefs:
    if nodeDef.getName() not in otherNodeDefSet:
        newNodeDefs.append(nodeDef)
    else:
        # Cache these two for more comparison
        otherNodeDef = otherNodeDefSet[nodeDef.getName()]
        compareDetails.append( [otherNodeDef, nodeDef] )

text = ''
if newNodeDefs:
    newNodeDefs = sorted(newNodeDefs, key=lambda x: x.getNodeString())
    text = text + '| Name | Category | NodeGroup |\n'
    text = text + '| :--- | :--- | :---| \n'
    for nd in newNodeDefs:
        text = text + '| <a href="../documents/definitions/%s.html">%s</a> | %s | %s |\n' % (nd.getNodeString(), nd.getName(), nd.getNodeString(), nd.getNodeGroup())

    delta = '%d node definitions added. (Sorted by category)' % len(newNodeDefs)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

<details open><summary>76 node definitions added. (Sorted by category)</summary>

| Name | Category | NodeGroup |
| :--- | :--- | :---| 
| <a href="../documents/definitions/LamaAdd.html">ND_lama_add_bsdf</a> | LamaAdd | pbr |
| <a href="../documents/definitions/LamaAdd.html">ND_lama_add_edf</a> | LamaAdd | pbr |
| <a href="../documents/definitions/LamaConductor.html">ND_lama_conductor</a> | LamaConductor | pbr |
| <a href="../documents/definitions/LamaDielectric.html">ND_lama_dielectric</a> | LamaDielectric | pbr |
| <a href="../documents/definitions/LamaDiffuse.html">ND_lama_diffuse</a> | LamaDiffuse | pbr |
| <a href="../documents/definitions/LamaEmission.html">ND_lama_emission</a> | LamaEmission | pbr |
| <a href="../documents/definitions/LamaLayer.html">ND_lama_layer_bsdf</a> | LamaLayer | pbr |
| <a href="../documents/definitions/LamaMix.html">ND_lama_mix_bsdf</a> | LamaMix | pbr |
| <a href="../documents/definitions/LamaMix.html">ND_lama_mix_edf</a> | LamaMix | pbr |
| <a href="../documents/definitions/LamaSSS.html">ND_lama_sss</a> | LamaSSS | pbr |
| <a href="../documents/definitions/LamaSheen.html">ND_lama_sheen</a> | LamaSheen | pbr |
| <a href="../documents/definitions/LamaTranslucent.html">ND_lama_translucent</a> | LamaTranslucent | pbr |
| <a href="../documents/definitions/UsdUVTexture.html">ND_UsdUVTexture_23</a> | UsdUVTexture | texture2d |
| <a href="../documents/definitions/acescg_to_lin_rec709.html">ND_acescg_to_lin_rec709_color3</a> | acescg_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/acescg_to_lin_rec709.html">ND_acescg_to_lin_rec709_color4</a> | acescg_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/adobergb_to_lin_rec709.html">ND_adobergb_to_lin_rec709_color3</a> | adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/adobergb_to_lin_rec709.html">ND_adobergb_to_lin_rec709_color4</a> | adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/bump.html">ND_bump_vector3</a> | bump | geometric |
| <a href="../documents/definitions/ceil.html">ND_ceil_integer</a> | ceil | math |
| <a href="../documents/definitions/checkerboard.html">ND_checkerboard_color3</a> | checkerboard | procedural2d |
| <a href="../documents/definitions/colorcorrect.html">ND_colorcorrect_color3</a> | colorcorrect | adjustment |
| <a href="../documents/definitions/colorcorrect.html">ND_colorcorrect_color4</a> | colorcorrect | adjustment |
| <a href="../documents/definitions/convert.html">ND_convert_color3_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_color4_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_float_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector2_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector3_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_vector4_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_integer_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/convert.html">ND_convert_boolean_surfaceshader</a> | convert | shader |
| <a href="../documents/definitions/distance.html">ND_distance_vector2</a> | distance | math |
| <a href="../documents/definitions/distance.html">ND_distance_vector3</a> | distance | math |
| <a href="../documents/definitions/distance.html">ND_distance_vector4</a> | distance | math |
| <a href="../documents/definitions/floor.html">ND_floor_integer</a> | floor | math |
| <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">ND_g18_rec709_to_lin_rec709_color3</a> | g18_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">ND_g18_rec709_to_lin_rec709_color4</a> | g18_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">ND_g22_ap1_to_lin_rec709_color3</a> | g22_ap1_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">ND_g22_ap1_to_lin_rec709_color4</a> | g22_ap1_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">ND_g22_rec709_to_lin_rec709_color3</a> | g22_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">ND_g22_rec709_to_lin_rec709_color4</a> | g22_rec709_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/generalized_schlick_edf.html">ND_generalized_schlick_edf</a> | generalized_schlick_edf | pbr |
| <a href="../documents/definitions/gltf_colorimage.html">ND_gltf_colorimage</a> | gltf_colorimage | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_color3_color3_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_color4_color4_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_float_float_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_image.html">ND_gltf_image_vector3_vector3_1_0</a> | gltf_image | texture2d |
| <a href="../documents/definitions/gltf_iridescence_thickness.html">ND_gltf_iridescence_thickness_float_1_0</a> | gltf_iridescence_thickness | texture2d |
| <a href="../documents/definitions/gltf_normalmap.html">ND_gltf_normalmap_vector3_1_0</a> | gltf_normalmap | texture2d |
| <a href="../documents/definitions/gltf_pbr.html">ND_gltf_pbr_surfaceshader</a> | gltf_pbr | pbr |
| <a href="../documents/definitions/layer.html">ND_layer_vdf</a> | layer | pbr |
| <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">ND_lin_adobergb_to_lin_rec709_color3</a> | lin_adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">ND_lin_adobergb_to_lin_rec709_color4</a> | lin_adobergb_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">ND_lin_displayp3_to_lin_rec709_color3</a> | lin_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">ND_lin_displayp3_to_lin_rec709_color4</a> | lin_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/mix.html">ND_mix_color3_color3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_color4_color4</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector2_vector2</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector3_vector3</a> | mix | compositing |
| <a href="../documents/definitions/mix.html">ND_mix_vector4_vector4</a> | mix | compositing |
| <a href="../documents/definitions/randomcolor.html">ND_randomcolor_float</a> | randomcolor | procedural3d |
| <a href="../documents/definitions/randomcolor.html">ND_randomcolor_integer</a> | randomcolor | procedural3d |
| <a href="../documents/definitions/randomfloat.html">ND_randomfloat_float</a> | randomfloat | procedural |
| <a href="../documents/definitions/randomfloat.html">ND_randomfloat_integer</a> | randomfloat | procedural |
| <a href="../documents/definitions/rec709_display_to_lin_rec709.html">ND_rec709_display_to_lin_rec709_color3</a> | rec709_display_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/rec709_display_to_lin_rec709.html">ND_rec709_display_to_lin_rec709_color4</a> | rec709_display_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">ND_srgb_displayp3_to_lin_rec709_color3</a> | srgb_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">ND_srgb_displayp3_to_lin_rec709_color4</a> | srgb_displayp3_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">ND_srgb_texture_to_lin_rec709_color3</a> | srgb_texture_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">ND_srgb_texture_to_lin_rec709_color4</a> | srgb_texture_to_lin_rec709 | colortransform |
| <a href="../documents/definitions/standard_surface.html">ND_standard_surface_surfaceshader_100</a> | standard_surface | pbr |
| <a href="../documents/definitions/standard_surface_to_UsdPreviewSurface.html">ND_standard_surface_to_UsdPreviewSurface</a> | standard_surface_to_UsdPreviewSurface | translation |
| <a href="../documents/definitions/standard_surface_to_gltf_pbr.html">ND_standard_surface_to_gltf_pbr</a> | standard_surface_to_gltf_pbr | translation |
| <a href="../documents/definitions/surface_unlit.html">ND_surface_unlit</a> | surface_unlit | shader |
| <a href="../documents/definitions/trianglewave.html">ND_trianglewave_float</a> | trianglewave | math |
| <a href="../documents/definitions/unifiednoise2d.html">ND_unifiednoise2d_float</a> | unifiednoise2d | procedural2d |
| <a href="../documents/definitions/unifiednoise3d.html">ND_unifiednoise3d_float</a> | unifiednoise3d | procedural3d |

</details>


We print out the category of these new definitions. It is possible that new categories of nodes have been added
in addition to adding new variants on existing categories.

As there is no formal list of such categories, a check needs to be made by scanning all definitions. 

In [56]:
oldCategories = set()
for nodeDef in otherNodeDefs:
    oldCategories.add(nodeDef.getNodeString())
# Sort old categories
oldCategories = sorted(oldCategories)

newCategories = {}
for nodeDef in currNodeDefs:
    newCategory = nodeDef.getNodeString()
    if newCategory not in oldCategories:
        newCategories[newCategory] = [ nodeDef.getNodeGroup(), nodeDef.getType() ]        

if newCategories:
    text = '| Node category | Node group | Type |\n'
    text = text + '| --- | --- | --- |\n' 
    sorted_keys = sorted(newCategories.keys())
    sortedNewCategories = {}
    for key in sorted_keys:
        sortedNewCategories[key] = newCategories[key]                                 
    for category in sortedNewCategories:
        text = text + '| [%s](../documents/definitions/%s.html) | %s | %s |\n' % (category, category, newCategories[category][0], newCategories[category][1])

    delta = '%d new node categories found' % len(newCategories)
    text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
    display_markdown(text, raw=True)

<details open><summary>38 new node categories found</summary>

| Node category | Node group | Type |
| --- | --- | --- |
| [LamaAdd](../documents/definitions/LamaAdd.html) | pbr | EDF |
| [LamaConductor](../documents/definitions/LamaConductor.html) | pbr | BSDF |
| [LamaDielectric](../documents/definitions/LamaDielectric.html) | pbr | BSDF |
| [LamaDiffuse](../documents/definitions/LamaDiffuse.html) | pbr | BSDF |
| [LamaEmission](../documents/definitions/LamaEmission.html) | pbr | EDF |
| [LamaLayer](../documents/definitions/LamaLayer.html) | pbr | BSDF |
| [LamaMix](../documents/definitions/LamaMix.html) | pbr | EDF |
| [LamaSSS](../documents/definitions/LamaSSS.html) | pbr | BSDF |
| [LamaSheen](../documents/definitions/LamaSheen.html) | pbr | BSDF |
| [LamaTranslucent](../documents/definitions/LamaTranslucent.html) | pbr | BSDF |
| [acescg_to_lin_rec709](../documents/definitions/acescg_to_lin_rec709.html) | colortransform | color4 |
| [adobergb_to_lin_rec709](../documents/definitions/adobergb_to_lin_rec709.html) | colortransform | color4 |
| [bump](../documents/definitions/bump.html) | geometric | vector3 |
| [checkerboard](../documents/definitions/checkerboard.html) | procedural2d | color3 |
| [colorcorrect](../documents/definitions/colorcorrect.html) | adjustment | color4 |
| [distance](../documents/definitions/distance.html) | math | float |
| [g18_rec709_to_lin_rec709](../documents/definitions/g18_rec709_to_lin_rec709.html) | colortransform | color4 |
| [g22_ap1_to_lin_rec709](../documents/definitions/g22_ap1_to_lin_rec709.html) | colortransform | color4 |
| [g22_rec709_to_lin_rec709](../documents/definitions/g22_rec709_to_lin_rec709.html) | colortransform | color4 |
| [generalized_schlick_edf](../documents/definitions/generalized_schlick_edf.html) | pbr | EDF |
| [gltf_colorimage](../documents/definitions/gltf_colorimage.html) | texture2d | multioutput |
| [gltf_image](../documents/definitions/gltf_image.html) | texture2d | vector3 |
| [gltf_iridescence_thickness](../documents/definitions/gltf_iridescence_thickness.html) | texture2d | float |
| [gltf_normalmap](../documents/definitions/gltf_normalmap.html) | texture2d | vector3 |
| [gltf_pbr](../documents/definitions/gltf_pbr.html) | pbr | surfaceshader |
| [lin_adobergb_to_lin_rec709](../documents/definitions/lin_adobergb_to_lin_rec709.html) | colortransform | color4 |
| [lin_displayp3_to_lin_rec709](../documents/definitions/lin_displayp3_to_lin_rec709.html) | colortransform | color4 |
| [randomcolor](../documents/definitions/randomcolor.html) | procedural3d | color3 |
| [randomfloat](../documents/definitions/randomfloat.html) | procedural | float |
| [rec709_display_to_lin_rec709](../documents/definitions/rec709_display_to_lin_rec709.html) | colortransform | color4 |
| [srgb_displayp3_to_lin_rec709](../documents/definitions/srgb_displayp3_to_lin_rec709.html) | colortransform | color4 |
| [srgb_texture_to_lin_rec709](../documents/definitions/srgb_texture_to_lin_rec709.html) | colortransform | color4 |
| [standard_surface_to_UsdPreviewSurface](../documents/definitions/standard_surface_to_UsdPreviewSurface.html) | translation | multioutput |
| [standard_surface_to_gltf_pbr](../documents/definitions/standard_surface_to_gltf_pbr.html) | translation | multioutput |
| [surface_unlit](../documents/definitions/surface_unlit.html) | shader | surfaceshader |
| [trianglewave](../documents/definitions/trianglewave.html) | math | float |
| [unifiednoise2d](../documents/definitions/unifiednoise2d.html) | procedural2d | float |
| [unifiednoise3d](../documents/definitions/unifiednoise3d.html) | procedural3d | float |

</details>


Additional checks can be added for non definition elements such as geometry properties, and type definitions.
For this notebook we will concentrate on definitions only.

### Detailed Element Comparison

A comparison between 2 Elements in MaterialX is available the the equivalence operator `==` on an Element.
Though this will indicate if they differ, it provides no information as to what has differed.

To accomplish this a new utility called `compareElements` has been written. In addition to providing explicit feedback on what has changed it will explicitly recurse to children by calling itself (instead of recursing using the `==` operator)

The basic properities tested are:
1. Element category
2. Element attribute names
3. Elememt attribute values
4. Child Element count and what children were added or removed.
5. Recursing on child Elements. This would generally be Inputs and Outputs in the case of defintions.
   If comparing graph elements (GraphElement), children could be child Nodes or graphs (GraphElements). 

> Note that this is the exact same logic as the `==` operator at time of writing. It can be extended as needed but the basic logic for checking attrbutes and then recursing on child Elements should remain the same. 

A few additional utilities are added to support comparison logic:

1. `getActiveChild()` and `getActiveChildren()` is added to support class inheritance. If this is not
considered then the children compared will only be the ones that are explicitly specified for a given class.
> Note that this would be a useful API to have and would complement the existing `getActiveInputs()`and `getActiveOutputs()` methods on an <a href="https://materialx.org/docs/api/class_interface_element.html" target="_blank">InterfaceElement</a>. It is mostly useful for comparison of implementations and definitions where the interface is being
compared. It is less useful for a node instance since comparison logic is aimed at what is explicitly specified on
instances.

2. `getActiveAttribute()` and `getActiveAttributes()` is added to support class inheritance when querying for attributes.
This serves the same purpose as the child methods to ensure inherited attributes are not missed.

3. `diffLists()` is added to find the intersection and difference between two lists.

Inheritance functions used the <a href="https://materialx.org/docs/api/class_element.html" target="_blank">
traverseInheritance()</a> API to traverse up the inheritance hierarchy.

In [57]:
def getActiveChild(element, name):
    '''
    Check if a child with a given name exists in the inheritance hierarchy for a given Element.
    '''
    for elem in element.traverseInheritance():
        child = elem.getChild(name)
        if child:
            return child
    return None

def getActiveChildren(element):
    '''
    Get all children (included inherited ones) for a given Element.
    '''
    children = []
    childset = set()
    for elem in element.traverseInheritance():
        for child in elem.getChildren():
            if child.getName() not in childset:
                children.append(child)
                childset.add(child.getName())
    return children    

def getActiveAttribute(element, name):
    ''' 
    Check if an attribute with a given name exists in the inheritance hierarchy for a given Element.
    '''
    for elem in element.traverseInheritance():
        child = elem.getAttribute(name)
        if child:
            return child
    return None

def getActiveAttributes(element):
    '''
    Get all attributes (included inherited ones) for a given Element.
    '''
    children = []
    childset = set()
    for elem in element.traverseInheritance():
        for child in elem.getAttributeNames():
            if child not in childset:
                children.append(child)
                childset.add(child)
    return children

def diffLists(list1, list2):
    '''
    Return items in either list1 or list2, and in both.
    '''
    if list1 != list2:
        # Find items in list1 not in list2
        onlyList1 = [item for item in list1 if item not in list2]
        # Find items in list2 not in list1
        onlyList2 = [item for item in list2 if item not in list1]
        # Find items in both lists
        common = [item for item in list1 if item in list2]
        return onlyList1, onlyList2, common
    return [], [], list1     

Using these utility functions, the logic for `compareElements()` is as follows:
1. Check category between elements
2. Compare attributes

In [58]:
def compareElements(nd1, nd2, indent, log):
    '''
    Compare two elements and print any differences.

    Tests performed:
    1. Element category
    2. Element attribute names
    3. Elememt attribute values
    4. Child Element count and what children are present in one but not the other.
    5. Recursing on child Elements.
    '''
    if not nd1 or not nd2:
        return True

    differs = False

    if nd1.getCategory() != nd2.getCategory():
        log.append(indent + '- Category changed on: `%s` from `%s` to `%s`' % (nd1.getName(), nd2.getCategory(), nd1.getCategory()))
        differs = True

    # Compare attributes.
    # Note: This does not work properly with inheritance yet.
    onlynd1, onlynd2, common = diffLists(getActiveAttributes(nd1), getActiveAttributes(nd2))
    if len(onlynd1) > 0 or len(onlynd2) > 0:
        for item in onlynd1:
            log.append(indent + '- attribute: `%s`=( `%s` ) removed from: `%s`' % 
                       ( item, getActiveAttribute(nd1, item), nd1.getName()))
        for item in onlynd2:
            log.append(indent + '- attribute: `%s`=( `%s` ) added to: `%s`' % 
                       ( item, getActiveAttribute(nd2, item), nd2.getName()))
        differs = True

    for attr in common:
        if nd2.getAttribute(attr) != nd1.getAttribute(attr):
            log.append(indent + '  - attribute: `%s` changed on `%s` from `( %s )` to `( %s )`' % 
                       (attr, nd1.getName(), nd1.getAttribute(attr), nd2.getAttribute(attr)))
            differs = True

    # Compare children. This also checks for children that inherited 
    c1 = [c.getName() for c in getActiveChildren(nd1)] 
    c2 = [c.getName() for c in getActiveChildren(nd2)]
    if len(c1) != len(c2):
        log.append(indent + '- Number of children on: `%s` changed from ( %s ) to ( %s )' % (nd1.getName(), len(c1), len(c2)))
        differs = True

    # Get list of added, removed, and common children.
    onlynd1, onlynd2, common = diffLists(c1, c2)

    # Log children that are removed. Emit value if is a value element.
    for childName in onlynd1:
        child = nd1.getChild(childName)
        childName = child.getNamePath().removeprefix(child.getParent().getNamePath())
        childType = child.getCategory()
        if child.isA(mx.ValueElement):
            log.append(indent + '- %s `%s` removed from `%s`. Value=( %s )' % (childType, childName, nd1.getNamePath(), child.getValueString()))
        else:
            log.append(indent + '- %s `%s` removed from `%s`' % (childType, childName, nd1.getNamePath()))
        differs = True

    # Log child that are added. Emit value if is a value element.
    for childName in onlynd2:
        child = nd2.getChild(childName)
        childName = child.getNamePath().removeprefix(child.getParent().getNamePath())
        childType = child.getCategory()
        if child.isA(mx.ValueElement):
            log.append(indent + '- %s `%s` added to `%s`. Value=( %s )' % (childType, childName, nd2.getNamePath(), child.getValueString()))
        else:
            log.append(indent + '- %s `%s` add to `%s`' % (childType, childName, nd2.getNamePath()))
        differs = True

    # Recurse on common children.
    for childName in common:
        c1child = getActiveChild(nd1, childName)
        c2child = getActiveChild(nd2, childName)
        if compareElements(c1child, c2child, indent, log):
            differs = True

    return differs

> Note that unlike the C++ implementation children are not compared by index as the ordering may have changed between versions, thus resulting in false mismatches appearing. 

> **It may be useful to update the C++ implementation to be independent of ordering.**

Given this utility, we scan through all the definitions which exist in both libraries.

Some items of interest include changes in default value for inputs on some nodes, as well as addition of uniform qualifiers to some nodes.

In [59]:
#text = '### Compare old and current definitions in detail\n'
text = '| Name | Change  |\n'
text = text + '| --- | --- |\n'

changed = 0
for item in compareDetails:
    nd1 = item[0]
    nd2 = item[1]
    # Use the built in comparitor first
    difference = (nd1 != nd2)
    if difference:
        # Perform details comparison
        text = text + '| <a href="../documents/definitions/%s.html">%s</a> | ' % (nd1.getNodeString(), nd1.getName())
        log = []
        difference = compareElements(nd1, nd2, '', log) 
        if difference:
            for line in log:
                text = text + line + '<br>'
            text = text + '|\n'
            changed = changed + 1

delta = '%d definitions modified' % changed
text = '<details open><summary>' + delta + '</summary>\n\n' + text + '\n' + '</details>\n' 
display_markdown(text, raw=True)


<details open><summary>36 definitions modified</summary>

| Name | Change  |
| --- | --- |
| <a href="../documents/definitions/disney_brdf_2012.html">ND_disney_brdf_2012_surface</a> | - attribute: `nodegroup`=( `pbr` ) added to: `ND_disney_brdf_2012_surface`<br>|
| <a href="../documents/definitions/disney_bsdf_2015.html">ND_disney_bsdf_2015_surface</a> | - attribute: `nodegroup`=( `pbr` ) added to: `ND_disney_bsdf_2015_surface`<br>|
| <a href="../documents/definitions/standard_surface.html">ND_standard_surface_surfaceshader</a> | - attribute: `inherit`=( `ND_standard_surface_surfaceshader_100` ) added to: `ND_standard_surface_surfaceshader`<br>- attribute: `uisoftmin`=( `1.0` ) added to: `specular_IOR`<br>  - attribute: `uimin` changed on `subsurface_anisotropy` from `( 0.0 )` to `( -1.0 )`<br>- attribute: `uisoftmin`=( `1.0` ) added to: `coat_IOR`<br>- attribute: `uisoftmin`=( `1.0` ) added to: `thin_film_IOR`<br>  - attribute: `doc` changed on `thin_walled` from `( If true the surface is double-sided and represents an infinitely thin shell. Suiteable for thin objects such as tree leafs or paper )` to `( If true the surface is double-sided and represents an infinitely thin shell. Suitable for thin objects such as tree leaves or paper. )`<br>|
| <a href="../documents/definitions/UsdPreviewSurface.html">ND_UsdPreviewSurface_surfaceshader</a> | - attribute: `uimin`=( `0,0,0` ) added to: `diffuseColor`<br>- attribute: `uimax`=( `1,1,1` ) added to: `diffuseColor`<br>- attribute: `uimin`=( `0,0,0` ) added to: `emissiveColor`<br>- attribute: `uisoftmax`=( `1,1,1` ) added to: `emissiveColor`<br>- attribute: `uimin`=( `0,0,0` ) added to: `specularColor`<br>- attribute: `uimax`=( `1,1,1` ) added to: `specularColor`<br>- attribute: `uimin`=( `0.0` ) added to: `metallic`<br>- attribute: `uimax`=( `1.0` ) added to: `metallic`<br>- attribute: `uimin`=( `0.0` ) added to: `roughness`<br>- attribute: `uimax`=( `1.0` ) added to: `roughness`<br>  - attribute: `value` changed on `roughness` from `( 0.01 )` to `( 0.5 )`<br>- attribute: `uimin`=( `0.0` ) added to: `clearcoat`<br>- attribute: `uimax`=( `1.0` ) added to: `clearcoat`<br>- attribute: `uimin`=( `0.0` ) added to: `clearcoatRoughness`<br>- attribute: `uimax`=( `1.0` ) added to: `clearcoatRoughness`<br>- attribute: `uimin`=( `0.0` ) added to: `opacity`<br>- attribute: `uimax`=( `1.0` ) added to: `opacity`<br>- attribute: `uimin`=( `0.0` ) added to: `opacityThreshold`<br>- attribute: `uimax`=( `1.0` ) added to: `opacityThreshold`<br>- attribute: `uimin`=( `0.0` ) added to: `ior`<br>- attribute: `uisoftmin`=( `1.0` ) added to: `ior`<br>- attribute: `uisoftmax`=( `3.0` ) added to: `ior`<br>- attribute: `uimin`=( `0.0` ) added to: `occlusion`<br>- attribute: `uimax`=( `1.0` ) added to: `occlusion`<br>|
| <a href="../documents/definitions/UsdUVTexture.html">ND_UsdUVTexture</a> | - attribute: `nodegroup`=( `texture2d` ) added to: `ND_UsdUVTexture`<br>- attribute: `version`=( `2.2` ) added to: `ND_UsdUVTexture`<br>- attribute: `inherit`=( `ND_UsdUVTexture_23` ) added to: `ND_UsdUVTexture`<br>- attribute: `isdefaultversion`=( `true` ) added to: `ND_UsdUVTexture`<br>- attribute: `value`=( `None` ) added to: `file`<br>  - attribute: `enum` changed on `wrapS` from `( black,clamp,periodic )` to `( black,clamp,periodic,mirror )`<br>  - attribute: `enum` changed on `wrapT` from `( black,clamp,periodic )` to `( black,clamp,periodic,mirror )`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_integer</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_integer`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_boolean</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_boolean`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_string</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_string`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_float</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_float`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector2</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_vector2`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector3</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_vector3`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">ND_UsdPrimvarReader_vector4</a> | - attribute: `nodegroup`=( `geometric` ) added to: `ND_UsdPrimvarReader_vector4`<br>- attribute: `value`=( `None` ) added to: `varname`<br>|
| <a href="../documents/definitions/UsdTransform2d.html">ND_UsdTransform2d</a> | - attribute: `nodegroup`=( `math` ) added to: `ND_UsdTransform2d`<br>- attribute: `value`=( `0, 0` ) added to: `in`<br>|
| <a href="../documents/definitions/conductor_bsdf.html">ND_conductor_bsdf</a> |   - attribute: `value` changed on `ior` from `( 0.271, 0.677, 1.316 )` to `( 0.183, 0.421, 1.373 )`<br>  - attribute: `value` changed on `extinction` from `( 3.609, 2.625, 2.292 )` to `( 3.424, 2.346, 1.770 )`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_bsdf</a> |   - attribute: `value` changed on `mix` from `( 1.0 )` to `( 0.0 )`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_edf</a> |   - attribute: `value` changed on `mix` from `( 1.0 )` to `( 0.0 )`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_vdf</a> |   - attribute: `value` changed on `mix` from `( 1.0 )` to `( 0.0 )`<br>|
| <a href="../documents/definitions/blackbody.html">ND_blackbody</a> |   - attribute: `type` changed on `out` from `( float )` to `( color3 )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_float</a> | - Number of children on: `ND_triplanarprojection_float` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_float`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_color3</a> | - Number of children on: `ND_triplanarprojection_color3` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_color3`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_color4</a> | - Number of children on: `ND_triplanarprojection_color4` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_color4`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector2</a> | - Number of children on: `ND_triplanarprojection_vector2` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_vector2`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector3</a> | - Number of children on: `ND_triplanarprojection_vector3` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_vector3`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/triplanarprojection.html">ND_triplanarprojection_vector4</a> | - Number of children on: `ND_triplanarprojection_vector4` changed from ( 14 ) to ( 15 )<br>- input `/blend` added to `ND_triplanarprojection_vector4`. Value=( 1.0 )<br>|
| <a href="../documents/definitions/geompropvalue.html">ND_geompropvalue_integer</a> | - attribute: `uniform`=( `true` ) removed from: `default`<br>|
| <a href="../documents/definitions/place2d.html">ND_place2d_vector2</a> | - Number of children on: `ND_place2d_vector2` changed from ( 6 ) to ( 7 )<br>- input `/operationorder` added to `ND_place2d_vector2`. Value=( 0 )<br>|
| <a href="../documents/definitions/mix.html">ND_mix_float</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_color3</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_color4</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_vector2</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_vector3</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_vector4</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_surfaceshader</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_displacementshader</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/mix.html">ND_mix_volumeshader</a> | - attribute: `uisoftmin`=( `0.0` ) added to: `mix`<br>- attribute: `uisoftmax`=( `1.0` ) added to: `mix`<br>|
| <a href="../documents/definitions/heighttonormal.html">ND_heighttonormal_vector3</a> |   - attribute: `default` changed on `out` from `( 0.0, 1.0, 0.0 )` to `( 0.5, 0.5, 1.0 )`<br>|

</details>


## Target and Implementation Comparison

Next we cover how add logic to see if implementations have been added or removed, or
if existing implementations have changed. This includes checking in the context of whether new `target`s have
been added or not.

### Target Check

The list of targets can be queried using `getTargetDefs()` on each librari respectively and the
compared.

In [60]:
## Get old targets
oldTargets = []
for target in otherLibrary.getTargetDefs():
    oldTargets.append(target.getName())
print('Old targets:', oldTargets)

## Check for new targets
currTargets = []
for target in currLibrary.getTargetDefs():
    currTargets.append(target.getName())
print('Current targets:', currTargets)

newTargets = list(set(currTargets) - set(oldTargets))
if newTargets:
    print('New targets:', newTargets)

Old targets: ['genglsl', 'genmdl', 'genosl']
Current targets: ['essl', 'genglsl', 'genmdl', 'genmsl', 'genosl']
New targets: ['essl', 'genmsl']


### Implementation Check

As with definitions, implementations can be compared, either in the context of their associated definition or independently. As an implementation is generally of no use unless it is used in a definition we will present the comparison logic using the latter context.

#### Listing Implementations

It can be useful to examine the available implementations based on what, if any, code generation `target``
they are associated with. 
* If a implementation has no target then it is generally a node graph implementation but could also be generated programmatically via a common code generator which has no specific code target. For example the <a href ="https://materialx.org/docs/api/class_shader_generator.html" target="_blank">ShaderGenerator</a> class implements non-target specific code generation for <a href="https://github.com/AcademySoftwareFoundation/MaterialX/tree/main/source/MaterialXGenShader/Nodes" target="_blank">certain node types</a>.
* Different targets may choose to implement logic differently than others. For example one may use a nodegraph, while another may use shader code, or generate the code programmatically.
* Some targets may not support implementations for certain nodes.

A utility called `get_implementations()` scans for all implementations for a given `target``.

In [61]:
def get_implementations(nodedefs, target, allimpls):
    '''
    Build a dictionary of nodedefs and their corresponding implementations.
    
    The dictionary is keyed by the nodedef name and the value is a list of the implementation and 
    source file name.
    '''
    implementations = {}
    for nodedef in nodedefs:
        impl = nodedef.getImplementation(target)
        # This may be a nodegraph or an implementation
        if impl:
            # Skip if already used
            if impl.getName() in allimpls:
                continue
            # Skip if it's a nodegraph and target is not empty
            if len(target) > 0 and impl.isA(mx.NodeGraph):
                continue

            if target != 'essl':
                allimpls.add(impl.getName())
            implementations[nodedef.getName()] = [impl, mx.FilePath(impl.getSourceUri()).getBaseName()]
    return implementations

This utility can then be used to scan through all `target`s for each library.

In [62]:
def print_implementations(nodedefs, targets, allimpls, title):
    '''
    Display all the implementations for the given nodedefs and targets.
    '''
    display_markdown(title, raw=True)
    for target in targets:
        impls = get_implementations(nodedefs, target, allimpls)

        if len(target) == 0:
            result = '| Definition | NodeGraph | File |\n'
        else:
            result = '| Definition | Implementation | File |\n'
        result = result + '| --- | --- | --- |\n'
        for impl in impls:
            result = result + '|' + impl + '|' + impls[impl][0].getName() + '|' + impls[impl][1] + '\n'
        text = '<details><summary>' + str(len(impls)) + (' shaders: ' + target if target else ' node graphs') + '</summary>\n\n' + result+ '\n</details>\n' 
        display_markdown(text, raw=True)

# Scan for all the current targets
title = '**Current Library Definitions / Implementations**\n'
alltargets = currTargets
alltargets.append('')
allimpls = set()
print_implementations(currNodeDefs, alltargets, allimpls, title)

# Scan for all the previous targets
title = '**Previous Library Definitions / Implementations**\n'
alltargets = oldTargets
alltargets.append('')
allimpls = set()
print_implementations(otherNodeDefs, alltargets, allimpls, title)

**Current Library Definitions / Implementations**


<details><summary>560 shaders: essl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genglsl|lights_genglsl_impl.mtlx
|ND_directional_light|IM_directional_light_genglsl|lights_genglsl_impl.mtlx
|ND_spot_light|IM_spot_light_genglsl|lights_genglsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surface|IM_surface_genglsl|pbrlib_genglsl_impl.mtlx
|ND_light|IM_light_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_edf|IM_add_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genglsl|pbrlib_genglsl_impl.mtlx
|ND_blackbody|IM_blackbody_genglsl|pbrlib_genglsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genglsl|stdlib_genglsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_float|IM_image_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color3|IM_image_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color4|IM_image_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_float|IM_constant_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_string|IM_constant_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_frame_float|IM_frame_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_time_float|IM_time_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_float|IM_add_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3|IM_add_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4|IM_add_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_float|IM_divide_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_float|IM_invert_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_float|IM_absval_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_float|IM_floor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_float|IM_power_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3|IM_power_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4|IM_power_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_float|IM_sin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_float|IM_cos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_float|IM_tan_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_float|IM_asin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_float|IM_acos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_float|IM_ln_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_float|IM_exp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_float|IM_sign_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_float|IM_min_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3|IM_min_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4|IM_min_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_float|IM_max_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3|IM_max_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4|IM_max_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap|IM_normalmap_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_float|IM_remap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_float|IM_plus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_float|IM_minus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_float|IM_difference_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_float|IM_burn_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_float|IM_screen_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_in_color4|IM_in_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_out_color4|IM_out_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_over_color4|IM_over_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_float|IM_inside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_float|IM_outside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_float|IM_mix_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_float|IM_switch_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_float|IM_blur_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_float|IM_dot_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_string|IM_dot_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genglsl|stdlib_genglsl_impl.mtlx

</details>


<details><summary>560 shaders: genglsl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genglsl|lights_genglsl_impl.mtlx
|ND_directional_light|IM_directional_light_genglsl|lights_genglsl_impl.mtlx
|ND_spot_light|IM_spot_light_genglsl|lights_genglsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surface|IM_surface_genglsl|pbrlib_genglsl_impl.mtlx
|ND_light|IM_light_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genglsl|pbrlib_genglsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_edf|IM_add_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genglsl|pbrlib_genglsl_impl.mtlx
|ND_blackbody|IM_blackbody_genglsl|pbrlib_genglsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genglsl|stdlib_genglsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_float|IM_image_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color3|IM_image_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color4|IM_image_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_float|IM_constant_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_string|IM_constant_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_frame_float|IM_frame_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_time_float|IM_time_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_float|IM_add_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3|IM_add_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4|IM_add_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_float|IM_divide_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_float|IM_invert_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_float|IM_absval_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_float|IM_floor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_float|IM_power_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3|IM_power_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4|IM_power_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_float|IM_sin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_float|IM_cos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_float|IM_tan_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_float|IM_asin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_float|IM_acos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_float|IM_ln_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_float|IM_exp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_float|IM_sign_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_float|IM_min_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3|IM_min_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4|IM_min_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_float|IM_max_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3|IM_max_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4|IM_max_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap|IM_normalmap_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_float|IM_remap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_float|IM_plus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_float|IM_minus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_float|IM_difference_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_float|IM_burn_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_float|IM_screen_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_in_color4|IM_in_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_out_color4|IM_out_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_over_color4|IM_over_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_float|IM_inside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_float|IM_outside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_float|IM_mix_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_float|IM_switch_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_float|IM_blur_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_float|IM_dot_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_string|IM_dot_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genglsl|stdlib_genglsl_impl.mtlx

</details>


<details><summary>569 shaders: genmdl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conical_edf|IM_conical_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_measured_edf|IM_measured_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_generalized_schlick_edf|IM_generalized_schlick_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_absorption_vdf|IM_absorption_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_surface|IM_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_surface|IM_thin_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_volume|IM_volume_genmdl|pbrlib_genmdl_impl.mtlx
|ND_light|IM_light_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genmdl|pbrlib_genmdl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_vdf|IM_mix_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_edf|IM_add_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_vdf|IM_add_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_vdfC|IM_multiply_vdfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_vdfF|IM_multiply_vdfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genmdl|pbrlib_genmdl_impl.mtlx
|ND_blackbody|IM_blackbody_genmdl|pbrlib_genmdl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genmdl|pbrlib_genmdl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genmdl|stdlib_genmdl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_float|IM_image_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color3|IM_image_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color4|IM_image_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_float|IM_constant_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_string|IM_constant_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ambientocclusion_float|IM_ambientocclusion_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_frame_float|IM_frame_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_time_float|IM_time_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_float|IM_add_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3|IM_add_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4|IM_add_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_float|IM_divide_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_float|IM_invert_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_float|IM_absval_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_float|IM_floor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_float|IM_power_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3|IM_power_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4|IM_power_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_float|IM_sin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_float|IM_cos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_float|IM_tan_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_float|IM_asin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_float|IM_acos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_float|IM_ln_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_float|IM_exp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_float|IM_sign_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_float|IM_min_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3|IM_min_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4|IM_min_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_float|IM_max_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3|IM_max_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4|IM_max_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalmap|IM_normalmap_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_float|IM_remap_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_float|IM_plus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_float|IM_minus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_float|IM_difference_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_float|IM_burn_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_float|IM_screen_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_in_color4|IM_in_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_out_color4|IM_out_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_over_color4|IM_over_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_float|IM_inside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_float|IM_outside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_float|IM_mix_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_displacementshader|IM_mix_displacementshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_volumeshader|IM_mix_volumeshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_float|IM_switch_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_float|IM_blur_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_float|IM_dot_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_string|IM_dot_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genmdl|stdlib_genmdl_impl.mtlx

</details>


<details><summary>559 shaders: genmsl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genmsl|lights_genmsl_impl.mtlx
|ND_directional_light|IM_directional_light_genmsl|lights_genmsl_impl.mtlx
|ND_spot_light|IM_spot_light_genmsl|lights_genmsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_surface|IM_surface_genmsl|pbrlib_genmsl_impl.mtlx
|ND_light|IM_light_genmsl|pbrlib_genmsl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genmsl|pbrlib_genmsl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genmsl|pbrlib_genmsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_layer_vdf|IM_layer_vdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_add_edf|IM_add_edf_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genmsl|pbrlib_genmsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genmsl|pbrlib_genmsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genmsl|pbrlib_genmsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genmsl|pbrlib_genmsl_impl.mtlx
|ND_blackbody|IM_blackbody_genmsl|pbrlib_genmsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genmsl|pbrlib_genmsl_impl.mtlx
|ND_surfacematerial|IM_surfacematerial_genmsl|stdlib_genmsl_impl.mtlx
|ND_surface_unlit|IM_surface_unlit_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_float|IM_image_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_color3|IM_image_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_color4|IM_image_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_float|IM_constant_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_string|IM_constant_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_float|IM_worleynoise2d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_vector2|IM_worleynoise2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise2d_vector3|IM_worleynoise2d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_float|IM_worleynoise3d_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_vector2|IM_worleynoise3d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_worleynoise3d_vector3|IM_worleynoise3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_frame_float|IM_frame_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_time_float|IM_time_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_float|IM_add_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color3|IM_add_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color4|IM_add_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_float|IM_divide_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_float|IM_invert_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_float|IM_absval_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_float|IM_floor_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_floor_integer|IM_floor_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ceil_integer|IM_ceil_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_float|IM_power_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color3|IM_power_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color4|IM_power_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_float|IM_sin_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_float|IM_cos_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_float|IM_tan_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_float|IM_asin_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_float|IM_acos_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_float|IM_ln_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_float|IM_exp_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_float|IM_sign_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_float|IM_min_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color3|IM_min_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color4|IM_min_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_float|IM_max_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color3|IM_max_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color4|IM_max_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genmsl|stdlib_genmsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_normalmap|IM_normalmap_genmsl|stdlib_genmsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_float|IM_remap_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genmsl|stdlib_genmsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_float|IM_plus_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_float|IM_minus_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_float|IM_difference_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_float|IM_burn_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_float|IM_screen_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_in_color4|IM_in_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_out_color4|IM_out_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_over_color4|IM_over_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_float|IM_inside_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_float|IM_outside_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_float|IM_mix_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color3_color3|IM_mix_color3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_color4_color4|IM_mix_color4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector2_vector2|IM_mix_vector2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector3_vector3|IM_mix_vector3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_vector4_vector4|IM_mix_vector4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genmsl|stdlib_genmsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_float|IM_switch_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genmsl|stdlib_genmsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_float|IM_blur_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_float|IM_dot_float_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_string|IM_dot_string_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genmsl|stdlib_genmsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genmsl|stdlib_genmsl_impl.mtlx

</details>


...

**Previous Library Definitions / Implementations**


<details><summary>542 shaders: genglsl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_point_light|IM_point_light_genglsl|lights_genglsl_impl.mtlx
|ND_directional_light|IM_directional_light_genglsl|lights_genglsl_impl.mtlx
|ND_spot_light|IM_spot_light_genglsl|lights_genglsl_impl.mtlx
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_surface|IM_surface_genglsl|pbrlib_genglsl_impl.mtlx
|ND_light|IM_light_genglsl|pbrlib_genglsl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_add_edf|IM_add_edf_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genglsl|pbrlib_genglsl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genglsl|pbrlib_genglsl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genglsl|pbrlib_genglsl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genglsl|pbrlib_genglsl_impl.mtlx
|ND_image_float|IM_image_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color3|IM_image_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_color4|IM_image_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_float|IM_constant_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_string|IM_constant_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_frame_float|IM_frame_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_time_float|IM_time_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_float|IM_add_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3|IM_add_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4|IM_add_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_surfaceshader|IM_add_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_surfaceshaderF|IM_multiply_surfaceshaderF_genglsl|stdlib_genglsl_impl.mtlx
|ND_multiply_surfaceshaderC|IM_multiply_surfaceshaderC_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_float|IM_divide_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_float|IM_invert_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_float|IM_absval_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_float|IM_floor_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_float|IM_power_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3|IM_power_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4|IM_power_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_float|IM_sin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_float|IM_cos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_float|IM_tan_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_float|IM_asin_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_float|IM_acos_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_float|IM_ln_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_float|IM_exp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_float|IM_sign_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_float|IM_min_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3|IM_min_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4|IM_min_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_float|IM_max_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3|IM_max_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4|IM_max_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genglsl|stdlib_genglsl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_normalmap|IM_normalmap_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_float|IM_remap_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_float|IM_plus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_float|IM_minus_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_float|IM_difference_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_float|IM_burn_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_float|IM_screen_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_in_color4|IM_in_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_out_color4|IM_out_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_over_color4|IM_over_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_float|IM_inside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_float|IM_outside_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_float|IM_mix_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genglsl|stdlib_genglsl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_float|IM_switch_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genglsl|stdlib_genglsl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_float|IM_blur_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_float|IM_dot_float_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_string|IM_dot_string_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genglsl|stdlib_genglsl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genglsl|stdlib_genglsl_impl.mtlx

</details>


<details><summary>550 shaders: genmdl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_conical_edf|IM_conical_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_measured_edf|IM_measured_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_absorption_vdf|IM_absorption_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_anisotropic_vdf|IM_anisotropic_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_surface|IM_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_thin_surface|IM_thin_surface_genmdl|pbrlib_genmdl_impl.mtlx
|ND_volume|IM_volume_genmdl|pbrlib_genmdl_impl.mtlx
|ND_light|IM_light_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genmdl|pbrlib_genmdl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genmdl|pbrlib_genmdl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_mix_vdf|IM_mix_vdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_add_edf|IM_add_edf_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genmdl|pbrlib_genmdl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genmdl|pbrlib_genmdl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genmdl|pbrlib_genmdl_impl.mtlx
|ND_blackbody|IM_blackbody_genmdl|pbrlib_genmdl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genmdl|pbrlib_genmdl_impl.mtlx
|ND_image_float|IM_image_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color3|IM_image_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_color4|IM_image_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_float|IM_constant_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_string|IM_constant_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ambientocclusion_float|IM_ambientocclusion_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_frame_float|IM_frame_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_time_float|IM_time_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_float|IM_add_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3|IM_add_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4|IM_add_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_surfaceshader|IM_add_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_surfaceshaderF|IM_multiply_surfaceshaderF_genmdl|stdlib_genmdl_impl.mtlx
|ND_multiply_surfaceshaderC|IM_multiply_surfaceshaderC_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_float|IM_divide_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_float|IM_invert_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_float|IM_absval_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_float|IM_floor_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_float|IM_power_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3|IM_power_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4|IM_power_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_float|IM_sin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_float|IM_cos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_float|IM_tan_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_float|IM_asin_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_float|IM_acos_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_float|IM_ln_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_float|IM_exp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_float|IM_sign_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_float|IM_min_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3|IM_min_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4|IM_min_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_float|IM_max_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3|IM_max_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4|IM_max_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genmdl|stdlib_genmdl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_normalmap|IM_normalmap_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_float|IM_remap_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_float|IM_plus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_float|IM_minus_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_float|IM_difference_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_float|IM_burn_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_float|IM_screen_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_in_color4|IM_in_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_out_color4|IM_out_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_over_color4|IM_over_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_float|IM_inside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_float|IM_outside_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_float|IM_mix_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genmdl|stdlib_genmdl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_float|IM_switch_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genmdl|stdlib_genmdl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_float|IM_blur_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_float|IM_dot_float_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_string|IM_dot_string_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genmdl|stdlib_genmdl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genmdl|stdlib_genmdl_impl.mtlx

</details>


<details><summary>541 shaders: genosl</summary>

| Definition | Implementation | File |
| --- | --- | --- |
|ND_oren_nayar_diffuse_bsdf|IM_oren_nayar_diffuse_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_burley_diffuse_bsdf|IM_burley_diffuse_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_translucent_bsdf|IM_translucent_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_dielectric_bsdf|IM_dielectric_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_conductor_bsdf|IM_conductor_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_generalized_schlick_bsdf|IM_generalized_schlick_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_subsurface_bsdf|IM_subsurface_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_sheen_bsdf|IM_sheen_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_thin_film_bsdf|IM_thin_film_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_uniform_edf|IM_uniform_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_surface|IM_surface_genosl|pbrlib_genosl_impl.mtlx
|ND_displacement_float|IM_displacement_float_genosl|pbrlib_genosl_impl.mtlx
|ND_displacement_vector3|IM_displacement_vector3_genosl|pbrlib_genosl_impl.mtlx
|ND_layer_bsdf|IM_layer_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_mix_bsdf|IM_mix_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_mix_edf|IM_mix_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_add_bsdf|IM_add_bsdf_genosl|pbrlib_genosl_impl.mtlx
|ND_add_edf|IM_add_edf_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_bsdfC|IM_multiply_bsdfC_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_bsdfF|IM_multiply_bsdfF_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_edfC|IM_multiply_edfC_genosl|pbrlib_genosl_impl.mtlx
|ND_multiply_edfF|IM_multiply_edfF_genosl|pbrlib_genosl_impl.mtlx
|ND_roughness_anisotropy|IM_roughness_anisotropy_genosl|pbrlib_genosl_impl.mtlx
|ND_roughness_dual|IM_roughness_dual_genosl|pbrlib_genosl_impl.mtlx
|ND_artistic_ior|IM_artistic_ior_genosl|pbrlib_genosl_impl.mtlx
|ND_image_float|IM_image_float_genosl|stdlib_genosl_impl.mtlx
|ND_image_color3|IM_image_color3_genosl|stdlib_genosl_impl.mtlx
|ND_image_color4|IM_image_color4_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector2|IM_image_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector3|IM_image_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_image_vector4|IM_image_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_float|IM_constant_float_genosl|stdlib_genosl_impl.mtlx
|ND_constant_color3|IM_constant_color3_genosl|stdlib_genosl_impl.mtlx
|ND_constant_color4|IM_constant_color4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector2|IM_constant_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector3|IM_constant_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_constant_vector4|IM_constant_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_constant_boolean|IM_constant_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_constant_integer|IM_constant_integer_genosl|stdlib_genosl_impl.mtlx
|ND_constant_matrix33|IM_constant_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_constant_matrix44|IM_constant_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_constant_string|IM_constant_string_genosl|stdlib_genosl_impl.mtlx
|ND_constant_filename|IM_constant_filename_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_float|IM_ramplr_float_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_color3|IM_ramplr_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_color4|IM_ramplr_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector2|IM_ramplr_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector3|IM_ramplr_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ramplr_vector4|IM_ramplr_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_float|IM_ramptb_float_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_color3|IM_ramptb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_color4|IM_ramptb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector2|IM_ramptb_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector3|IM_ramptb_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ramptb_vector4|IM_ramptb_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_float|IM_splitlr_float_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_color3|IM_splitlr_color3_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_color4|IM_splitlr_color4_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector2|IM_splitlr_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector3|IM_splitlr_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_splitlr_vector4|IM_splitlr_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_float|IM_splittb_float_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_color3|IM_splittb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_color4|IM_splittb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector2|IM_splittb_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector3|IM_splittb_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_splittb_vector4|IM_splittb_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_float|IM_noise2d_float_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_color3|IM_noise2d_color3_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_color4|IM_noise2d_color4_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector2|IM_noise2d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector3|IM_noise2d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector4|IM_noise2d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_color3FA|IM_noise2d_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_color4FA|IM_noise2d_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector2FA|IM_noise2d_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector3FA|IM_noise2d_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise2d_vector4FA|IM_noise2d_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_float|IM_noise3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_color3|IM_noise3d_color3_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_color4|IM_noise3d_color4_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector2|IM_noise3d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector3|IM_noise3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector4|IM_noise3d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_color3FA|IM_noise3d_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_color4FA|IM_noise3d_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector2FA|IM_noise3d_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector3FA|IM_noise3d_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_noise3d_vector4FA|IM_noise3d_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_float|IM_fractal3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_color3|IM_fractal3d_color3_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_color4|IM_fractal3d_color4_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector2|IM_fractal3d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector3|IM_fractal3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector4|IM_fractal3d_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_color3FA|IM_fractal3d_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_color4FA|IM_fractal3d_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector2FA|IM_fractal3d_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector3FA|IM_fractal3d_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_fractal3d_vector4FA|IM_fractal3d_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_cellnoise2d_float|IM_cellnoise2d_float_genosl|stdlib_genosl_impl.mtlx
|ND_cellnoise3d_float|IM_cellnoise3d_float_genosl|stdlib_genosl_impl.mtlx
|ND_position_vector3|IM_position_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_normal_vector3|IM_normal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_tangent_vector3|IM_tangent_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_bitangent_vector3|IM_bitangent_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_texcoord_vector2|IM_texcoord_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_texcoord_vector3|IM_texcoord_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_float|IM_geomcolor_float_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_color3|IM_geomcolor_color3_genosl|stdlib_genosl_impl.mtlx
|ND_geomcolor_color4|IM_geomcolor_color4_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_integer|IM_geompropvalue_integer_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_boolean|IM_geompropvalue_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_string|IM_geompropvalue_string_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_float|IM_geompropvalue_float_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_color3|IM_geompropvalue_color3_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_color4|IM_geompropvalue_color4_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector2|IM_geompropvalue_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector3|IM_geompropvalue_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_geompropvalue_vector4|IM_geompropvalue_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ambientocclusion_float|IM_ambientocclusion_float_genosl|stdlib_genosl_impl.mtlx
|ND_frame_float|IM_frame_float_genosl|stdlib_genosl_impl.mtlx
|ND_time_float|IM_time_float_genosl|stdlib_genosl_impl.mtlx
|ND_add_float|IM_add_float_genosl|stdlib_genosl_impl.mtlx
|ND_add_color3|IM_add_color3_genosl|stdlib_genosl_impl.mtlx
|ND_add_color4|IM_add_color4_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector2|IM_add_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector3|IM_add_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector4|IM_add_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix33|IM_add_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix44|IM_add_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_add_surfaceshader|IM_add_surfaceshader_genosl|stdlib_genosl_impl.mtlx
|ND_add_color3FA|IM_add_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_color4FA|IM_add_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector2FA|IM_add_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector3FA|IM_add_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_vector4FA|IM_add_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix33FA|IM_add_matrix33FA_genosl|stdlib_genosl_impl.mtlx
|ND_add_matrix44FA|IM_add_matrix44FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_float|IM_subtract_float_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color3|IM_subtract_color3_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color4|IM_subtract_color4_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector2|IM_subtract_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector3|IM_subtract_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector4|IM_subtract_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix33|IM_subtract_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix44|IM_subtract_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color3FA|IM_subtract_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_color4FA|IM_subtract_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector2FA|IM_subtract_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector3FA|IM_subtract_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_vector4FA|IM_subtract_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix33FA|IM_subtract_matrix33FA_genosl|stdlib_genosl_impl.mtlx
|ND_subtract_matrix44FA|IM_subtract_matrix44FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_float|IM_multiply_float_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color3|IM_multiply_color3_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color4|IM_multiply_color4_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector2|IM_multiply_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector3|IM_multiply_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector4|IM_multiply_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_matrix33|IM_multiply_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_matrix44|IM_multiply_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color3FA|IM_multiply_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_color4FA|IM_multiply_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector2FA|IM_multiply_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector3FA|IM_multiply_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_vector4FA|IM_multiply_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_surfaceshaderF|IM_multiply_surfaceshaderF_genosl|stdlib_genosl_impl.mtlx
|ND_multiply_surfaceshaderC|IM_multiply_surfaceshaderC_genosl|stdlib_genosl_impl.mtlx
|ND_divide_float|IM_divide_float_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color3|IM_divide_color3_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color4|IM_divide_color4_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector2|IM_divide_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector3|IM_divide_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector4|IM_divide_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_divide_matrix33|IM_divide_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_divide_matrix44|IM_divide_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color3FA|IM_divide_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_color4FA|IM_divide_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector2FA|IM_divide_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector3FA|IM_divide_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_divide_vector4FA|IM_divide_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_float|IM_modulo_float_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color3|IM_modulo_color3_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color4|IM_modulo_color4_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector2|IM_modulo_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector3|IM_modulo_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector4|IM_modulo_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color3FA|IM_modulo_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_color4FA|IM_modulo_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector2FA|IM_modulo_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector3FA|IM_modulo_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_modulo_vector4FA|IM_modulo_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_float|IM_invert_float_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color3|IM_invert_color3_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color4|IM_invert_color4_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector2|IM_invert_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector3|IM_invert_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector4|IM_invert_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color3FA|IM_invert_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_color4FA|IM_invert_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector2FA|IM_invert_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector3FA|IM_invert_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_invert_vector4FA|IM_invert_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_absval_float|IM_absval_float_genosl|stdlib_genosl_impl.mtlx
|ND_absval_color3|IM_absval_color3_genosl|stdlib_genosl_impl.mtlx
|ND_absval_color4|IM_absval_color4_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector2|IM_absval_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector3|IM_absval_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_absval_vector4|IM_absval_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_floor_float|IM_floor_float_genosl|stdlib_genosl_impl.mtlx
|ND_floor_color3|IM_floor_color3_genosl|stdlib_genosl_impl.mtlx
|ND_floor_color4|IM_floor_color4_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector2|IM_floor_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector3|IM_floor_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_floor_vector4|IM_floor_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_float|IM_ceil_float_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_color3|IM_ceil_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_color4|IM_ceil_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector2|IM_ceil_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector3|IM_ceil_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ceil_vector4|IM_ceil_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_power_float|IM_power_float_genosl|stdlib_genosl_impl.mtlx
|ND_power_color3|IM_power_color3_genosl|stdlib_genosl_impl.mtlx
|ND_power_color4|IM_power_color4_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector2|IM_power_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector3|IM_power_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector4|IM_power_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_power_color3FA|IM_power_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_color4FA|IM_power_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector2FA|IM_power_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector3FA|IM_power_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_power_vector4FA|IM_power_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_sin_float|IM_sin_float_genosl|stdlib_genosl_impl.mtlx
|ND_cos_float|IM_cos_float_genosl|stdlib_genosl_impl.mtlx
|ND_tan_float|IM_tan_float_genosl|stdlib_genosl_impl.mtlx
|ND_asin_float|IM_asin_float_genosl|stdlib_genosl_impl.mtlx
|ND_acos_float|IM_acos_float_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_float|IM_atan2_float_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector2|IM_sin_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector2|IM_cos_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector2|IM_tan_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector2|IM_asin_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector2|IM_acos_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector2|IM_atan2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector3|IM_sin_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector3|IM_cos_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector3|IM_tan_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector3|IM_asin_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector3|IM_acos_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector3|IM_atan2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sin_vector4|IM_sin_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_cos_vector4|IM_cos_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_tan_vector4|IM_tan_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_asin_vector4|IM_asin_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_acos_vector4|IM_acos_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_atan2_vector4|IM_atan2_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_float|IM_sqrt_float_genosl|stdlib_genosl_impl.mtlx
|ND_ln_float|IM_ln_float_genosl|stdlib_genosl_impl.mtlx
|ND_exp_float|IM_exp_float_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector2|IM_sqrt_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector2|IM_ln_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector2|IM_exp_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector3|IM_sqrt_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector3|IM_ln_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector3|IM_exp_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sqrt_vector4|IM_sqrt_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ln_vector4|IM_ln_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_exp_vector4|IM_exp_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_sign_float|IM_sign_float_genosl|stdlib_genosl_impl.mtlx
|ND_sign_color3|IM_sign_color3_genosl|stdlib_genosl_impl.mtlx
|ND_sign_color4|IM_sign_color4_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector2|IM_sign_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector3|IM_sign_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_sign_vector4|IM_sign_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_float|IM_clamp_float_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color3|IM_clamp_color3_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color4|IM_clamp_color4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector2|IM_clamp_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector3|IM_clamp_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector4|IM_clamp_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color3FA|IM_clamp_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_color4FA|IM_clamp_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector2FA|IM_clamp_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector3FA|IM_clamp_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_clamp_vector4FA|IM_clamp_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_float|IM_min_float_genosl|stdlib_genosl_impl.mtlx
|ND_min_color3|IM_min_color3_genosl|stdlib_genosl_impl.mtlx
|ND_min_color4|IM_min_color4_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector2|IM_min_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector3|IM_min_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector4|IM_min_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_min_color3FA|IM_min_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_color4FA|IM_min_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector2FA|IM_min_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector3FA|IM_min_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_min_vector4FA|IM_min_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_float|IM_max_float_genosl|stdlib_genosl_impl.mtlx
|ND_max_color3|IM_max_color3_genosl|stdlib_genosl_impl.mtlx
|ND_max_color4|IM_max_color4_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector2|IM_max_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector3|IM_max_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector4|IM_max_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_max_color3FA|IM_max_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_color4FA|IM_max_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector2FA|IM_max_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector3FA|IM_max_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_max_vector4FA|IM_max_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector2|IM_normalize_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector3|IM_normalize_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_normalize_vector4|IM_normalize_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector2|IM_magnitude_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector3|IM_magnitude_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_magnitude_vector4|IM_magnitude_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector2|IM_dotproduct_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector3|IM_dotproduct_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dotproduct_vector4|IM_dotproduct_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_crossproduct_vector3|IM_crossproduct_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformpoint_vector3|IM_transformpoint_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformvector_vector3|IM_transformvector_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformnormal_vector3|IM_transformnormal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector2M3|IM_transformmatrix_vector2M3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector3|IM_transformmatrix_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector3M4|IM_transformmatrix_vector3M4_genosl|stdlib_genosl_impl.mtlx
|ND_transformmatrix_vector4|IM_transformmatrix_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_normalmap|IM_normalmap_genosl|stdlib_genosl_impl.mtlx
|ND_transpose_matrix33|IM_transpose_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_transpose_matrix44|IM_transpose_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_determinant_matrix33|IM_determinant_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_determinant_matrix44|IM_determinant_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_invertmatrix_matrix33|IM_invertmatrix_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_invertmatrix_matrix44|IM_invertmatrix_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_rotate2d_vector2|IM_rotate2d_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_rotate3d_vector3|IM_rotate3d_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_float|IM_remap_float_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color3|IM_remap_color3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color4|IM_remap_color4_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector2|IM_remap_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector3|IM_remap_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector4|IM_remap_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color3FA|IM_remap_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_color4FA|IM_remap_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector2FA|IM_remap_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector3FA|IM_remap_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_remap_vector4FA|IM_remap_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_float|IM_smoothstep_float_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_color3|IM_smoothstep_color3_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_color4|IM_smoothstep_color4_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector2|IM_smoothstep_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector3|IM_smoothstep_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector4|IM_smoothstep_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_color3FA|IM_smoothstep_color3FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_color4FA|IM_smoothstep_color4FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector2FA|IM_smoothstep_vector2FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector3FA|IM_smoothstep_vector3FA_genosl|stdlib_genosl_impl.mtlx
|ND_smoothstep_vector4FA|IM_smoothstep_vector4FA_genosl|stdlib_genosl_impl.mtlx
|ND_luminance_color3|IM_luminance_color3_genosl|stdlib_genosl_impl.mtlx
|ND_luminance_color4|IM_luminance_color4_genosl|stdlib_genosl_impl.mtlx
|ND_rgbtohsv_color3|IM_rgbtohsv_color3_genosl|stdlib_genosl_impl.mtlx
|ND_rgbtohsv_color4|IM_rgbtohsv_color4_genosl|stdlib_genosl_impl.mtlx
|ND_hsvtorgb_color3|IM_hsvtorgb_color3_genosl|stdlib_genosl_impl.mtlx
|ND_hsvtorgb_color4|IM_hsvtorgb_color4_genosl|stdlib_genosl_impl.mtlx
|ND_premult_color4|IM_premult_color4_genosl|stdlib_genosl_impl.mtlx
|ND_unpremult_color4|IM_unpremult_color4_genosl|stdlib_genosl_impl.mtlx
|ND_plus_float|IM_plus_float_genosl|stdlib_genosl_impl.mtlx
|ND_plus_color3|IM_plus_color3_genosl|stdlib_genosl_impl.mtlx
|ND_plus_color4|IM_plus_color4_genosl|stdlib_genosl_impl.mtlx
|ND_minus_float|IM_minus_float_genosl|stdlib_genosl_impl.mtlx
|ND_minus_color3|IM_minus_color3_genosl|stdlib_genosl_impl.mtlx
|ND_minus_color4|IM_minus_color4_genosl|stdlib_genosl_impl.mtlx
|ND_difference_float|IM_difference_float_genosl|stdlib_genosl_impl.mtlx
|ND_difference_color3|IM_difference_color3_genosl|stdlib_genosl_impl.mtlx
|ND_difference_color4|IM_difference_color4_genosl|stdlib_genosl_impl.mtlx
|ND_burn_float|IM_burn_float_genosl|stdlib_genosl_impl.mtlx
|ND_burn_color3|IM_burn_color3_genosl|stdlib_genosl_impl.mtlx
|ND_burn_color4|IM_burn_color4_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_float|IM_dodge_float_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_color3|IM_dodge_color3_genosl|stdlib_genosl_impl.mtlx
|ND_dodge_color4|IM_dodge_color4_genosl|stdlib_genosl_impl.mtlx
|ND_screen_float|IM_screen_float_genosl|stdlib_genosl_impl.mtlx
|ND_screen_color3|IM_screen_color3_genosl|stdlib_genosl_impl.mtlx
|ND_screen_color4|IM_screen_color4_genosl|stdlib_genosl_impl.mtlx
|ND_overlay_float|IM_overlay_float_genosl|stdlib_genosl_impl.mtlx
|ND_overlay_color3|IM_overlay_color3_genosl|stdlib_genosl_impl.mtlx
|ND_overlay_color4|IM_overlay_color4_genosl|stdlib_genosl_impl.mtlx
|ND_disjointover_color4|IM_disjointover_color4_genosl|stdlib_genosl_impl.mtlx
|ND_in_color4|IM_in_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mask_color4|IM_mask_color4_genosl|stdlib_genosl_impl.mtlx
|ND_matte_color4|IM_matte_color4_genosl|stdlib_genosl_impl.mtlx
|ND_out_color4|IM_out_color4_genosl|stdlib_genosl_impl.mtlx
|ND_over_color4|IM_over_color4_genosl|stdlib_genosl_impl.mtlx
|ND_inside_float|IM_inside_float_genosl|stdlib_genosl_impl.mtlx
|ND_inside_color3|IM_inside_color3_genosl|stdlib_genosl_impl.mtlx
|ND_inside_color4|IM_inside_color4_genosl|stdlib_genosl_impl.mtlx
|ND_outside_float|IM_outside_float_genosl|stdlib_genosl_impl.mtlx
|ND_outside_color3|IM_outside_color3_genosl|stdlib_genosl_impl.mtlx
|ND_outside_color4|IM_outside_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_float|IM_mix_float_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color3|IM_mix_color3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_color4|IM_mix_color4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector2|IM_mix_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector3|IM_mix_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_mix_vector4|IM_mix_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_mix_surfaceshader|IM_mix_surfaceshader_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_float|IM_ifgreater_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color3|IM_ifgreater_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color4|IM_ifgreater_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector2|IM_ifgreater_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector3|IM_ifgreater_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector4|IM_ifgreater_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_floatI|IM_ifgreater_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color3I|IM_ifgreater_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_color4I|IM_ifgreater_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector2I|IM_ifgreater_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector3I|IM_ifgreater_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreater_vector4I|IM_ifgreater_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_float|IM_ifgreatereq_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color3|IM_ifgreatereq_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color4|IM_ifgreatereq_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector2|IM_ifgreatereq_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector3|IM_ifgreatereq_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector4|IM_ifgreatereq_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_floatI|IM_ifgreatereq_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color3I|IM_ifgreatereq_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_color4I|IM_ifgreatereq_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector2I|IM_ifgreatereq_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector3I|IM_ifgreatereq_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifgreatereq_vector4I|IM_ifgreatereq_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_float|IM_ifequal_float_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3|IM_ifequal_color3_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4|IM_ifequal_color4_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2|IM_ifequal_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3|IM_ifequal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4|IM_ifequal_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_floatI|IM_ifequal_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3I|IM_ifequal_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4I|IM_ifequal_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2I|IM_ifequal_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3I|IM_ifequal_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4I|IM_ifequal_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_floatB|IM_ifequal_floatB_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color3B|IM_ifequal_color3B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_color4B|IM_ifequal_color4B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector2B|IM_ifequal_vector2B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector3B|IM_ifequal_vector3B_genosl|stdlib_genosl_impl.mtlx
|ND_ifequal_vector4B|IM_ifequal_vector4B_genosl|stdlib_genosl_impl.mtlx
|ND_switch_float|IM_switch_float_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color3|IM_switch_color3_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color4|IM_switch_color4_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector2|IM_switch_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector3|IM_switch_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector4|IM_switch_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_switch_floatI|IM_switch_floatI_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color3I|IM_switch_color3I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_color4I|IM_switch_color4I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector2I|IM_switch_vector2I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector3I|IM_switch_vector3I_genosl|stdlib_genosl_impl.mtlx
|ND_switch_vector4I|IM_switch_vector4I_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_color3|IM_convert_float_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_color4|IM_convert_float_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector2|IM_convert_float_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector3|IM_convert_float_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_float_vector4|IM_convert_float_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector2_vector3|IM_convert_vector2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_color3|IM_convert_vector3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_vector2|IM_convert_vector3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector3_vector4|IM_convert_vector3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector4_color4|IM_convert_vector4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_vector4_vector3|IM_convert_vector4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color3_vector3|IM_convert_color3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color4_vector4|IM_convert_color4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color3_color4|IM_convert_color3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_convert_color4_color3|IM_convert_color4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_convert_boolean_float|IM_convert_boolean_float_genosl|stdlib_genosl_impl.mtlx
|ND_convert_integer_float|IM_convert_integer_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_color3|IM_swizzle_float_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_color4|IM_swizzle_float_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector2|IM_swizzle_float_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector3|IM_swizzle_float_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_float_vector4|IM_swizzle_float_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_float|IM_swizzle_color3_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_color3|IM_swizzle_color3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_color4|IM_swizzle_color3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector2|IM_swizzle_color3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector3|IM_swizzle_color3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color3_vector4|IM_swizzle_color3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_float|IM_swizzle_color4_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_color3|IM_swizzle_color4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_color4|IM_swizzle_color4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector2|IM_swizzle_color4_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector3|IM_swizzle_color4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_color4_vector4|IM_swizzle_color4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_float|IM_swizzle_vector2_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_color3|IM_swizzle_vector2_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_color4|IM_swizzle_vector2_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector2|IM_swizzle_vector2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector3|IM_swizzle_vector2_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector2_vector4|IM_swizzle_vector2_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_float|IM_swizzle_vector3_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_color3|IM_swizzle_vector3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_color4|IM_swizzle_vector3_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector2|IM_swizzle_vector3_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector3|IM_swizzle_vector3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector3_vector4|IM_swizzle_vector3_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_float|IM_swizzle_vector4_float_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_color3|IM_swizzle_vector4_color3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_color4|IM_swizzle_vector4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector2|IM_swizzle_vector4_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector3|IM_swizzle_vector4_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_swizzle_vector4_vector4|IM_swizzle_vector4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector2|IM_combine2_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_color4CF|IM_combine2_color4CF_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector4VF|IM_combine2_vector4VF_genosl|stdlib_genosl_impl.mtlx
|ND_combine2_vector4VV|IM_combine2_vector4VV_genosl|stdlib_genosl_impl.mtlx
|ND_combine3_color3|IM_combine3_color3_genosl|stdlib_genosl_impl.mtlx
|ND_combine3_vector3|IM_combine3_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_combine4_color4|IM_combine4_color4_genosl|stdlib_genosl_impl.mtlx
|ND_combine4_vector4|IM_combine4_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_blur_float|IM_blur_float_genosl|stdlib_genosl_impl.mtlx
|ND_blur_color3|IM_blur_color3_genosl|stdlib_genosl_impl.mtlx
|ND_blur_color4|IM_blur_color4_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector2|IM_blur_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector3|IM_blur_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_blur_vector4|IM_blur_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_heighttonormal_vector3|IM_heighttonormal_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_float|IM_dot_float_genosl|stdlib_genosl_impl.mtlx
|ND_dot_color3|IM_dot_color3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_color4|IM_dot_color4_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector2|IM_dot_vector2_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector3|IM_dot_vector3_genosl|stdlib_genosl_impl.mtlx
|ND_dot_vector4|IM_dot_vector4_genosl|stdlib_genosl_impl.mtlx
|ND_dot_boolean|IM_dot_boolean_genosl|stdlib_genosl_impl.mtlx
|ND_dot_integer|IM_dot_integer_genosl|stdlib_genosl_impl.mtlx
|ND_dot_matrix33|IM_dot_matrix33_genosl|stdlib_genosl_impl.mtlx
|ND_dot_matrix44|IM_dot_matrix44_genosl|stdlib_genosl_impl.mtlx
|ND_dot_string|IM_dot_string_genosl|stdlib_genosl_impl.mtlx
|ND_dot_filename|IM_dot_filename_genosl|stdlib_genosl_impl.mtlx
|ND_dot_surfaceshader|IM_dot_surfaceshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_displacementshader|IM_dot_displacementshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_volumeshader|IM_dot_volumeshader_genosl|stdlib_genosl_impl.mtlx
|ND_dot_lightshader|IM_dot_lightshader_genosl|stdlib_genosl_impl.mtlx

</details>


<details><summary>95 node graphs</summary>

| Definition | NodeGraph | File |
| --- | --- | --- |
|ND_disney_brdf_2012_surface|IM_disney_brdf_2012_surface_brdf_explorer|disney_brdf_2012.mtlx
|ND_disney_bsdf_2015_surface|IM_disney_bsdf_2015_surface_pbrt|disney_brdf_2015.mtlx
|ND_standard_surface_surfaceshader|IMPL_standard_surface_surfaceshader|standard_surface.mtlx
|ND_UsdPreviewSurface_surfaceshader|IMP_UsdPreviewSurface_surfaceshader|usd_preview_surface.mtlx
|ND_UsdUVTexture|IMP_UsdUVTexture|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_integer|IMP_UsdPrimvarReader_integer|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_boolean|IMP_UsdPrimvarReader_boolean|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_string|IMP_UsdPrimvarReader_string|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_float|IMP_UsdPrimvarReader_float|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector2|IMP_UsdPrimvarReader_vector2|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector3|IMP_UsdPrimvarReader_vector3|usd_preview_surface.mtlx
|ND_UsdPrimvarReader_vector4|IMP_UsdPrimvarReader_vector4|usd_preview_surface.mtlx
|ND_UsdTransform2d|IMP_UsdTransform2d|usd_preview_surface.mtlx
|ND_glossiness_anisotropy|IMP_glossiness_anisotropy|pbrlib_ng.mtlx
|ND_tiledimage_float|NG_tiledimage_float|stdlib_ng.mtlx
|ND_tiledimage_color3|NG_tiledimage_color3|stdlib_ng.mtlx
|ND_tiledimage_color4|NG_tiledimage_color4|stdlib_ng.mtlx
|ND_tiledimage_vector2|NG_tiledimage_vector2|stdlib_ng.mtlx
|ND_tiledimage_vector3|NG_tiledimage_vector3|stdlib_ng.mtlx
|ND_tiledimage_vector4|NG_tiledimage_vector4|stdlib_ng.mtlx
|ND_triplanarprojection_float|NG_triplanarprojection_float|stdlib_ng.mtlx
|ND_triplanarprojection_color3|NG_triplanarprojection_color3|stdlib_ng.mtlx
|ND_triplanarprojection_color4|NG_triplanarprojection_color4|stdlib_ng.mtlx
|ND_triplanarprojection_vector2|NG_triplanarprojection_vector2|stdlib_ng.mtlx
|ND_triplanarprojection_vector3|NG_triplanarprojection_vector3|stdlib_ng.mtlx
|ND_triplanarprojection_vector4|NG_triplanarprojection_vector4|stdlib_ng.mtlx
|ND_ramp4_float|NG_ramp4_float|stdlib_ng.mtlx
|ND_ramp4_color3|NG_ramp4_color3|stdlib_ng.mtlx
|ND_ramp4_color4|NG_ramp4_color4|stdlib_ng.mtlx
|ND_ramp4_vector2|NG_ramp4_vector2|stdlib_ng.mtlx
|ND_ramp4_vector3|NG_ramp4_vector3|stdlib_ng.mtlx
|ND_ramp4_vector4|NG_ramp4_vector4|stdlib_ng.mtlx
|ND_add_displacementshader|IM_add_displacementshader_osl|stdlib_osl_impl.mtlx
|ND_add_volumeshader|IM_add_volumeshader_osl|stdlib_osl_impl.mtlx
|ND_place2d_vector2|NG_place2d_vector2|stdlib_ng.mtlx
|ND_arrayappend_integer_integerarray|IM_arrayappend_integer_integerarray_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_integerarray_integerarray|IM_arrayappend_integerarray_integerarray_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_float_floatarray|IM_arrayappend_float_floatarray_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_floatarray_floatarray|IM_arrayappend_floatarray_floatarray_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_color3_color3array|IM_arrayappend_color3_color3array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_color3array_color3array|IM_arrayappend_color3array_color3array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_color4_color4array|IM_arrayappend_color4_color4array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_color4array_color4array|IM_arrayappend_color4array_color4array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector2_vector2array|IM_arrayappend_vector2_vector2array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector2array_vector2array|IM_arrayappend_vector2array_vector2array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector3_vector3array|IM_arrayappend_vector3_vector3array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector3array_vector3array|IM_arrayappend_vector3array_vector3array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector4_vector4array|IM_arrayappend_vector4_vector4array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_vector4array_vector4array|IM_arrayappend_vector4array_vector4array_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_string_stringarray|IM_arrayappend_string_stringarray_osl|stdlib_osl_impl.mtlx
|ND_arrayappend_stringarray_stringarray|IM_arrayappend_stringarray_stringarray_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_float|IM_curveadjust_float_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_color3|IM_curveadjust_color3_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_color4|IM_curveadjust_color4_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_vector2|IM_curveadjust_vector2_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_vector3|IM_curveadjust_vector3_osl|stdlib_osl_impl.mtlx
|ND_curveadjust_vector4|IM_curveadjust_vector4_osl|stdlib_osl_impl.mtlx
|ND_contrast_float|NG_contrast_float|stdlib_ng.mtlx
|ND_contrast_color3|NG_contrast_color3|stdlib_ng.mtlx
|ND_contrast_color4|NG_contrast_color4|stdlib_ng.mtlx
|ND_contrast_vector2|NG_contrast_vector2|stdlib_ng.mtlx
|ND_contrast_vector3|NG_contrast_vector3|stdlib_ng.mtlx
|ND_contrast_vector4|NG_contrast_vector4|stdlib_ng.mtlx
|ND_contrast_color3FA|NG_contrast_color3FA|stdlib_ng.mtlx
|ND_contrast_color4FA|NG_contrast_color4FA|stdlib_ng.mtlx
|ND_contrast_vector2FA|NG_contrast_vector2FA|stdlib_ng.mtlx
|ND_contrast_vector3FA|NG_contrast_vector3FA|stdlib_ng.mtlx
|ND_contrast_vector4FA|NG_contrast_vector4FA|stdlib_ng.mtlx
|ND_range_float|NG_range_float|stdlib_ng.mtlx
|ND_range_color3|NG_range_color3|stdlib_ng.mtlx
|ND_range_color4|NG_range_color4|stdlib_ng.mtlx
|ND_range_vector2|NG_range_vector2|stdlib_ng.mtlx
|ND_range_vector3|NG_range_vector3|stdlib_ng.mtlx
|ND_range_vector4|NG_range_vector4|stdlib_ng.mtlx
|ND_range_color3FA|NG_range_color3FA|stdlib_ng.mtlx
|ND_range_color4FA|NG_range_color4FA|stdlib_ng.mtlx
|ND_range_vector2FA|NG_range_vector2FA|stdlib_ng.mtlx
|ND_range_vector3FA|NG_range_vector3FA|stdlib_ng.mtlx
|ND_range_vector4FA|NG_range_vector4FA|stdlib_ng.mtlx
|ND_hsvadjust_color3|NG_hsvadjust_color3|stdlib_ng.mtlx
|ND_hsvadjust_color4|NG_hsvadjust_color4|stdlib_ng.mtlx
|ND_saturate_color3|NG_saturate_color3|stdlib_ng.mtlx
|ND_saturate_color4|NG_saturate_color4|stdlib_ng.mtlx
|ND_mix_displacementshader|IM_mix_displacementshader_osl|stdlib_osl_impl.mtlx
|ND_mix_volumeshader|IM_mix_volumeshader_osl|stdlib_osl_impl.mtlx
|ND_extract_color3|NG_extract_color3|stdlib_ng.mtlx
|ND_extract_color4|NG_extract_color4|stdlib_ng.mtlx
|ND_extract_vector2|NG_extract_vector2|stdlib_ng.mtlx
|ND_extract_vector3|NG_extract_vector3|stdlib_ng.mtlx
|ND_extract_vector4|NG_extract_vector4|stdlib_ng.mtlx
|ND_separate2_vector2|NG_separate2_vector2|stdlib_ng.mtlx
|ND_separate3_color3|NG_separate3_color3|stdlib_ng.mtlx
|ND_separate3_vector3|NG_separate3_vector3|stdlib_ng.mtlx
|ND_separate4_color4|NG_separate4_color4|stdlib_ng.mtlx
|ND_separate4_vector4|NG_separate4_vector4|stdlib_ng.mtlx

</details>


We can see from the results, that 2 new targets were introduced as well as a number of new implementations written as source code and node graphs. The discrepancy, in the source code counts gives a hint as to the extent support for certain node definitions. That is, implementations for a given target may not be available for certain nodes. An example is `ambientocclusion` which is not supported by hardware shader targets at time of
writing.

#### Comparing Implementations

We will first check the number of implementations which are associated with a definition in each library.
We use the `getNodeDef()` to skip any unassociated implementations.
 
We must check for implementations which are either:
1. Source implementations using <a href="https://materialx.org/docs/api/class_document.html" target="_blank">
getImplementations()</a> but also check explicity for:
2. Node graphs using  <a href="https://materialx.org/docs/api/class_document.html" target="_blank">
getNodeGraphs()</a>

This differs from when getting an implementation from a node definition which will return either a source
implementation or a node graph implementation when using `getImplementation()`.`

In [63]:
# Collect all Implementation and NodeGraph elements from two libraries
allImpls = set()
implsNoDef = set()
for i in currLibrary.getImplementations():
    if i.getNodeDef():
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
for i in currLibrary.getNodeGraphs():
    if i.getNodeDef():
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
print('Current implementation count:', len(allImpls))

allImpls2 = set()
for i in otherLibrary.getImplementations():
    if i.getNodeDef():
        allImpls2.add(i.getName())
        allImpls.add(i.getName())
    else:
        implsNoDef.add(i.getName())
for i in otherLibrary.getNodeGraphs():
    if i.getNodeDef():
        allImpls2.add(i.getName())
        allImpls.add(i.getName())
print('Previous implementation count:', len(allImpls2))

Current implementation count: 2381
Previous implementation count: 2172


As with definitions we can compare the implementations which are in both libraries in detail.

In [64]:
title = '| Name | Node Category | Node Type |\n'
title = title + '| --- | --- | --- |\n'

allImpls = sorted(allImpls)
added = 0
addedText = title
removed = 0
removedText = title

title = '| Name | Change |\n| --- | --- |\n'
changed = 0
changedText = title

for item in allImpls:
    
    # Check for new implementations. Note that we check for source implementations
    # as well as node graph implementations.
    impl = otherLibrary.getImplementation(item)
    if not impl:
        impl = otherLibrary.getNodeGraph(item)
    if not impl:
        ns = 'None'
        nt = 'None'
        curimpl = currLibrary.getChild(item)
        if curimpl:
            nd = curimpl.getNodeDef()
            if nd:
                ns = nd.getNodeString()
                nt = nd.getType()
        addedText = addedText + '| %s | <a href="../documents/definitions/%s.html">%s</a> | %s | \n' % (item, ns, ns, nt) 
        added = added + 1
        continue
    
    # Check for implementations which have been removed. Note that we check for source implementations
    # as well as node graph implementations.
    impl2 = currLibrary.getImplementation(item)
    if not impl2:
        impl2 = currLibrary.getNodeGraph(item)
    if not impl2:
        oldimpl = currLibrary.getChild(item)
        if oldimpl:
            nd = oldimpl.getNodeDef()
            if nd:
                ns = nd.getNodeString()
                nt = nd.getType()        
        removedText = removedText + '| %s | %s | %s | \n' % (item, ns, nt)
        removed = removed + 1
        continue
    
    # Check for implementations which have been modified.
    # Use the built in comparitor first before performing a detailed comparison.
    difference = (impl != impl2)
    if difference:
        ns = 'None'
        nt = 'None'
        if impl2:
            nd = impl2.getNodeDef()
            if nd:
                ns = nd.getNodeString()                
        changedText = changedText + '| <a href="../documents/definitions/%s.html">%s</a> | ' % (ns, item)
        # Perform detailed comparison
        log = []
        difference = compareElements(impl, impl2, '', log) 
        if difference:
            for line in log:
                changedText = changedText + line + '<br>'
            changedText = changedText + '|\n'
            changed = changed + 1


delta = '%d implementations added' % added
text = '<details><summary>' + delta + '</summary>\n\n' + addedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)

delta = '%d implementations removed' % removed
text = '<details><summary>' + delta + '</summary>\n\n' +  removedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)

delta = '%d implementations modified' % changed
text = '<details><summary>' + delta + '</summary>\n\n' + changedText + '\n' + '</details>\n' 
display_markdown(text, raw=True)


<details><summary>690 implementations added</summary>

| Name | Node Category | Node Type |
| --- | --- | --- |
| IMPL_gltf_pbr_surfaceshader | <a href="../documents/definitions/gltf_pbr.html">gltf_pbr</a> | surfaceshader | 
| IMPL_lama_conductor | <a href="../documents/definitions/LamaConductor.html">LamaConductor</a> | BSDF | 
| IMPL_lama_dielectric | <a href="../documents/definitions/LamaDielectric.html">LamaDielectric</a> | BSDF | 
| IMPL_lama_emission | <a href="../documents/definitions/LamaEmission.html">LamaEmission</a> | EDF | 
| IMPL_lama_sheen | <a href="../documents/definitions/LamaSheen.html">LamaSheen</a> | BSDF | 
| IMPL_lama_sss | <a href="../documents/definitions/LamaSSS.html">LamaSSS</a> | BSDF | 
| IMPL_standard_surface_surfaceshader_100 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| IMPL_standard_surface_surfaceshader_101 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| IMP_UsdUVTexture_22 | <a href="../documents/definitions/UsdUVTexture.html">UsdUVTexture</a> | multioutput | 
| IMP_UsdUVTexture_23 | <a href="../documents/definitions/UsdUVTexture.html">UsdUVTexture</a> | multioutput | 
| IM_absval_color3_genmsl | <a href="../documents/definitions/absval.html">absval</a> | color3 | 
| IM_absval_color4_genmsl | <a href="../documents/definitions/absval.html">absval</a> | color4 | 
| IM_absval_float_genmsl | <a href="../documents/definitions/absval.html">absval</a> | float | 
| IM_absval_vector2_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector2 | 
| IM_absval_vector3_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector3 | 
| IM_absval_vector4_genmsl | <a href="../documents/definitions/absval.html">absval</a> | vector4 | 
| IM_acos_float_genmsl | <a href="../documents/definitions/acos.html">acos</a> | float | 
| IM_acos_vector2_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector2 | 
| IM_acos_vector3_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector3 | 
| IM_acos_vector4_genmsl | <a href="../documents/definitions/acos.html">acos</a> | vector4 | 
| IM_add_bsdf_genmsl | <a href="../documents/definitions/add.html">add</a> | BSDF | 
| IM_add_color3FA_genmsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color3_genmsl | <a href="../documents/definitions/add.html">add</a> | color3 | 
| IM_add_color4FA_genmsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_color4_genmsl | <a href="../documents/definitions/add.html">add</a> | color4 | 
| IM_add_edf_genmsl | <a href="../documents/definitions/add.html">add</a> | EDF | 
| IM_add_float_genmsl | <a href="../documents/definitions/add.html">add</a> | float | 
| IM_add_matrix33FA_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix33_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix33 | 
| IM_add_matrix44FA_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_matrix44_genmsl | <a href="../documents/definitions/add.html">add</a> | matrix44 | 
| IM_add_vdf_genmdl | <a href="../documents/definitions/add.html">add</a> | VDF | 
| IM_add_vector2FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector2_genmsl | <a href="../documents/definitions/add.html">add</a> | vector2 | 
| IM_add_vector3FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector3_genmsl | <a href="../documents/definitions/add.html">add</a> | vector3 | 
| IM_add_vector4FA_genmsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_add_vector4_genmsl | <a href="../documents/definitions/add.html">add</a> | vector4 | 
| IM_anisotropic_vdf_genglsl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_anisotropic_vdf_genmsl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_anisotropic_vdf_genosl | <a href="../documents/definitions/anisotropic_vdf.html">anisotropic_vdf</a> | VDF | 
| IM_artistic_ior_genmsl | <a href="../documents/definitions/artistic_ior.html">artistic_ior</a> | multioutput | 
| IM_asin_float_genmsl | <a href="../documents/definitions/asin.html">asin</a> | float | 
| IM_asin_vector2_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector2 | 
| IM_asin_vector3_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector3 | 
| IM_asin_vector4_genmsl | <a href="../documents/definitions/asin.html">asin</a> | vector4 | 
| IM_atan2_float_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | float | 
| IM_atan2_vector2_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector2 | 
| IM_atan2_vector3_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector3 | 
| IM_atan2_vector4_genmsl | <a href="../documents/definitions/atan2.html">atan2</a> | vector4 | 
| IM_bitangent_vector3_genmsl | <a href="../documents/definitions/bitangent.html">bitangent</a> | vector3 | 
| IM_blackbody_genglsl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blackbody_genmsl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blackbody_genosl | <a href="../documents/definitions/blackbody.html">blackbody</a> | color3 | 
| IM_blur_color3_genmsl | <a href="../documents/definitions/blur.html">blur</a> | color3 | 
| IM_blur_color4_genmsl | <a href="../documents/definitions/blur.html">blur</a> | color4 | 
| IM_blur_float_genmsl | <a href="../documents/definitions/blur.html">blur</a> | float | 
| IM_blur_vector2_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector2 | 
| IM_blur_vector3_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector3 | 
| IM_blur_vector4_genmsl | <a href="../documents/definitions/blur.html">blur</a> | vector4 | 
| IM_burley_diffuse_bsdf_genmsl | <a href="../documents/definitions/burley_diffuse_bsdf.html">burley_diffuse_bsdf</a> | BSDF | 
| IM_burn_color3_genmsl | <a href="../documents/definitions/burn.html">burn</a> | color3 | 
| IM_burn_color4_genmsl | <a href="../documents/definitions/burn.html">burn</a> | color4 | 
| IM_burn_float_genmsl | <a href="../documents/definitions/burn.html">burn</a> | float | 
| IM_ceil_color3_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | color3 | 
| IM_ceil_color4_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | color4 | 
| IM_ceil_float_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | float | 
| IM_ceil_integer_genglsl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genmdl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_integer_genosl | <a href="../documents/definitions/ceil.html">ceil</a> | integer | 
| IM_ceil_vector2_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector2 | 
| IM_ceil_vector3_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector3 | 
| IM_ceil_vector4_genmsl | <a href="../documents/definitions/ceil.html">ceil</a> | vector4 | 
| IM_cellnoise2d_float_genmsl | <a href="../documents/definitions/cellnoise2d.html">cellnoise2d</a> | float | 
| IM_cellnoise3d_float_genmsl | <a href="../documents/definitions/cellnoise3d.html">cellnoise3d</a> | float | 
| IM_clamp_color3FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color3_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color3 | 
| IM_clamp_color4FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_color4_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | color4 | 
| IM_clamp_float_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | float | 
| IM_clamp_vector2FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector2_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector2 | 
| IM_clamp_vector3FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector3_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector3 | 
| IM_clamp_vector4FA_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_clamp_vector4_genmsl | <a href="../documents/definitions/clamp.html">clamp</a> | vector4 | 
| IM_combine2_color4CF_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | color4 | 
| IM_combine2_vector2_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector2 | 
| IM_combine2_vector4VF_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine2_vector4VV_genmsl | <a href="../documents/definitions/combine2.html">combine2</a> | vector4 | 
| IM_combine3_color3_genmsl | <a href="../documents/definitions/combine3.html">combine3</a> | color3 | 
| IM_combine3_vector3_genmsl | <a href="../documents/definitions/combine3.html">combine3</a> | vector3 | 
| IM_combine4_color4_genmsl | <a href="../documents/definitions/combine4.html">combine4</a> | color4 | 
| IM_combine4_vector4_genmsl | <a href="../documents/definitions/combine4.html">combine4</a> | vector4 | 
| IM_conductor_bsdf_genmsl | <a href="../documents/definitions/conductor_bsdf.html">conductor_bsdf</a> | BSDF | 
| IM_constant_boolean_genmsl | <a href="../documents/definitions/constant.html">constant</a> | boolean | 
| IM_constant_color3_genmsl | <a href="../documents/definitions/constant.html">constant</a> | color3 | 
| IM_constant_color4_genmsl | <a href="../documents/definitions/constant.html">constant</a> | color4 | 
| IM_constant_filename_genmsl | <a href="../documents/definitions/constant.html">constant</a> | filename | 
| IM_constant_float_genmsl | <a href="../documents/definitions/constant.html">constant</a> | float | 
| IM_constant_integer_genmsl | <a href="../documents/definitions/constant.html">constant</a> | integer | 
| IM_constant_matrix33_genmsl | <a href="../documents/definitions/constant.html">constant</a> | matrix33 | 
| IM_constant_matrix44_genmsl | <a href="../documents/definitions/constant.html">constant</a> | matrix44 | 
| IM_constant_string_genmsl | <a href="../documents/definitions/constant.html">constant</a> | string | 
| IM_constant_vector2_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector2 | 
| IM_constant_vector3_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector3 | 
| IM_constant_vector4_genmsl | <a href="../documents/definitions/constant.html">constant</a> | vector4 | 
| IM_convert_boolean_float_genmsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_color3_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_color3_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_color4_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_color4_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_float_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_float_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_float_vector2_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_float_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_float_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_integer_float_genmsl | <a href="../documents/definitions/convert.html">convert</a> | float | 
| IM_convert_vector2_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_convert_vector3_color3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color3 | 
| IM_convert_vector3_vector2_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector2 | 
| IM_convert_vector3_vector4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector4 | 
| IM_convert_vector4_color4_genmsl | <a href="../documents/definitions/convert.html">convert</a> | color4 | 
| IM_convert_vector4_vector3_genmsl | <a href="../documents/definitions/convert.html">convert</a> | vector3 | 
| IM_cos_float_genmsl | <a href="../documents/definitions/cos.html">cos</a> | float | 
| IM_cos_vector2_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector2 | 
| IM_cos_vector3_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector3 | 
| IM_cos_vector4_genmsl | <a href="../documents/definitions/cos.html">cos</a> | vector4 | 
| IM_crossproduct_vector3_genmsl | <a href="../documents/definitions/crossproduct.html">crossproduct</a> | vector3 | 
| IM_determinant_matrix33_genmsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_determinant_matrix44_genmsl | <a href="../documents/definitions/determinant.html">determinant</a> | float | 
| IM_dielectric_bsdf_genmsl | <a href="../documents/definitions/dielectric_bsdf.html">dielectric_bsdf</a> | BSDF | 
| IM_difference_color3_genmsl | <a href="../documents/definitions/difference.html">difference</a> | color3 | 
| IM_difference_color4_genmsl | <a href="../documents/definitions/difference.html">difference</a> | color4 | 
| IM_difference_float_genmsl | <a href="../documents/definitions/difference.html">difference</a> | float | 
| IM_directional_light_genmsl | <a href="../documents/definitions/directional_light.html">directional_light</a> | lightshader | 
| IM_disjointover_color4_genmsl | <a href="../documents/definitions/disjointover.html">disjointover</a> | color4 | 
| IM_displacement_float_genglsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_float_genmsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genglsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_displacement_vector3_genmsl | <a href="../documents/definitions/displacement.html">displacement</a> | displacementshader | 
| IM_divide_color3FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color3_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color3 | 
| IM_divide_color4FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_color4_genmsl | <a href="../documents/definitions/divide.html">divide</a> | color4 | 
| IM_divide_float_genmsl | <a href="../documents/definitions/divide.html">divide</a> | float | 
| IM_divide_matrix33_genmsl | <a href="../documents/definitions/divide.html">divide</a> | matrix33 | 
| IM_divide_matrix44_genmsl | <a href="../documents/definitions/divide.html">divide</a> | matrix44 | 
| IM_divide_vector2FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector2_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector2 | 
| IM_divide_vector3FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector3_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector3 | 
| IM_divide_vector4FA_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_divide_vector4_genmsl | <a href="../documents/definitions/divide.html">divide</a> | vector4 | 
| IM_dodge_color3_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | color3 | 
| IM_dodge_color4_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | color4 | 
| IM_dodge_float_genmsl | <a href="../documents/definitions/dodge.html">dodge</a> | float | 
| IM_dot_boolean_genmsl | <a href="../documents/definitions/dot.html">dot</a> | boolean | 
| IM_dot_color3_genmsl | <a href="../documents/definitions/dot.html">dot</a> | color3 | 
| IM_dot_color4_genmsl | <a href="../documents/definitions/dot.html">dot</a> | color4 | 
| IM_dot_displacementshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | displacementshader | 
| IM_dot_filename_genmsl | <a href="../documents/definitions/dot.html">dot</a> | filename | 
| IM_dot_float_genmsl | <a href="../documents/definitions/dot.html">dot</a> | float | 
| IM_dot_integer_genmsl | <a href="../documents/definitions/dot.html">dot</a> | integer | 
| IM_dot_lightshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | lightshader | 
| IM_dot_matrix33_genmsl | <a href="../documents/definitions/dot.html">dot</a> | matrix33 | 
| IM_dot_matrix44_genmsl | <a href="../documents/definitions/dot.html">dot</a> | matrix44 | 
| IM_dot_string_genmsl | <a href="../documents/definitions/dot.html">dot</a> | string | 
| IM_dot_surfaceshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | surfaceshader | 
| IM_dot_vector2_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector2 | 
| IM_dot_vector3_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector3 | 
| IM_dot_vector4_genmsl | <a href="../documents/definitions/dot.html">dot</a> | vector4 | 
| IM_dot_volumeshader_genmsl | <a href="../documents/definitions/dot.html">dot</a> | volumeshader | 
| IM_dotproduct_vector2_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector3_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_dotproduct_vector4_genmsl | <a href="../documents/definitions/dotproduct.html">dotproduct</a> | float | 
| IM_exp_float_genmsl | <a href="../documents/definitions/exp.html">exp</a> | float | 
| IM_exp_vector2_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector2 | 
| IM_exp_vector3_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector3 | 
| IM_exp_vector4_genmsl | <a href="../documents/definitions/exp.html">exp</a> | vector4 | 
| IM_floor_color3_genmsl | <a href="../documents/definitions/floor.html">floor</a> | color3 | 
| IM_floor_color4_genmsl | <a href="../documents/definitions/floor.html">floor</a> | color4 | 
| IM_floor_float_genmsl | <a href="../documents/definitions/floor.html">floor</a> | float | 
| IM_floor_integer_genglsl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genmdl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genmsl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_integer_genosl | <a href="../documents/definitions/floor.html">floor</a> | integer | 
| IM_floor_vector2_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector2 | 
| IM_floor_vector3_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector3 | 
| IM_floor_vector4_genmsl | <a href="../documents/definitions/floor.html">floor</a> | vector4 | 
| IM_fractal3d_color3FA_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color3 | 
| IM_fractal3d_color3_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color3 | 
| IM_fractal3d_color4FA_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color4 | 
| IM_fractal3d_color4_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | color4 | 
| IM_fractal3d_float_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | float | 
| IM_fractal3d_vector2FA_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector2_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector2 | 
| IM_fractal3d_vector3FA_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector3_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector3 | 
| IM_fractal3d_vector4FA_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_fractal3d_vector4_genmsl | <a href="../documents/definitions/fractal3d.html">fractal3d</a> | vector4 | 
| IM_frame_float_genmsl | <a href="../documents/definitions/frame.html">frame</a> | float | 
| IM_generalized_schlick_bsdf_genmsl | <a href="../documents/definitions/generalized_schlick_bsdf.html">generalized_schlick_bsdf</a> | BSDF | 
| IM_generalized_schlick_edf_genglsl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_generalized_schlick_edf_genmdl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_generalized_schlick_edf_genosl | <a href="../documents/definitions/generalized_schlick_edf.html">generalized_schlick_edf</a> | EDF | 
| IM_geomcolor_color3_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color3 | 
| IM_geomcolor_color4_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | color4 | 
| IM_geomcolor_float_genmsl | <a href="../documents/definitions/geomcolor.html">geomcolor</a> | float | 
| IM_geompropvalue_boolean_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | boolean | 
| IM_geompropvalue_color3_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color3 | 
| IM_geompropvalue_color4_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | color4 | 
| IM_geompropvalue_float_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | float | 
| IM_geompropvalue_integer_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | integer | 
| IM_geompropvalue_string_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | string | 
| IM_geompropvalue_vector2_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector2 | 
| IM_geompropvalue_vector3_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector3 | 
| IM_geompropvalue_vector4_genmsl | <a href="../documents/definitions/geompropvalue.html">geompropvalue</a> | vector4 | 
| IM_heighttonormal_vector3_genmsl | <a href="../documents/definitions/heighttonormal.html">heighttonormal</a> | vector3 | 
| IM_hsvtorgb_color3_genmsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color3 | 
| IM_hsvtorgb_color4_genmsl | <a href="../documents/definitions/hsvtorgb.html">hsvtorgb</a> | color4 | 
| IM_ifequal_color3B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color3_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color3 | 
| IM_ifequal_color4B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_color4_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | color4 | 
| IM_ifequal_floatB_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_floatI_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_float_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | float | 
| IM_ifequal_vector2B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector2_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector2 | 
| IM_ifequal_vector3B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector3_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector3 | 
| IM_ifequal_vector4B_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4I_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifequal_vector4_genmsl | <a href="../documents/definitions/ifequal.html">ifequal</a> | vector4 | 
| IM_ifgreater_color3I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color3_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color3 | 
| IM_ifgreater_color4I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_color4_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | color4 | 
| IM_ifgreater_floatI_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_float_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | float | 
| IM_ifgreater_vector2I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector2_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector2 | 
| IM_ifgreater_vector3I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector3_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector3 | 
| IM_ifgreater_vector4I_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreater_vector4_genmsl | <a href="../documents/definitions/ifgreater.html">ifgreater</a> | vector4 | 
| IM_ifgreatereq_color3I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color3_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color3 | 
| IM_ifgreatereq_color4I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_color4_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | color4 | 
| IM_ifgreatereq_floatI_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_float_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | float | 
| IM_ifgreatereq_vector2I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector2_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector2 | 
| IM_ifgreatereq_vector3I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector3_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector3 | 
| IM_ifgreatereq_vector4I_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_ifgreatereq_vector4_genmsl | <a href="../documents/definitions/ifgreatereq.html">ifgreatereq</a> | vector4 | 
| IM_image_color3_genmsl | <a href="../documents/definitions/image.html">image</a> | color3 | 
| IM_image_color4_genmsl | <a href="../documents/definitions/image.html">image</a> | color4 | 
| IM_image_float_genmsl | <a href="../documents/definitions/image.html">image</a> | float | 
| IM_image_vector2_genmsl | <a href="../documents/definitions/image.html">image</a> | vector2 | 
| IM_image_vector3_genmsl | <a href="../documents/definitions/image.html">image</a> | vector3 | 
| IM_image_vector4_genmsl | <a href="../documents/definitions/image.html">image</a> | vector4 | 
| IM_in_color4_genmsl | <a href="../documents/definitions/in.html">in</a> | color4 | 
| IM_inside_color3_genmsl | <a href="../documents/definitions/inside.html">inside</a> | color3 | 
| IM_inside_color4_genmsl | <a href="../documents/definitions/inside.html">inside</a> | color4 | 
| IM_inside_float_genmsl | <a href="../documents/definitions/inside.html">inside</a> | float | 
| IM_invert_color3FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color3_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color3 | 
| IM_invert_color4FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_color4_genmsl | <a href="../documents/definitions/invert.html">invert</a> | color4 | 
| IM_invert_float_genmsl | <a href="../documents/definitions/invert.html">invert</a> | float | 
| IM_invert_vector2FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector2_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector2 | 
| IM_invert_vector3FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector3_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector3 | 
| IM_invert_vector4FA_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invert_vector4_genmsl | <a href="../documents/definitions/invert.html">invert</a> | vector4 | 
| IM_invertmatrix_matrix33_genmsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix33 | 
| IM_invertmatrix_matrix44_genmsl | <a href="../documents/definitions/invertmatrix.html">invertmatrix</a> | matrix44 | 
| IM_layer_bsdf_genmsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genglsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genmdl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genmsl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_layer_vdf_genosl | <a href="../documents/definitions/layer.html">layer</a> | BSDF | 
| IM_light_genmsl | <a href="../documents/definitions/light.html">light</a> | lightshader | 
| IM_ln_float_genmsl | <a href="../documents/definitions/ln.html">ln</a> | float | 
| IM_ln_vector2_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector2 | 
| IM_ln_vector3_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector3 | 
| IM_ln_vector4_genmsl | <a href="../documents/definitions/ln.html">ln</a> | vector4 | 
| IM_luminance_color3_genmsl | <a href="../documents/definitions/luminance.html">luminance</a> | color3 | 
| IM_luminance_color4_genmsl | <a href="../documents/definitions/luminance.html">luminance</a> | color4 | 
| IM_magnitude_vector2_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector3_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_magnitude_vector4_genmsl | <a href="../documents/definitions/magnitude.html">magnitude</a> | float | 
| IM_mask_color4_genmsl | <a href="../documents/definitions/mask.html">mask</a> | color4 | 
| IM_matte_color4_genmsl | <a href="../documents/definitions/matte.html">matte</a> | color4 | 
| IM_max_color3FA_genmsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color3_genmsl | <a href="../documents/definitions/max.html">max</a> | color3 | 
| IM_max_color4FA_genmsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_color4_genmsl | <a href="../documents/definitions/max.html">max</a> | color4 | 
| IM_max_float_genmsl | <a href="../documents/definitions/max.html">max</a> | float | 
| IM_max_vector2FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector2_genmsl | <a href="../documents/definitions/max.html">max</a> | vector2 | 
| IM_max_vector3FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector3_genmsl | <a href="../documents/definitions/max.html">max</a> | vector3 | 
| IM_max_vector4FA_genmsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_max_vector4_genmsl | <a href="../documents/definitions/max.html">max</a> | vector4 | 
| IM_min_color3FA_genmsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color3_genmsl | <a href="../documents/definitions/min.html">min</a> | color3 | 
| IM_min_color4FA_genmsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_color4_genmsl | <a href="../documents/definitions/min.html">min</a> | color4 | 
| IM_min_float_genmsl | <a href="../documents/definitions/min.html">min</a> | float | 
| IM_min_vector2FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector2_genmsl | <a href="../documents/definitions/min.html">min</a> | vector2 | 
| IM_min_vector3FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector3_genmsl | <a href="../documents/definitions/min.html">min</a> | vector3 | 
| IM_min_vector4FA_genmsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_min_vector4_genmsl | <a href="../documents/definitions/min.html">min</a> | vector4 | 
| IM_minus_color3_genmsl | <a href="../documents/definitions/minus.html">minus</a> | color3 | 
| IM_minus_color4_genmsl | <a href="../documents/definitions/minus.html">minus</a> | color4 | 
| IM_minus_float_genmsl | <a href="../documents/definitions/minus.html">minus</a> | float | 
| IM_mix_bsdf_genmsl | <a href="../documents/definitions/mix.html">mix</a> | BSDF | 
| IM_mix_color3_color3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_color3_genosl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color3 | 
| IM_mix_color4_color4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_color4_genosl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_color4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | color4 | 
| IM_mix_displacementshader_genmdl | <a href="../documents/definitions/mix.html">mix</a> | displacementshader | 
| IM_mix_edf_genmsl | <a href="../documents/definitions/mix.html">mix</a> | EDF | 
| IM_mix_float_genmsl | <a href="../documents/definitions/mix.html">mix</a> | float | 
| IM_mix_surfaceshader_genmsl | <a href="../documents/definitions/mix.html">mix</a> | surfaceshader | 
| IM_mix_vector2_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector2_vector2_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector2 | 
| IM_mix_vector3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector3_vector3_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector3 | 
| IM_mix_vector4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genglsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genmdl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genmsl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_vector4_vector4_genosl | <a href="../documents/definitions/mix.html">mix</a> | vector4 | 
| IM_mix_volumeshader_genmdl | <a href="../documents/definitions/mix.html">mix</a> | volumeshader | 
| IM_modulo_color3FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color3_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color3 | 
| IM_modulo_color4FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_color4_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | color4 | 
| IM_modulo_float_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | float | 
| IM_modulo_vector2FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector2_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector2 | 
| IM_modulo_vector3FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector3_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector3 | 
| IM_modulo_vector4FA_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_modulo_vector4_genmsl | <a href="../documents/definitions/modulo.html">modulo</a> | vector4 | 
| IM_multiply_bsdfC_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_bsdfF_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | BSDF | 
| IM_multiply_color3FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color3_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color3 | 
| IM_multiply_color4FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_color4_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | color4 | 
| IM_multiply_edfC_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_edfF_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | EDF | 
| IM_multiply_float_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | float | 
| IM_multiply_matrix33_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix33 | 
| IM_multiply_matrix44_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | matrix44 | 
| IM_multiply_vdfC_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | VDF | 
| IM_multiply_vdfF_genmdl | <a href="../documents/definitions/multiply.html">multiply</a> | VDF | 
| IM_multiply_vector2FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector2_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector2 | 
| IM_multiply_vector3FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector3_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector3 | 
| IM_multiply_vector4FA_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_multiply_vector4_genmsl | <a href="../documents/definitions/multiply.html">multiply</a> | vector4 | 
| IM_noise2d_color3FA_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | color3 | 
| IM_noise2d_color3_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | color3 | 
| IM_noise2d_color4FA_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | color4 | 
| IM_noise2d_color4_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | color4 | 
| IM_noise2d_float_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | float | 
| IM_noise2d_vector2FA_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector2_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector2 | 
| IM_noise2d_vector3FA_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector3_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector3 | 
| IM_noise2d_vector4FA_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise2d_vector4_genmsl | <a href="../documents/definitions/noise2d.html">noise2d</a> | vector4 | 
| IM_noise3d_color3FA_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | color3 | 
| IM_noise3d_color3_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | color3 | 
| IM_noise3d_color4FA_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | color4 | 
| IM_noise3d_color4_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | color4 | 
| IM_noise3d_float_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | float | 
| IM_noise3d_vector2FA_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector2_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector2 | 
| IM_noise3d_vector3FA_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector3_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector3 | 
| IM_noise3d_vector4FA_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_noise3d_vector4_genmsl | <a href="../documents/definitions/noise3d.html">noise3d</a> | vector4 | 
| IM_normal_vector3_genmsl | <a href="../documents/definitions/normal.html">normal</a> | vector3 | 
| IM_normalize_vector2_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector2 | 
| IM_normalize_vector3_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector3 | 
| IM_normalize_vector4_genmsl | <a href="../documents/definitions/normalize.html">normalize</a> | vector4 | 
| IM_normalmap_genmsl | <a href="../documents/definitions/normalmap.html">normalmap</a> | vector3 | 
| IM_oren_nayar_diffuse_bsdf_genmsl | <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">oren_nayar_diffuse_bsdf</a> | BSDF | 
| IM_out_color4_genmsl | <a href="../documents/definitions/out.html">out</a> | color4 | 
| IM_outside_color3_genmsl | <a href="../documents/definitions/outside.html">outside</a> | color3 | 
| IM_outside_color4_genmsl | <a href="../documents/definitions/outside.html">outside</a> | color4 | 
| IM_outside_float_genmsl | <a href="../documents/definitions/outside.html">outside</a> | float | 
| IM_over_color4_genmsl | <a href="../documents/definitions/over.html">over</a> | color4 | 
| IM_overlay_color3_genmsl | <a href="../documents/definitions/overlay.html">overlay</a> | color3 | 
| IM_overlay_color4_genmsl | <a href="../documents/definitions/overlay.html">overlay</a> | color4 | 
| IM_overlay_float_genmsl | <a href="../documents/definitions/overlay.html">overlay</a> | float | 
| IM_plus_color3_genmsl | <a href="../documents/definitions/plus.html">plus</a> | color3 | 
| IM_plus_color4_genmsl | <a href="../documents/definitions/plus.html">plus</a> | color4 | 
| IM_plus_float_genmsl | <a href="../documents/definitions/plus.html">plus</a> | float | 
| IM_point_light_genmsl | <a href="../documents/definitions/point_light.html">point_light</a> | lightshader | 
| IM_position_vector3_genmsl | <a href="../documents/definitions/position.html">position</a> | vector3 | 
| IM_power_color3FA_genmsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color3_genmsl | <a href="../documents/definitions/power.html">power</a> | color3 | 
| IM_power_color4FA_genmsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_color4_genmsl | <a href="../documents/definitions/power.html">power</a> | color4 | 
| IM_power_float_genmsl | <a href="../documents/definitions/power.html">power</a> | float | 
| IM_power_vector2FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector2_genmsl | <a href="../documents/definitions/power.html">power</a> | vector2 | 
| IM_power_vector3FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector3_genmsl | <a href="../documents/definitions/power.html">power</a> | vector3 | 
| IM_power_vector4FA_genmsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_power_vector4_genmsl | <a href="../documents/definitions/power.html">power</a> | vector4 | 
| IM_premult_color4_genmsl | <a href="../documents/definitions/premult.html">premult</a> | color4 | 
| IM_ramplr_color3_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color3 | 
| IM_ramplr_color4_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | color4 | 
| IM_ramplr_float_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | float | 
| IM_ramplr_vector2_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector2 | 
| IM_ramplr_vector3_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector3 | 
| IM_ramplr_vector4_genmsl | <a href="../documents/definitions/ramplr.html">ramplr</a> | vector4 | 
| IM_ramptb_color3_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color3 | 
| IM_ramptb_color4_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | color4 | 
| IM_ramptb_float_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | float | 
| IM_ramptb_vector2_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector2 | 
| IM_ramptb_vector3_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector3 | 
| IM_ramptb_vector4_genmsl | <a href="../documents/definitions/ramptb.html">ramptb</a> | vector4 | 
| IM_remap_color3FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color3_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color3 | 
| IM_remap_color4FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_color4_genmsl | <a href="../documents/definitions/remap.html">remap</a> | color4 | 
| IM_remap_float_genmsl | <a href="../documents/definitions/remap.html">remap</a> | float | 
| IM_remap_vector2FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector2_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector2 | 
| IM_remap_vector3FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector3_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector3 | 
| IM_remap_vector4FA_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_remap_vector4_genmsl | <a href="../documents/definitions/remap.html">remap</a> | vector4 | 
| IM_rgbtohsv_color3_genmsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color3 | 
| IM_rgbtohsv_color4_genmsl | <a href="../documents/definitions/rgbtohsv.html">rgbtohsv</a> | color4 | 
| IM_rotate2d_vector2_genmsl | <a href="../documents/definitions/rotate2d.html">rotate2d</a> | vector2 | 
| IM_rotate3d_vector3_genmsl | <a href="../documents/definitions/rotate3d.html">rotate3d</a> | vector3 | 
| IM_roughness_anisotropy_genmsl | <a href="../documents/definitions/roughness_anisotropy.html">roughness_anisotropy</a> | vector2 | 
| IM_roughness_dual_genmsl | <a href="../documents/definitions/roughness_dual.html">roughness_dual</a> | vector2 | 
| IM_screen_color3_genmsl | <a href="../documents/definitions/screen.html">screen</a> | color3 | 
| IM_screen_color4_genmsl | <a href="../documents/definitions/screen.html">screen</a> | color4 | 
| IM_screen_float_genmsl | <a href="../documents/definitions/screen.html">screen</a> | float | 
| IM_sheen_bsdf_genmsl | <a href="../documents/definitions/sheen_bsdf.html">sheen_bsdf</a> | BSDF | 
| IM_sign_color3_genmsl | <a href="../documents/definitions/sign.html">sign</a> | color3 | 
| IM_sign_color4_genmsl | <a href="../documents/definitions/sign.html">sign</a> | color4 | 
| IM_sign_float_genmsl | <a href="../documents/definitions/sign.html">sign</a> | float | 
| IM_sign_vector2_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector2 | 
| IM_sign_vector3_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector3 | 
| IM_sign_vector4_genmsl | <a href="../documents/definitions/sign.html">sign</a> | vector4 | 
| IM_sin_float_genmsl | <a href="../documents/definitions/sin.html">sin</a> | float | 
| IM_sin_vector2_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector2 | 
| IM_sin_vector3_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector3 | 
| IM_sin_vector4_genmsl | <a href="../documents/definitions/sin.html">sin</a> | vector4 | 
| IM_smoothstep_color3FA_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color3 | 
| IM_smoothstep_color3_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color3 | 
| IM_smoothstep_color4FA_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color4 | 
| IM_smoothstep_color4_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | color4 | 
| IM_smoothstep_float_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | float | 
| IM_smoothstep_vector2FA_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector2_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector2 | 
| IM_smoothstep_vector3FA_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector3_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector3 | 
| IM_smoothstep_vector4FA_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_smoothstep_vector4_genmsl | <a href="../documents/definitions/smoothstep.html">smoothstep</a> | vector4 | 
| IM_splitlr_color3_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color3 | 
| IM_splitlr_color4_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | color4 | 
| IM_splitlr_float_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | float | 
| IM_splitlr_vector2_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector2 | 
| IM_splitlr_vector3_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector3 | 
| IM_splitlr_vector4_genmsl | <a href="../documents/definitions/splitlr.html">splitlr</a> | vector4 | 
| IM_splittb_color3_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | color3 | 
| IM_splittb_color4_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | color4 | 
| IM_splittb_float_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | float | 
| IM_splittb_vector2_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector2 | 
| IM_splittb_vector3_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector3 | 
| IM_splittb_vector4_genmsl | <a href="../documents/definitions/splittb.html">splittb</a> | vector4 | 
| IM_spot_light_genmsl | <a href="../documents/definitions/spot_light.html">spot_light</a> | lightshader | 
| IM_sqrt_float_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | float | 
| IM_sqrt_vector2_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector2 | 
| IM_sqrt_vector3_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector3 | 
| IM_sqrt_vector4_genmsl | <a href="../documents/definitions/sqrt.html">sqrt</a> | vector4 | 
| IM_subsurface_bsdf_genmsl | <a href="../documents/definitions/subsurface_bsdf.html">subsurface_bsdf</a> | BSDF | 
| IM_subtract_color3FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color3_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color3 | 
| IM_subtract_color4FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_color4_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | color4 | 
| IM_subtract_float_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | float | 
| IM_subtract_matrix33FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix33_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix33 | 
| IM_subtract_matrix44FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_matrix44_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | matrix44 | 
| IM_subtract_vector2FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector2_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector2 | 
| IM_subtract_vector3FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector3_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector3 | 
| IM_subtract_vector4FA_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_subtract_vector4_genmsl | <a href="../documents/definitions/subtract.html">subtract</a> | vector4 | 
| IM_surface_genmsl | <a href="../documents/definitions/surface.html">surface</a> | surfaceshader | 
| IM_surface_unlit_genglsl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genmdl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genmsl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surface_unlit_genosl | <a href="../documents/definitions/surface_unlit.html">surface_unlit</a> | surfaceshader | 
| IM_surfacematerial_genglsl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genmdl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genmsl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_surfacematerial_genosl | <a href="../documents/definitions/surfacematerial.html">surfacematerial</a> | material | 
| IM_switch_color3I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color3_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color3 | 
| IM_switch_color4I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_color4_genmsl | <a href="../documents/definitions/switch.html">switch</a> | color4 | 
| IM_switch_floatI_genmsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_float_genmsl | <a href="../documents/definitions/switch.html">switch</a> | float | 
| IM_switch_vector2I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector2_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector2 | 
| IM_switch_vector3I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector3_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector3 | 
| IM_switch_vector4I_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_switch_vector4_genmsl | <a href="../documents/definitions/switch.html">switch</a> | vector4 | 
| IM_swizzle_color3_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color3_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color3_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color3_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color3_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color3_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_color4_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_color4_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_color4_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_color4_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_color4_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_color4_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_float_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_float_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_float_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_float_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_float_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector2_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector2_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector2_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector2_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector2_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector2_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector3_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector3_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector3_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector3_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector3_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector3_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_swizzle_vector4_color3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color3 | 
| IM_swizzle_vector4_color4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | color4 | 
| IM_swizzle_vector4_float_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | float | 
| IM_swizzle_vector4_vector2_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector2 | 
| IM_swizzle_vector4_vector3_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector3 | 
| IM_swizzle_vector4_vector4_genmsl | <a href="../documents/definitions/swizzle.html">swizzle</a> | vector4 | 
| IM_tan_float_genmsl | <a href="../documents/definitions/tan.html">tan</a> | float | 
| IM_tan_vector2_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector2 | 
| IM_tan_vector3_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector3 | 
| IM_tan_vector4_genmsl | <a href="../documents/definitions/tan.html">tan</a> | vector4 | 
| IM_tangent_vector3_genmsl | <a href="../documents/definitions/tangent.html">tangent</a> | vector3 | 
| IM_texcoord_vector2_genmsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector2 | 
| IM_texcoord_vector3_genmsl | <a href="../documents/definitions/texcoord.html">texcoord</a> | vector3 | 
| IM_thin_film_bsdf_genmsl | <a href="../documents/definitions/thin_film_bsdf.html">thin_film_bsdf</a> | BSDF | 
| IM_time_float_genmsl | <a href="../documents/definitions/time.html">time</a> | float | 
| IM_transformmatrix_vector2M3_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector2 | 
| IM_transformmatrix_vector3M4_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector3_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector3 | 
| IM_transformmatrix_vector4_genmsl | <a href="../documents/definitions/transformmatrix.html">transformmatrix</a> | vector4 | 
| IM_transformnormal_vector3_genmsl | <a href="../documents/definitions/transformnormal.html">transformnormal</a> | vector3 | 
| IM_transformpoint_vector3_genmsl | <a href="../documents/definitions/transformpoint.html">transformpoint</a> | vector3 | 
| IM_transformvector_vector3_genmsl | <a href="../documents/definitions/transformvector.html">transformvector</a> | vector3 | 
| IM_translucent_bsdf_genmsl | <a href="../documents/definitions/translucent_bsdf.html">translucent_bsdf</a> | BSDF | 
| IM_transpose_matrix33_genmsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix33 | 
| IM_transpose_matrix44_genmsl | <a href="../documents/definitions/transpose.html">transpose</a> | matrix44 | 
| IM_uniform_edf_genmsl | <a href="../documents/definitions/uniform_edf.html">uniform_edf</a> | EDF | 
| IM_unpremult_color4_genmsl | <a href="../documents/definitions/unpremult.html">unpremult</a> | color4 | 
| IM_worleynoise2d_float_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_float_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | float | 
| IM_worleynoise2d_vector2_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector2_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector2 | 
| IM_worleynoise2d_vector3_genglsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genmdl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genmsl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise2d_vector3_genosl | <a href="../documents/definitions/worleynoise2d.html">worleynoise2d</a> | vector3 | 
| IM_worleynoise3d_float_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_float_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | float | 
| IM_worleynoise3d_vector2_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector2_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector2 | 
| IM_worleynoise3d_vector3_genglsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genmdl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genmsl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| IM_worleynoise3d_vector3_genosl | <a href="../documents/definitions/worleynoise3d.html">worleynoise3d</a> | vector3 | 
| NG_NG_gltf_image_color3_color3_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | color3 | 
| NG_acescg_to_lin_rec709_color3 | <a href="../documents/definitions/acescg_to_lin_rec709.html">acescg_to_lin_rec709</a> | color3 | 
| NG_acescg_to_lin_rec709_color4 | <a href="../documents/definitions/acescg_to_lin_rec709.html">acescg_to_lin_rec709</a> | color4 | 
| NG_adobergb_to_lin_rec709_color3 | <a href="../documents/definitions/adobergb_to_lin_rec709.html">adobergb_to_lin_rec709</a> | color3 | 
| NG_adobergb_to_lin_rec709_color4 | <a href="../documents/definitions/adobergb_to_lin_rec709.html">adobergb_to_lin_rec709</a> | color4 | 
| NG_bump_vector3 | <a href="../documents/definitions/bump.html">bump</a> | vector3 | 
| NG_checkerboard_color3 | <a href="../documents/definitions/checkerboard.html">checkerboard</a> | color3 | 
| NG_colorcorrect_color3 | <a href="../documents/definitions/colorcorrect.html">colorcorrect</a> | color3 | 
| NG_colorcorrect_color4 | <a href="../documents/definitions/colorcorrect.html">colorcorrect</a> | color4 | 
| NG_convert_boolean_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_color3_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_color4_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_float_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_integer_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector2_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector3_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_convert_vector4_surfaceshader | <a href="../documents/definitions/convert.html">convert</a> | surfaceshader | 
| NG_distance_vector2 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_distance_vector3 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_distance_vector4 | <a href="../documents/definitions/distance.html">distance</a> | float | 
| NG_g18_rec709_to_lin_rec709_color3 | <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">g18_rec709_to_lin_rec709</a> | color3 | 
| NG_g18_rec709_to_lin_rec709_color4 | <a href="../documents/definitions/g18_rec709_to_lin_rec709.html">g18_rec709_to_lin_rec709</a> | color4 | 
| NG_g22_ap1_to_lin_rec709_color3 | <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">g22_ap1_to_lin_rec709</a> | color3 | 
| NG_g22_ap1_to_lin_rec709_color4 | <a href="../documents/definitions/g22_ap1_to_lin_rec709.html">g22_ap1_to_lin_rec709</a> | color4 | 
| NG_g22_rec709_to_lin_rec709_color3 | <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">g22_rec709_to_lin_rec709</a> | color3 | 
| NG_g22_rec709_to_lin_rec709_color4 | <a href="../documents/definitions/g22_rec709_to_lin_rec709.html">g22_rec709_to_lin_rec709</a> | color4 | 
| NG_gltf_colorimage | <a href="../documents/definitions/gltf_colorimage.html">gltf_colorimage</a> | multioutput | 
| NG_gltf_image_color4_color4_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | color4 | 
| NG_gltf_image_float_float_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | float | 
| NG_gltf_image_vector3_vector3_1_0 | <a href="../documents/definitions/gltf_image.html">gltf_image</a> | vector3 | 
| NG_gltf_iridescence_thickness_float_1_0 | <a href="../documents/definitions/gltf_iridescence_thickness.html">gltf_iridescence_thickness</a> | float | 
| NG_gltf_normalmap_vector3_1_0 | <a href="../documents/definitions/gltf_normalmap.html">gltf_normalmap</a> | vector3 | 
| NG_lama_add_bsdf | <a href="../documents/definitions/LamaAdd.html">LamaAdd</a> | BSDF | 
| NG_lama_add_edf | <a href="../documents/definitions/LamaAdd.html">LamaAdd</a> | EDF | 
| NG_lama_diffuse | <a href="../documents/definitions/LamaDiffuse.html">LamaDiffuse</a> | BSDF | 
| NG_lama_layer_bsdf | <a href="../documents/definitions/LamaLayer.html">LamaLayer</a> | BSDF | 
| NG_lama_mix_bsdf | <a href="../documents/definitions/LamaMix.html">LamaMix</a> | BSDF | 
| NG_lama_mix_edf | <a href="../documents/definitions/LamaMix.html">LamaMix</a> | EDF | 
| NG_lama_translucent | <a href="../documents/definitions/LamaTranslucent.html">LamaTranslucent</a> | BSDF | 
| NG_lin_adobergb_to_lin_rec709_color3 | <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">lin_adobergb_to_lin_rec709</a> | color3 | 
| NG_lin_adobergb_to_lin_rec709_color4 | <a href="../documents/definitions/lin_adobergb_to_lin_rec709.html">lin_adobergb_to_lin_rec709</a> | color4 | 
| NG_lin_displayp3_to_lin_rec709_color3 | <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">lin_displayp3_to_lin_rec709</a> | color3 | 
| NG_lin_displayp3_to_lin_rec709_color4 | <a href="../documents/definitions/lin_displayp3_to_lin_rec709.html">lin_displayp3_to_lin_rec709</a> | color4 | 
| NG_randomcolor_float | <a href="../documents/definitions/randomcolor.html">randomcolor</a> | color3 | 
| NG_randomcolor_integer | <a href="../documents/definitions/randomcolor.html">randomcolor</a> | color3 | 
| NG_randomfloat_float | <a href="../documents/definitions/randomfloat.html">randomfloat</a> | float | 
| NG_randomfloat_integer | <a href="../documents/definitions/randomfloat.html">randomfloat</a> | float | 
| NG_rec709_display_to_lin_rec709_color3 | <a href="../documents/definitions/rec709_display_to_lin_rec709.html">rec709_display_to_lin_rec709</a> | color3 | 
| NG_rec709_display_to_lin_rec709_color4 | <a href="../documents/definitions/rec709_display_to_lin_rec709.html">rec709_display_to_lin_rec709</a> | color4 | 
| NG_srgb_displayp3_to_lin_rec709_color3 | <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">srgb_displayp3_to_lin_rec709</a> | color3 | 
| NG_srgb_displayp3_to_lin_rec709_color4 | <a href="../documents/definitions/srgb_displayp3_to_lin_rec709.html">srgb_displayp3_to_lin_rec709</a> | color4 | 
| NG_srgb_texture_to_lin_rec709_color3 | <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">srgb_texture_to_lin_rec709</a> | color3 | 
| NG_srgb_texture_to_lin_rec709_color4 | <a href="../documents/definitions/srgb_texture_to_lin_rec709.html">srgb_texture_to_lin_rec709</a> | color4 | 
| NG_standard_surface_surfaceshader_100 | <a href="../documents/definitions/standard_surface.html">standard_surface</a> | surfaceshader | 
| NG_standard_surface_to_UsdPreviewSurface | <a href="../documents/definitions/standard_surface_to_UsdPreviewSurface.html">standard_surface_to_UsdPreviewSurface</a> | multioutput | 
| NG_standard_surface_to_gltf_pbr | <a href="../documents/definitions/standard_surface_to_gltf_pbr.html">standard_surface_to_gltf_pbr</a> | multioutput | 
| NG_trianglewave_float | <a href="../documents/definitions/trianglewave.html">trianglewave</a> | float | 
| NG_unifiednoise2d_float | <a href="../documents/definitions/unifiednoise2d.html">unifiednoise2d</a> | float | 
| NG_unifiednoise3d_float | <a href="../documents/definitions/unifiednoise3d.html">unifiednoise3d</a> | float | 

</details>


<details><summary>481 implementations removed</summary>

| Name | Node Category | Node Type |
| --- | --- | --- |
| IMPL_standard_surface_surfaceshader | LamaSSS | BSDF | 
| IMP_UsdUVTexture | UsdPrimvarReader | None | 
| IM_absval_color3_osl | absval | None | 
| IM_absval_color4_osl | absval | None | 
| IM_absval_float_osl | absval | None | 
| IM_absval_vector2_osl | absval | None | 
| IM_absval_vector3_osl | absval | None | 
| IM_absval_vector4_osl | absval | None | 
| IM_acos_float_osl | acos | None | 
| IM_add_color3FA_osl | add | None | 
| IM_add_color3_osl | add | None | 
| IM_add_color4FA_osl | add | None | 
| IM_add_color4_osl | add | None | 
| IM_add_displacementshader_osl | add | None | 
| IM_add_float_osl | add | None | 
| IM_add_matrix33FA_osl | add | None | 
| IM_add_matrix33_osl | add | None | 
| IM_add_matrix44FA_osl | add | None | 
| IM_add_matrix44_osl | add | None | 
| IM_add_surfaceshader_genglsl | add | None | 
| IM_add_surfaceshader_genmdl | add | None | 
| IM_add_surfaceshader_genosl | add | None | 
| IM_add_surfaceshader_osl | add | None | 
| IM_add_vector2FA_osl | add | None | 
| IM_add_vector2_osl | add | None | 
| IM_add_vector3FA_osl | add | None | 
| IM_add_vector3_osl | add | None | 
| IM_add_vector4FA_osl | add | None | 
| IM_add_vector4_osl | add | None | 
| IM_add_volumeshader_osl | add | None | 
| IM_ambientocclusion_float_osl | ambientocclusion | None | 
| IM_arrayappend_color3_color3array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_color3array_color3array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_color4_color4array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_color4array_color4array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_float_floatarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_floatarray_floatarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_integer_integerarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_integerarray_integerarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_string_stringarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_stringarray_stringarray_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector2_vector2array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector2array_vector2array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector3_vector3array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector3array_vector3array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector4_vector4array_osl | anisotropic_vdf | VDF | 
| IM_arrayappend_vector4array_vector4array_osl | anisotropic_vdf | VDF | 
| IM_asin_float_osl | asin | None | 
| IM_atan2_float_osl | atan2 | None | 
| IM_bitangent_vector3_osl | bitangent | None | 
| IM_blur_color3_osl | blur | color3 | 
| IM_blur_color4_osl | blur | color4 | 
| IM_blur_float_osl | blur | float | 
| IM_blur_vector2_osl | blur | vector2 | 
| IM_blur_vector3_osl | blur | vector3 | 
| IM_blur_vector4_osl | blur | vector4 | 
| IM_burn_color3_osl | burn | None | 
| IM_burn_color4_osl | burn | None | 
| IM_burn_float_osl | burn | None | 
| IM_ceil_color3_osl | ceil | None | 
| IM_ceil_color4_osl | ceil | None | 
| IM_ceil_float_osl | ceil | None | 
| IM_ceil_vector2_osl | ceil | None | 
| IM_ceil_vector3_osl | ceil | None | 
| IM_ceil_vector4_osl | ceil | None | 
| IM_cellnoise2d_float_osl | cellnoise2d | None | 
| IM_cellnoise3d_float_osl | cellnoise3d | None | 
| IM_clamp_color3FA_osl | clamp | None | 
| IM_clamp_color3_osl | clamp | None | 
| IM_clamp_color4FA_osl | clamp | None | 
| IM_clamp_color4_osl | clamp | None | 
| IM_clamp_float_osl | clamp | None | 
| IM_clamp_vector2FA_osl | clamp | None | 
| IM_clamp_vector2_osl | clamp | None | 
| IM_clamp_vector3FA_osl | clamp | None | 
| IM_clamp_vector3_osl | clamp | None | 
| IM_clamp_vector4FA_osl | clamp | None | 
| IM_clamp_vector4_osl | clamp | None | 
| IM_constant_boolean_osl | constant | None | 
| IM_constant_color3_osl | constant | None | 
| IM_constant_color4_osl | constant | None | 
| IM_constant_filename_osl | constant | None | 
| IM_constant_float_osl | constant | None | 
| IM_constant_integer_osl | constant | None | 
| IM_constant_matrix33_osl | constant | None | 
| IM_constant_matrix44_osl | constant | None | 
| IM_constant_string_osl | constant | None | 
| IM_constant_vector2_osl | constant | None | 
| IM_constant_vector3_osl | constant | None | 
| IM_constant_vector4_osl | constant | None | 
| IM_convert_color3_color4_osl | convert | color4 | 
| IM_convert_color3_vector3_osl | convert | vector3 | 
| IM_convert_color4_color3_osl | convert | color3 | 
| IM_convert_color4_vector4_osl | convert | vector4 | 
| IM_convert_float_color3_osl | convert | color3 | 
| IM_convert_float_color4_osl | convert | color4 | 
| IM_convert_float_vector2_osl | convert | vector2 | 
| IM_convert_float_vector3_osl | convert | vector3 | 
| IM_convert_float_vector4_osl | convert | vector4 | 
| IM_convert_vector3_color3_osl | convert | color3 | 
| IM_convert_vector4_color4_osl | convert | color4 | 
| IM_cos_float_osl | cos | None | 
| IM_crossproduct_vector3_osl | crossproduct | None | 
| IM_curveadjust_color3_osl | crossproduct | None | 
| IM_curveadjust_color4_osl | crossproduct | None | 
| IM_curveadjust_float_osl | crossproduct | None | 
| IM_curveadjust_vector2_osl | crossproduct | None | 
| IM_curveadjust_vector3_osl | crossproduct | None | 
| IM_curveadjust_vector4_osl | crossproduct | None | 
| IM_determinant_matrix33_osl | determinant | None | 
| IM_determinant_matrix44_osl | determinant | None | 
| IM_difference_color3_osl | difference | None | 
| IM_difference_color4_osl | difference | None | 
| IM_difference_float_osl | difference | None | 
| IM_disjointover_color4_osl | disjointover | None | 
| IM_divide_color3FA_osl | divide | None | 
| IM_divide_color3_osl | divide | None | 
| IM_divide_color4FA_osl | divide | None | 
| IM_divide_color4_osl | divide | None | 
| IM_divide_float_osl | divide | None | 
| IM_divide_matrix33_osl | divide | None | 
| IM_divide_matrix44_osl | divide | None | 
| IM_divide_vector2FA_osl | divide | None | 
| IM_divide_vector2_osl | divide | None | 
| IM_divide_vector3FA_osl | divide | None | 
| IM_divide_vector3_osl | divide | None | 
| IM_divide_vector4FA_osl | divide | None | 
| IM_divide_vector4_osl | divide | None | 
| IM_dodge_color3_osl | dodge | None | 
| IM_dodge_color4_osl | dodge | None | 
| IM_dodge_float_osl | dodge | None | 
| IM_dot_boolean_osl | dot | None | 
| IM_dot_color3_osl | dot | None | 
| IM_dot_color4_osl | dot | None | 
| IM_dot_displacementshader_osl | dot | None | 
| IM_dot_filename_osl | dot | None | 
| IM_dot_float_osl | dot | None | 
| IM_dot_integer_osl | dot | None | 
| IM_dot_lightshader_osl | dot | None | 
| IM_dot_matrix33_osl | dot | None | 
| IM_dot_matrix44_osl | dot | None | 
| IM_dot_string_osl | dot | None | 
| IM_dot_surfaceshader_osl | dot | None | 
| IM_dot_vector2_osl | dot | None | 
| IM_dot_vector3_osl | dot | None | 
| IM_dot_vector4_osl | dot | None | 
| IM_dot_volumeshader_osl | dot | None | 
| IM_dotproduct_vector2_osl | dotproduct | None | 
| IM_dotproduct_vector3_osl | dotproduct | None | 
| IM_dotproduct_vector4_osl | dotproduct | None | 
| IM_exp_float_osl | exp | None | 
| IM_floor_color3_osl | floor | None | 
| IM_floor_color4_osl | floor | None | 
| IM_floor_float_osl | floor | None | 
| IM_floor_vector2_osl | floor | None | 
| IM_floor_vector3_osl | floor | None | 
| IM_floor_vector4_osl | floor | None | 
| IM_fractal3d_color3FA_osl | fractal3d | None | 
| IM_fractal3d_color3_osl | fractal3d | None | 
| IM_fractal3d_color4FA_osl | fractal3d | None | 
| IM_fractal3d_color4_osl | fractal3d | None | 
| IM_fractal3d_float_osl | fractal3d | None | 
| IM_fractal3d_vector2FA_osl | fractal3d | None | 
| IM_fractal3d_vector2_osl | fractal3d | None | 
| IM_fractal3d_vector3FA_osl | fractal3d | None | 
| IM_fractal3d_vector3_osl | fractal3d | None | 
| IM_fractal3d_vector4FA_osl | fractal3d | None | 
| IM_fractal3d_vector4_osl | fractal3d | None | 
| IM_frame_float_osl | frame | None | 
| IM_geomcolor_color3_osl | geomcolor | None | 
| IM_geomcolor_color4_osl | geomcolor | None | 
| IM_geomcolor_float_osl | geomcolor | None | 
| IM_geompropvalue_boolean_osl | geompropvalue | None | 
| IM_geompropvalue_color3_osl | geompropvalue | None | 
| IM_geompropvalue_color4_osl | geompropvalue | None | 
| IM_geompropvalue_float_osl | geompropvalue | None | 
| IM_geompropvalue_integer_osl | geompropvalue | None | 
| IM_geompropvalue_string_osl | geompropvalue | None | 
| IM_geompropvalue_vector2_osl | geompropvalue | None | 
| IM_geompropvalue_vector3_osl | geompropvalue | None | 
| IM_geompropvalue_vector4_osl | geompropvalue | None | 
| IM_heighttonormal_vector3_osl | heighttonormal | None | 
| IM_hsvadjust_color3_osl | heighttonormal | None | 
| IM_hsvadjust_color4_osl | heighttonormal | None | 
| IM_hsvtorgb_color3_osl | hsvtorgb | None | 
| IM_hsvtorgb_color4_osl | hsvtorgb | None | 
| IM_image_color3_osl | image | None | 
| IM_image_color4_osl | image | None | 
| IM_image_float_osl | image | None | 
| IM_image_vector2_osl | image | None | 
| IM_image_vector3_osl | image | None | 
| IM_image_vector4_osl | image | None | 
| IM_in_color4_osl | in | None | 
| IM_inside_color3_osl | inside | None | 
| IM_inside_color4_osl | inside | None | 
| IM_inside_float_osl | inside | None | 
| IM_invert_color3FA_osl | invert | None | 
| IM_invert_color3_osl | invert | None | 
| IM_invert_color4FA_osl | invert | None | 
| IM_invert_color4_osl | invert | None | 
| IM_invert_float_osl | invert | None | 
| IM_invert_vector2FA_osl | invert | None | 
| IM_invert_vector2_osl | invert | None | 
| IM_invert_vector3FA_osl | invert | None | 
| IM_invert_vector3_osl | invert | None | 
| IM_invert_vector4FA_osl | invert | None | 
| IM_invert_vector4_osl | invert | None | 
| IM_ln_float_osl | ln | None | 
| IM_luminance_color3_osl | luminance | None | 
| IM_luminance_color4_osl | luminance | None | 
| IM_magnitude_vector2_osl | magnitude | None | 
| IM_magnitude_vector3_osl | magnitude | None | 
| IM_magnitude_vector4_osl | magnitude | None | 
| IM_mask_color4_osl | mask | None | 
| IM_matte_color4_osl | matte | None | 
| IM_max_color3FA_osl | max | None | 
| IM_max_color3_osl | max | None | 
| IM_max_color4FA_osl | max | None | 
| IM_max_color4_osl | max | None | 
| IM_max_float_osl | max | None | 
| IM_max_vector2FA_osl | max | None | 
| IM_max_vector2_osl | max | None | 
| IM_max_vector3FA_osl | max | None | 
| IM_max_vector3_osl | max | None | 
| IM_max_vector4FA_osl | max | None | 
| IM_max_vector4_osl | max | None | 
| IM_min_color3FA_osl | min | None | 
| IM_min_color3_osl | min | None | 
| IM_min_color4FA_osl | min | None | 
| IM_min_color4_osl | min | None | 
| IM_min_float_osl | min | None | 
| IM_min_vector2FA_osl | min | None | 
| IM_min_vector2_osl | min | None | 
| IM_min_vector3FA_osl | min | None | 
| IM_min_vector3_osl | min | None | 
| IM_min_vector4FA_osl | min | None | 
| IM_min_vector4_osl | min | None | 
| IM_minus_color3_osl | minus | None | 
| IM_minus_color4_osl | minus | None | 
| IM_minus_float_osl | minus | None | 
| IM_mix_color3_osl | mix | None | 
| IM_mix_color4_osl | mix | None | 
| IM_mix_displacementshader_osl | mix | displacementshader | 
| IM_mix_float_osl | mix | None | 
| IM_mix_surfaceshader_osl | mix | None | 
| IM_mix_vector2_osl | mix | None | 
| IM_mix_vector3_osl | mix | None | 
| IM_mix_vector4_osl | mix | None | 
| IM_mix_volumeshader_osl | mix | volumeshader | 
| IM_modulo_color3FA_osl | modulo | None | 
| IM_modulo_color3_osl | modulo | None | 
| IM_modulo_color4FA_osl | modulo | None | 
| IM_modulo_color4_osl | modulo | None | 
| IM_modulo_float_osl | modulo | None | 
| IM_modulo_vector2FA_osl | modulo | None | 
| IM_modulo_vector2_osl | modulo | None | 
| IM_modulo_vector3FA_osl | modulo | None | 
| IM_modulo_vector3_osl | modulo | None | 
| IM_modulo_vector4FA_osl | modulo | None | 
| IM_modulo_vector4_osl | modulo | None | 
| IM_multiply_color3FA_osl | multiply | None | 
| IM_multiply_color3_osl | multiply | None | 
| IM_multiply_color4FA_osl | multiply | None | 
| IM_multiply_color4_osl | multiply | None | 
| IM_multiply_float_osl | multiply | None | 
| IM_multiply_matrix33_osl | multiply | None | 
| IM_multiply_matrix44_osl | multiply | None | 
| IM_multiply_surfaceshaderC_genglsl | multiply | None | 
| IM_multiply_surfaceshaderC_genmdl | multiply | None | 
| IM_multiply_surfaceshaderC_genosl | multiply | None | 
| IM_multiply_surfaceshaderC_osl | multiply | None | 
| IM_multiply_surfaceshaderF_genglsl | multiply | None | 
| IM_multiply_surfaceshaderF_genmdl | multiply | None | 
| IM_multiply_surfaceshaderF_genosl | multiply | None | 
| IM_multiply_surfaceshaderF_osl | multiply | None | 
| IM_multiply_vector2FA_osl | multiply | None | 
| IM_multiply_vector2_osl | multiply | None | 
| IM_multiply_vector3FA_osl | multiply | None | 
| IM_multiply_vector3_osl | multiply | None | 
| IM_multiply_vector4FA_osl | multiply | None | 
| IM_multiply_vector4_osl | multiply | None | 
| IM_noise2d_color3FA_osl | noise2d | None | 
| IM_noise2d_color3_osl | noise2d | None | 
| IM_noise2d_color4FA_osl | noise2d | None | 
| IM_noise2d_color4_osl | noise2d | None | 
| IM_noise2d_float_osl | noise2d | None | 
| IM_noise2d_vector2FA_osl | noise2d | None | 
| IM_noise2d_vector2_osl | noise2d | None | 
| IM_noise2d_vector3FA_osl | noise2d | None | 
| IM_noise2d_vector3_osl | noise2d | None | 
| IM_noise2d_vector4FA_osl | noise2d | None | 
| IM_noise2d_vector4_osl | noise2d | None | 
| IM_noise3d_color3FA_osl | noise3d | None | 
| IM_noise3d_color3_osl | noise3d | None | 
| IM_noise3d_color4FA_osl | noise3d | None | 
| IM_noise3d_color4_osl | noise3d | None | 
| IM_noise3d_float_osl | noise3d | None | 
| IM_noise3d_vector2FA_osl | noise3d | None | 
| IM_noise3d_vector2_osl | noise3d | None | 
| IM_noise3d_vector3FA_osl | noise3d | None | 
| IM_noise3d_vector3_osl | noise3d | None | 
| IM_noise3d_vector4FA_osl | noise3d | None | 
| IM_noise3d_vector4_osl | noise3d | None | 
| IM_normal_vector3_osl | normal | None | 
| IM_normalize_vector2_osl | normalize | None | 
| IM_normalize_vector3_osl | normalize | None | 
| IM_normalize_vector4_osl | normalize | None | 
| IM_out_color4_osl | out | None | 
| IM_outside_color3_osl | outside | None | 
| IM_outside_color4_osl | outside | None | 
| IM_outside_float_osl | outside | None | 
| IM_over_color4_osl | over | None | 
| IM_overlay_color3_osl | overlay | None | 
| IM_overlay_color4_osl | overlay | None | 
| IM_overlay_float_osl | overlay | None | 
| IM_plus_color3_osl | plus | None | 
| IM_plus_color4_osl | plus | None | 
| IM_plus_float_osl | plus | None | 
| IM_position_vector3_osl | position | None | 
| IM_power_color3FA_osl | power | None | 
| IM_power_color3_osl | power | None | 
| IM_power_color4FA_osl | power | None | 
| IM_power_color4_osl | power | None | 
| IM_power_float_osl | power | None | 
| IM_power_vector2FA_osl | power | None | 
| IM_power_vector2_osl | power | None | 
| IM_power_vector3FA_osl | power | None | 
| IM_power_vector3_osl | power | None | 
| IM_power_vector4FA_osl | power | None | 
| IM_power_vector4_osl | power | None | 
| IM_premult_color4_osl | premult | None | 
| IM_ramp4_color3_osl | premult | None | 
| IM_ramp4_color4_osl | premult | None | 
| IM_ramp4_float_osl | premult | None | 
| IM_ramp4_vector2_osl | premult | None | 
| IM_ramp4_vector3_osl | premult | None | 
| IM_ramp4_vector4_osl | premult | None | 
| IM_ramplr_color3_osl | ramplr | None | 
| IM_ramplr_color4_osl | ramplr | None | 
| IM_ramplr_float_osl | ramplr | None | 
| IM_ramplr_vector2_osl | ramplr | None | 
| IM_ramplr_vector3_osl | ramplr | None | 
| IM_ramplr_vector4_osl | ramplr | None | 
| IM_ramptb_color3_osl | ramptb | None | 
| IM_ramptb_color4_osl | ramptb | None | 
| IM_ramptb_float_osl | ramptb | None | 
| IM_ramptb_vector2_osl | ramptb | None | 
| IM_ramptb_vector3_osl | ramptb | None | 
| IM_ramptb_vector4_osl | ramptb | None | 
| IM_remap_color3FA_osl | remap | None | 
| IM_remap_color3_osl | remap | None | 
| IM_remap_color4FA_osl | remap | None | 
| IM_remap_color4_osl | remap | None | 
| IM_remap_float_osl | remap | None | 
| IM_remap_vector2FA_osl | remap | None | 
| IM_remap_vector2_osl | remap | None | 
| IM_remap_vector3FA_osl | remap | None | 
| IM_remap_vector3_osl | remap | None | 
| IM_remap_vector4FA_osl | remap | None | 
| IM_remap_vector4_osl | remap | None | 
| IM_rgbtohsv_color3_osl | rgbtohsv | None | 
| IM_rgbtohsv_color4_osl | rgbtohsv | None | 
| IM_screen_color3_osl | screen | None | 
| IM_screen_color4_osl | screen | None | 
| IM_screen_float_osl | screen | None | 
| IM_sign_color3_osl | sign | None | 
| IM_sign_color4_osl | sign | None | 
| IM_sign_float_osl | sign | None | 
| IM_sign_vector2_osl | sign | None | 
| IM_sign_vector3_osl | sign | None | 
| IM_sign_vector4_osl | sign | None | 
| IM_sin_float_osl | sin | None | 
| IM_smoothstep_color3FA_osl | smoothstep | None | 
| IM_smoothstep_color3_osl | smoothstep | None | 
| IM_smoothstep_color4FA_osl | smoothstep | None | 
| IM_smoothstep_color4_osl | smoothstep | None | 
| IM_smoothstep_float_osl | smoothstep | None | 
| IM_smoothstep_vector2FA_osl | smoothstep | None | 
| IM_smoothstep_vector2_osl | smoothstep | None | 
| IM_smoothstep_vector3FA_osl | smoothstep | None | 
| IM_smoothstep_vector3_osl | smoothstep | None | 
| IM_smoothstep_vector4FA_osl | smoothstep | None | 
| IM_smoothstep_vector4_osl | smoothstep | None | 
| IM_splitlr_color3_osl | splitlr | None | 
| IM_splitlr_color4_osl | splitlr | None | 
| IM_splitlr_float_osl | splitlr | None | 
| IM_splitlr_vector2_osl | splitlr | None | 
| IM_splitlr_vector3_osl | splitlr | None | 
| IM_splitlr_vector4_osl | splitlr | None | 
| IM_splittb_color3_osl | splittb | None | 
| IM_splittb_color4_osl | splittb | None | 
| IM_splittb_float_osl | splittb | None | 
| IM_splittb_vector2_osl | splittb | None | 
| IM_splittb_vector3_osl | splittb | None | 
| IM_splittb_vector4_osl | splittb | None | 
| IM_sqrt_float_osl | sqrt | None | 
| IM_subtract_color3FA_osl | subtract | None | 
| IM_subtract_color3_osl | subtract | None | 
| IM_subtract_color4FA_osl | subtract | None | 
| IM_subtract_color4_osl | subtract | None | 
| IM_subtract_float_osl | subtract | None | 
| IM_subtract_matrix33FA_osl | subtract | None | 
| IM_subtract_matrix33_osl | subtract | None | 
| IM_subtract_matrix44FA_osl | subtract | None | 
| IM_subtract_matrix44_osl | subtract | None | 
| IM_subtract_vector2FA_osl | subtract | None | 
| IM_subtract_vector2_osl | subtract | None | 
| IM_subtract_vector3FA_osl | subtract | None | 
| IM_subtract_vector3_osl | subtract | None | 
| IM_subtract_vector4FA_osl | subtract | None | 
| IM_subtract_vector4_osl | subtract | None | 
| IM_switch_color3I_osl | switch | color3 | 
| IM_switch_color3_osl | switch | color3 | 
| IM_switch_color4I_osl | switch | color4 | 
| IM_switch_color4_osl | switch | color4 | 
| IM_switch_floatI_osl | switch | float | 
| IM_switch_float_osl | switch | float | 
| IM_switch_vector2I_osl | switch | vector2 | 
| IM_switch_vector2_osl | switch | vector2 | 
| IM_switch_vector3I_osl | switch | vector3 | 
| IM_switch_vector3_osl | switch | vector3 | 
| IM_switch_vector4I_osl | switch | vector4 | 
| IM_switch_vector4_osl | switch | vector4 | 
| IM_swizzle_color3_color3_osl | swizzle | color3 | 
| IM_swizzle_color3_color4_osl | swizzle | color4 | 
| IM_swizzle_color3_float_osl | swizzle | float | 
| IM_swizzle_color3_vector2_osl | swizzle | vector2 | 
| IM_swizzle_color3_vector3_osl | swizzle | vector3 | 
| IM_swizzle_color3_vector4_osl | swizzle | vector4 | 
| IM_swizzle_color4_color3_osl | swizzle | color3 | 
| IM_swizzle_color4_color4_osl | swizzle | color4 | 
| IM_swizzle_color4_float_osl | swizzle | float | 
| IM_swizzle_color4_vector2_osl | swizzle | vector2 | 
| IM_swizzle_color4_vector3_osl | swizzle | vector3 | 
| IM_swizzle_color4_vector4_osl | swizzle | vector4 | 
| IM_swizzle_float_color3_osl | swizzle | color3 | 
| IM_swizzle_float_color4_osl | swizzle | color4 | 
| IM_swizzle_float_vector2_osl | swizzle | vector2 | 
| IM_swizzle_float_vector3_osl | swizzle | vector3 | 
| IM_swizzle_float_vector4_osl | swizzle | vector4 | 
| IM_swizzle_vector2_color3_osl | swizzle | color3 | 
| IM_swizzle_vector2_color4_osl | swizzle | color4 | 
| IM_swizzle_vector2_float_osl | swizzle | float | 
| IM_swizzle_vector2_vector2_osl | swizzle | vector2 | 
| IM_swizzle_vector2_vector3_osl | swizzle | vector3 | 
| IM_swizzle_vector2_vector4_osl | swizzle | vector4 | 
| IM_swizzle_vector3_color3_osl | swizzle | color3 | 
| IM_swizzle_vector3_color4_osl | swizzle | color4 | 
| IM_swizzle_vector3_float_osl | swizzle | float | 
| IM_swizzle_vector3_vector2_osl | swizzle | vector2 | 
| IM_swizzle_vector3_vector3_osl | swizzle | vector3 | 
| IM_swizzle_vector3_vector4_osl | swizzle | vector4 | 
| IM_swizzle_vector4_color3_osl | swizzle | color3 | 
| IM_swizzle_vector4_color4_osl | swizzle | color4 | 
| IM_swizzle_vector4_float_osl | swizzle | float | 
| IM_swizzle_vector4_vector2_osl | swizzle | vector2 | 
| IM_swizzle_vector4_vector3_osl | swizzle | vector3 | 
| IM_swizzle_vector4_vector4_osl | swizzle | vector4 | 
| IM_tan_float_osl | tan | None | 
| IM_tangent_vector3_osl | tangent | None | 
| IM_texcoord_vector2_osl | texcoord | None | 
| IM_texcoord_vector3_osl | texcoord | None | 
| IM_tiledimage_color3_osl | thin_surface | None | 
| IM_tiledimage_color4_osl | thin_surface | None | 
| IM_tiledimage_float_osl | thin_surface | None | 
| IM_tiledimage_vector2_osl | thin_surface | None | 
| IM_tiledimage_vector3_osl | thin_surface | None | 
| IM_tiledimage_vector4_osl | thin_surface | None | 
| IM_time_float_osl | time | None | 
| IM_transformnormal_vector3_osl | transformnormal | None | 
| IM_transformpoint_vector3_osl | transformpoint | None | 
| IM_transformvector_vector3_osl | transformvector | None | 
| IM_transpose_matrix33_osl | transpose | None | 
| IM_transpose_matrix44_osl | transpose | None | 
| IM_triplanarprojection_color3_osl | transpose | None | 
| IM_triplanarprojection_color4_osl | transpose | None | 
| IM_triplanarprojection_float_osl | transpose | None | 
| IM_triplanarprojection_vector2_osl | transpose | None | 
| IM_triplanarprojection_vector3_osl | transpose | None | 
| IM_triplanarprojection_vector4_osl | transpose | None | 
| IM_unpremult_color4_osl | unpremult | None | 

</details>


<details><summary>928 implementations modified</summary>

| Name | Change |
| --- | --- |
| <a href="../documents/definitions/UsdPreviewSurface.html">IMP_UsdPreviewSurface_surfaceshader</a> | - Number of children on: `IMP_UsdPreviewSurface_surfaceshader` changed from ( 28 ) to ( 33 )<br>- divide `/div_ior` removed from `IMP_UsdPreviewSurface_surfaceshader`<br>- clamp `/opacity_clamped` removed from `IMP_UsdPreviewSurface_surfaceshader`<br>- subtract `/inverse_metalness` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- mix `/diffuse_bsdf_weight` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- divide `/R` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- multiply `/R_sq` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- mix `/specular_color_metallic` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- multiply `/specular_color_metallic_R_sq` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- convert `/coat_F0` add to `IMP_UsdPreviewSurface_surfaceshader`<br>- attribute: `value`=( `1` ) removed from: `weight`<br>- attribute: `nodename`=( `diffuse_bsdf_weight` ) added to: `weight`<br>- Number of children on: `transmission_bsdf` changed from ( 5 ) to ( 6 )<br>- input `/roughness` added to `IMP_UsdPreviewSurface_surfaceshader/transmission_bsdf`. Value=( 0.0, 0.0 )<br>- Category changed on: `F0` from `mix` to `multiply`<br>  - attribute: `type` changed on `F0` from `( float )` to `( color3 )`<br>- Number of children on: `F0` changed from ( 2 ) to ( 3 )<br>- input `/in1` removed from `IMP_UsdPreviewSurface_surfaceshader/F0`. Value=(  )<br>- input `/in2` removed from `IMP_UsdPreviewSurface_surfaceshader/F0`. Value=(  )<br>- input `/fg` added to `IMP_UsdPreviewSurface_surfaceshader/F0`. Value=(  )<br>- input `/bg` added to `IMP_UsdPreviewSurface_surfaceshader/F0`. Value=(  )<br>- input `/mix` added to `IMP_UsdPreviewSurface_surfaceshader/F0`. Value=(  )<br>- attribute: `channels`=( `rrr` ) removed from: `color0`<br>- attribute: `value`=( `1, 1, 1` ) removed from: `color90`<br>- attribute: `nodename`=( `specular_color_metallic` ) added to: `color90`<br>- Category changed on: `coat_dielectric_bsdf` from `generalized_schlick_bsdf` to `dielectric_bsdf`<br>- input `/tint` removed from `IMP_UsdPreviewSurface_surfaceshader/coat_dielectric_bsdf`. Value=( 1, 1, 1 )<br>- input `/ior` removed from `IMP_UsdPreviewSurface_surfaceshader/coat_dielectric_bsdf`. Value=( 1.5 )<br>- input `/color0` added to `IMP_UsdPreviewSurface_surfaceshader/coat_dielectric_bsdf`. Value=(  )<br>- input `/color90` added to `IMP_UsdPreviewSurface_surfaceshader/coat_dielectric_bsdf`. Value=( 1, 1, 1 )<br>- Category changed on: `cutout_opacity` from `ifgreatereq` to `ifgreater`<br>- attribute: `nodename`=( `opacity_clamped` ) removed from: `value1`<br>- attribute: `interfacename`=( `opacity` ) added to: `value1`<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_boolean</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_boolean/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_float</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_float/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_integer</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_integer/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_string</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_string/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_vector2</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_vector2/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_vector3</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_vector3/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/UsdPrimvarReader.html">IMP_UsdPrimvarReader_vector4</a> | - Number of children on: `primvar` changed from ( 1 ) to ( 2 )<br>- input `/default` added to `IMP_UsdPrimvarReader_vector4/primvar`. Value=(  )<br>|
| <a href="../documents/definitions/absval.html">IM_absval_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_color3_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_color3_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_color3_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_color3_genosl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_color4_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_color4_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_color4_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_color4_genosl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_float_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_float_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_float_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_float_genosl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_vector2_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector2_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_vector2_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector2_genosl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_vector3_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector3_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_vector3_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector3_genosl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_absval.inline` ) removed from: `IM_absval_vector4_genglsl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector4_genglsl`<br>|
| <a href="../documents/definitions/absval.html">IM_absval_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_absval.inline` ) removed from: `IM_absval_vector4_genosl`<br>- attribute: `sourcecode`=( `abs({{in}})` ) added to: `IM_absval_vector4_genosl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_acos.inline` ) removed from: `IM_acos_float_genglsl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_float_genglsl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_acos.inline` ) removed from: `IM_acos_float_genosl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_float_genosl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_acos.inline` ) removed from: `IM_acos_vector2_genglsl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector2_genglsl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_acos.inline` ) removed from: `IM_acos_vector2_genosl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector2_genosl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_acos.inline` ) removed from: `IM_acos_vector3_genglsl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector3_genglsl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_acos.inline` ) removed from: `IM_acos_vector3_genosl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector3_genosl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_acos.inline` ) removed from: `IM_acos_vector4_genglsl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector4_genglsl`<br>|
| <a href="../documents/definitions/acos.html">IM_acos_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_acos.inline` ) removed from: `IM_acos_vector4_genosl`<br>- attribute: `sourcecode`=( `acos({{in}})` ) added to: `IM_acos_vector4_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_bsdf_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_add_bsdf.glsl` ) removed from: `IM_add_bsdf_genglsl`<br>- attribute: `function`=( `mx_add_bsdf` ) removed from: `IM_add_bsdf_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_bsdf_genmdl</a> |   - attribute: `sourcecode` changed on `IM_add_bsdf_genmdl` from `( {{in1}} )` to `( mx::pbrlib::mx_add_bsdf(mxp_in1:{{in1}}, mxp_in2:{{in2}}) )`<br>|
| <a href="../documents/definitions/add.html">IM_add_bsdf_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_add.inline` ) removed from: `IM_add_bsdf_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color3FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_color3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color3FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color3_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_color3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color3_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color4FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_color4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color4FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color4_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_color4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_color4_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_edf_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_add_edf.glsl` ) removed from: `IM_add_edf_genglsl`<br>- attribute: `function`=( `mx_add_edf` ) removed from: `IM_add_edf_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_edf_genmdl</a> |   - attribute: `sourcecode` changed on `IM_add_edf_genmdl` from `( {{in2}} )` to `( mx::pbrlib::mx_add_edf(mxp_in1:{{in1}}, mxp_in2:{{in2}}) )`<br>|
| <a href="../documents/definitions/add.html">IM_add_edf_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_add.inline` ) removed from: `IM_add_edf_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_float_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_float_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_float_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_float_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix33FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_matrix33FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_matrix33FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix33FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_matrix33FA_genosl`<br>- attribute: `sourcecode`=( `mx_add({{in1}}, {{in2}})` ) added to: `IM_add_matrix33FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_matrix33_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_matrix33_genosl`<br>- attribute: `sourcecode`=( `mx_add({{in1}}, {{in2}})` ) added to: `IM_add_matrix33_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix44FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_matrix44FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_matrix44FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix44FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_matrix44FA_genosl`<br>- attribute: `sourcecode`=( `mx_add({{in1}}, {{in2}})` ) added to: `IM_add_matrix44FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_matrix44_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_matrix44_genosl`<br>- attribute: `sourcecode`=( `mx_add({{in1}}, {{in2}})` ) added to: `IM_add_matrix44_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector2FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector2FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector2_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector2_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector2_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector3FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector3_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector3_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector4FA_genosl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_add.inline` ) removed from: `IM_add_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector4_genglsl`<br>|
| <a href="../documents/definitions/add.html">IM_add_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_add.inline` ) removed from: `IM_add_vector4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} + {{in2}}` ) added to: `IM_add_vector4_genosl`<br>|
| <a href="../documents/definitions/ambientocclusion.html">IM_ambientocclusion_float_genosl</a> |   - attribute: `file` changed on `IM_ambientocclusion_float_genosl` from `( stdlib/genosl/mx_ambientocclusion_float.osl )` to `( mx_ambientocclusion_float.osl )`<br>|
| <a href="../documents/definitions/artistic_ior.html">IM_artistic_ior_genglsl</a> |   - attribute: `file` changed on `IM_artistic_ior_genglsl` from `( pbrlib/genglsl/mx_artistic_ior.glsl )` to `( mx_artistic_ior.glsl )`<br>|
| <a href="../documents/definitions/artistic_ior.html">IM_artistic_ior_genosl</a> |   - attribute: `file` changed on `IM_artistic_ior_genosl` from `( pbrlib/genosl/mx_artistic_ior.osl )` to `( mx_artistic_ior.osl )`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_asin.inline` ) removed from: `IM_asin_float_genglsl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_float_genglsl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_asin.inline` ) removed from: `IM_asin_float_genosl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_float_genosl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_asin.inline` ) removed from: `IM_asin_vector2_genglsl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector2_genglsl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_asin.inline` ) removed from: `IM_asin_vector2_genosl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector2_genosl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_asin.inline` ) removed from: `IM_asin_vector3_genglsl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector3_genglsl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_asin.inline` ) removed from: `IM_asin_vector3_genosl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector3_genosl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_asin.inline` ) removed from: `IM_asin_vector4_genglsl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector4_genglsl`<br>|
| <a href="../documents/definitions/asin.html">IM_asin_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_asin.inline` ) removed from: `IM_asin_vector4_genosl`<br>- attribute: `sourcecode`=( `asin({{in}})` ) added to: `IM_asin_vector4_genosl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_atan2.inline` ) removed from: `IM_atan2_float_genglsl`<br>- attribute: `sourcecode`=( `atan({{in1}}, {{in2}})` ) added to: `IM_atan2_float_genglsl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_atan2.inline` ) removed from: `IM_atan2_float_genosl`<br>- attribute: `sourcecode`=( `atan2({{in1}},{{in2}})` ) added to: `IM_atan2_float_genosl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_atan2.inline` ) removed from: `IM_atan2_vector2_genglsl`<br>- attribute: `sourcecode`=( `atan({{in1}}, {{in2}})` ) added to: `IM_atan2_vector2_genglsl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_atan2.inline` ) removed from: `IM_atan2_vector2_genosl`<br>- attribute: `sourcecode`=( `atan2({{in1}},{{in2}})` ) added to: `IM_atan2_vector2_genosl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_atan2.inline` ) removed from: `IM_atan2_vector3_genglsl`<br>- attribute: `sourcecode`=( `atan({{in1}}, {{in2}})` ) added to: `IM_atan2_vector3_genglsl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_atan2.inline` ) removed from: `IM_atan2_vector3_genosl`<br>- attribute: `sourcecode`=( `atan2({{in1}},{{in2}})` ) added to: `IM_atan2_vector3_genosl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_atan2.inline` ) removed from: `IM_atan2_vector4_genglsl`<br>- attribute: `sourcecode`=( `atan({{in1}}, {{in2}})` ) added to: `IM_atan2_vector4_genglsl`<br>|
| <a href="../documents/definitions/atan2.html">IM_atan2_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_atan2.inline` ) removed from: `IM_atan2_vector4_genosl`<br>- attribute: `sourcecode`=( `atan2({{in1}},{{in2}})` ) added to: `IM_atan2_vector4_genosl`<br>|
| <a href="../documents/definitions/bitangent.html">IM_bitangent_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_bitangent_vector3.inline` ) removed from: `IM_bitangent_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{space}}, normalize(dPdv))` ) added to: `IM_bitangent_vector3_genosl`<br>|
| <a href="../documents/definitions/burley_diffuse_bsdf.html">IM_burley_diffuse_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_burley_diffuse_bsdf_genglsl` from `( pbrlib/genglsl/mx_burley_diffuse_bsdf.glsl )` to `( mx_burley_diffuse_bsdf.glsl )`<br>|
| <a href="../documents/definitions/burley_diffuse_bsdf.html">IM_burley_diffuse_bsdf_genosl</a> |   - attribute: `file` changed on `IM_burley_diffuse_bsdf_genosl` from `( pbrlib/genosl/mx_burley_diffuse_bsdf.osl )` to `( legacy/mx_burley_diffuse_bsdf.osl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_color3_genglsl</a> |   - attribute: `file` changed on `IM_burn_color3_genglsl` from `( stdlib/genglsl/mx_burn_color3.glsl )` to `( mx_burn_color3.glsl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_color3_genosl</a> |   - attribute: `file` changed on `IM_burn_color3_genosl` from `( stdlib/genosl/mx_burn_color3.osl )` to `( mx_burn_color3.osl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_color4_genglsl</a> |   - attribute: `file` changed on `IM_burn_color4_genglsl` from `( stdlib/genglsl/mx_burn_color4.glsl )` to `( mx_burn_color4.glsl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_color4_genosl</a> |   - attribute: `file` changed on `IM_burn_color4_genosl` from `( stdlib/genosl/mx_burn_color4.osl )` to `( mx_burn_color4.osl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_float_genglsl</a> |   - attribute: `file` changed on `IM_burn_float_genglsl` from `( stdlib/genglsl/mx_burn_float.glsl )` to `( mx_burn_float.glsl )`<br>|
| <a href="../documents/definitions/burn.html">IM_burn_float_genosl</a> |   - attribute: `file` changed on `IM_burn_float_genosl` from `( stdlib/genosl/mx_burn_float.osl )` to `( mx_burn_float.osl )`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_color3_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_color3_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_color3_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_color3_genosl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_color4_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_color4_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_color4_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_color4_genosl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_float_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_float_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_float_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_float_genosl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_vector2_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector2_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_vector2_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector2_genosl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_vector3_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector3_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_vector3_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector3_genosl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ceil.inline` ) removed from: `IM_ceil_vector4_genglsl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector4_genglsl`<br>|
| <a href="../documents/definitions/ceil.html">IM_ceil_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ceil.inline` ) removed from: `IM_ceil_vector4_genosl`<br>- attribute: `sourcecode`=( `ceil({{in}})` ) added to: `IM_ceil_vector4_genosl`<br>|
| <a href="../documents/definitions/cellnoise2d.html">IM_cellnoise2d_float_genglsl</a> |   - attribute: `file` changed on `IM_cellnoise2d_float_genglsl` from `( stdlib/genglsl/mx_cellnoise2d_float.glsl )` to `( mx_cellnoise2d_float.glsl )`<br>|
| <a href="../documents/definitions/cellnoise2d.html">IM_cellnoise2d_float_genosl</a> |   - attribute: `file` changed on `IM_cellnoise2d_float_genosl` from `( stdlib/genosl/mx_cellnoise2d_float.osl )` to `( mx_cellnoise2d_float.osl )`<br>|
| <a href="../documents/definitions/cellnoise3d.html">IM_cellnoise3d_float_genglsl</a> |   - attribute: `file` changed on `IM_cellnoise3d_float_genglsl` from `( stdlib/genglsl/mx_cellnoise3d_float.glsl )` to `( mx_cellnoise3d_float.glsl )`<br>|
| <a href="../documents/definitions/cellnoise3d.html">IM_cellnoise3d_float_genosl</a> |   - attribute: `file` changed on `IM_cellnoise3d_float_genosl` from `( stdlib/genosl/mx_cellnoise3d_float.osl )` to `( mx_cellnoise3d_float.osl )`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_color3FA_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color3FA_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_color3FA_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color3FA_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_color3_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color3_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_color3_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color3_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_color4FA_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color4FA_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_color4FA_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color4FA_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_color4_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color4_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_color4_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_color4_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_float_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_float_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_float_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_float_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector2FA_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector2FA_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector2_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector2_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector2_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector2_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector3FA_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector3FA_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector3_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector3_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector3_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector3_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector4FA_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector4FA_genosl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_clamp.inline` ) removed from: `IM_clamp_vector4_genglsl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector4_genglsl`<br>|
| <a href="../documents/definitions/clamp.html">IM_clamp_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_clamp.inline` ) removed from: `IM_clamp_vector4_genosl`<br>- attribute: `sourcecode`=( `clamp({{in}}, {{low}}, {{high}})` ) added to: `IM_clamp_vector4_genosl`<br>|
| <a href="../documents/definitions/conductor_bsdf.html">IM_conductor_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_conductor_bsdf_genglsl` from `( pbrlib/genglsl/mx_conductor_bsdf.glsl )` to `( mx_conductor_bsdf.glsl )`<br>|
| <a href="../documents/definitions/conductor_bsdf.html">IM_conductor_bsdf_genmdl</a> |   - attribute: `sourcecode` changed on `IM_conductor_bsdf_genmdl` from `( mx::pbrlib::mx_conductor_bsdf(mxp_weight:{{weight}}, mxp_ior:{{ior}}, mxp_extinction:{{extinction}}, mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}) )` to `( mx::pbrlib::mx_conductor_bsdf(mxp_weight:{{weight}}, mxp_ior:{{ior}}, mxp_extinction:{{extinction}}, mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}, mxp_distribution:{{distribution}}, mxp_thinfilm_thickness:{{thinfilm_thickness}}, mxp_thinfilm_ior:{{thinfilm_ior}}) )`<br>|
| <a href="../documents/definitions/conductor_bsdf.html">IM_conductor_bsdf_genosl</a> |   - attribute: `file` changed on `IM_conductor_bsdf_genosl` from `( pbrlib/genosl/mx_conductor_bsdf.osl )` to `( legacy/mx_conductor_bsdf.osl )`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_boolean_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_boolean_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_boolean_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_boolean_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_boolean_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_boolean_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_color3_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_color3_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_color3_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_color3_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_color4_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_color4_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_color4_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_color4_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_filename_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_filename_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_filename_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_filename_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_filename_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_filename_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_float_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_float_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_float_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_float_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_integer_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_integer_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_integer_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_integer_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_integer_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_integer_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_matrix33_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_matrix33_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_matrix33_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_matrix44_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_matrix44_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_matrix44_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_string_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_string_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_string_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_string_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_string_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_string_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector2_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_vector2_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector2_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector3_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_vector3_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector3_genosl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_constant.inline` ) removed from: `IM_constant_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector4_genglsl`<br>|
| <a href="../documents/definitions/constant.html">IM_constant_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_constant.inline` ) removed from: `IM_constant_vector4_genosl`<br>- attribute: `sourcecode`=( `{{value}}` ) added to: `IM_constant_vector4_genosl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_cos.inline` ) removed from: `IM_cos_float_genglsl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_float_genglsl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_cos.inline` ) removed from: `IM_cos_float_genosl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_float_genosl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_cos.inline` ) removed from: `IM_cos_vector2_genglsl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector2_genglsl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_cos.inline` ) removed from: `IM_cos_vector2_genosl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector2_genosl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_cos.inline` ) removed from: `IM_cos_vector3_genglsl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector3_genglsl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_cos.inline` ) removed from: `IM_cos_vector3_genosl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector3_genosl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_cos.inline` ) removed from: `IM_cos_vector4_genglsl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector4_genglsl`<br>|
| <a href="../documents/definitions/cos.html">IM_cos_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_cos.inline` ) removed from: `IM_cos_vector4_genosl`<br>- attribute: `sourcecode`=( `cos({{in}})` ) added to: `IM_cos_vector4_genosl`<br>|
| <a href="../documents/definitions/crossproduct.html">IM_crossproduct_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_crossproduct.inline` ) removed from: `IM_crossproduct_vector3_genglsl`<br>- attribute: `sourcecode`=( `cross({{in1}}, {{in2}})` ) added to: `IM_crossproduct_vector3_genglsl`<br>|
| <a href="../documents/definitions/crossproduct.html">IM_crossproduct_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_crossproduct.inline` ) removed from: `IM_crossproduct_vector3_genosl`<br>- attribute: `sourcecode`=( `cross({{in1}}, {{in2}})` ) added to: `IM_crossproduct_vector3_genosl`<br>|
| <a href="../documents/definitions/determinant.html">IM_determinant_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_determinant.inline` ) removed from: `IM_determinant_matrix33_genglsl`<br>- attribute: `sourcecode`=( `determinant({{in}})` ) added to: `IM_determinant_matrix33_genglsl`<br>|
| <a href="../documents/definitions/determinant.html">IM_determinant_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_determinant.inline` ) removed from: `IM_determinant_matrix33_genosl`<br>- attribute: `sourcecode`=( `determinant({{in}})` ) added to: `IM_determinant_matrix33_genosl`<br>|
| <a href="../documents/definitions/determinant.html">IM_determinant_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_determinant.inline` ) removed from: `IM_determinant_matrix44_genglsl`<br>- attribute: `sourcecode`=( `determinant({{in}})` ) added to: `IM_determinant_matrix44_genglsl`<br>|
| <a href="../documents/definitions/determinant.html">IM_determinant_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_determinant.inline` ) removed from: `IM_determinant_matrix44_genosl`<br>- attribute: `sourcecode`=( `determinant({{in}})` ) added to: `IM_determinant_matrix44_genosl`<br>|
| <a href="../documents/definitions/dielectric_bsdf.html">IM_dielectric_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_dielectric_bsdf_genglsl` from `( pbrlib/genglsl/mx_dielectric_bsdf.glsl )` to `( mx_dielectric_bsdf.glsl )`<br>|
| <a href="../documents/definitions/dielectric_bsdf.html">IM_dielectric_bsdf_genmdl</a> |   - attribute: `sourcecode` changed on `IM_dielectric_bsdf_genmdl` from `( mx::pbrlib::mx_dielectric_bsdf(mxp_weight:{{weight}}, mxp_tint:{{tint}}, mxp_ior:{{ior}}, mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}, mxp_scatter_mode:{{scatter_mode}}, mxp_base:{{base}}) )` to `( mx::pbrlib::mx_dielectric_bsdf(mxp_weight:{{weight}}, mxp_tint:{{tint}}, mxp_ior:{{ior}}, mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}, mxp_distribution:{{distribution}}, mxp_scatter_mode:{{scatter_mode}}, mxp_base:{{base}}, mxp_thinfilm_thickness:{{thinfilm_thickness}}, mxp_thinfilm_ior:{{thinfilm_ior}}) )`<br>|
| <a href="../documents/definitions/dielectric_bsdf.html">IM_dielectric_bsdf_genosl</a> |   - attribute: `file` changed on `IM_dielectric_bsdf_genosl` from `( pbrlib/genosl/mx_dielectric_bsdf.osl )` to `( legacy/mx_dielectric_bsdf.osl )`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_difference.inline` ) removed from: `IM_difference_color3_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_color3_genglsl`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_difference.inline` ) removed from: `IM_difference_color3_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_color3_genosl`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_difference.inline` ) removed from: `IM_difference_color4_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_color4_genglsl`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_difference.inline` ) removed from: `IM_difference_color4_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_color4_genosl`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_difference.inline` ) removed from: `IM_difference_float_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_float_genglsl`<br>|
| <a href="../documents/definitions/difference.html">IM_difference_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_difference.inline` ) removed from: `IM_difference_float_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*abs({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_difference_float_genosl`<br>|
| <a href="../documents/definitions/directional_light.html">IM_directional_light_genglsl</a> |   - attribute: `file` changed on `IM_directional_light_genglsl` from `( lights/genglsl/mx_directional_light.glsl )` to `( mx_directional_light.glsl )`<br>|
| <a href="../documents/definitions/disjointover.html">IM_disjointover_color4_genglsl</a> |   - attribute: `file` changed on `IM_disjointover_color4_genglsl` from `( stdlib/genglsl/mx_disjointover_color4.glsl )` to `( mx_disjointover_color4.glsl )`<br>|
| <a href="../documents/definitions/disjointover.html">IM_disjointover_color4_genosl</a> |   - attribute: `file` changed on `IM_disjointover_color4_genosl` from `( stdlib/genosl/mx_disjointover_color4.osl )` to `( mx_disjointover_color4.osl )`<br>|
| <a href="../documents/definitions/displacement.html">IM_displacement_float_genosl</a> |   - attribute: `file` changed on `IM_displacement_float_genosl` from `( pbrlib/genosl/mx_displacement_float.osl )` to `( mx_displacement_float.osl )`<br>|
| <a href="../documents/definitions/displacement.html">IM_displacement_vector3_genosl</a> |   - attribute: `file` changed on `IM_displacement_vector3_genosl` from `( pbrlib/genosl/mx_displacement_vector3.osl )` to `( mx_displacement_vector3.osl )`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color3FA_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_color3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color3FA_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color3_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_color3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color3_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color4FA_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_color4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color4FA_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color4_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_color4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_color4_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_float_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_float_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_float_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_float_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_matrix33_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_matrix33_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_matrix33_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_matrix44_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_matrix44_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_matrix44_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector2FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector2FA_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector2_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector2_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector2_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector3FA_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector3_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector3_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector4FA_genosl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_divide.inline` ) removed from: `IM_divide_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector4_genglsl`<br>|
| <a href="../documents/definitions/divide.html">IM_divide_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_divide.inline` ) removed from: `IM_divide_vector4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} / {{in2}}` ) added to: `IM_divide_vector4_genosl`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_color3_genglsl</a> |   - attribute: `file` changed on `IM_dodge_color3_genglsl` from `( stdlib/genglsl/mx_dodge_color3.glsl )` to `( mx_dodge_color3.glsl )`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_color3_genosl</a> |   - attribute: `file` changed on `IM_dodge_color3_genosl` from `( stdlib/genosl/mx_dodge_color3.osl )` to `( mx_dodge_color3.osl )`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_color4_genglsl</a> |   - attribute: `file` changed on `IM_dodge_color4_genglsl` from `( stdlib/genglsl/mx_dodge_color4.glsl )` to `( mx_dodge_color4.glsl )`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_color4_genosl</a> |   - attribute: `file` changed on `IM_dodge_color4_genosl` from `( stdlib/genosl/mx_dodge_color4.osl )` to `( mx_dodge_color4.osl )`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_float_genglsl</a> |   - attribute: `file` changed on `IM_dodge_float_genglsl` from `( stdlib/genglsl/mx_dodge_float.glsl )` to `( mx_dodge_float.glsl )`<br>|
| <a href="../documents/definitions/dodge.html">IM_dodge_float_genosl</a> |   - attribute: `file` changed on `IM_dodge_float_genosl` from `( stdlib/genosl/mx_dodge_float.osl )` to `( mx_dodge_float.osl )`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_boolean_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_boolean_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_boolean_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_boolean_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_boolean_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_boolean_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_color3_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_color3_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_color3_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_color4_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_color4_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_color4_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_displacementshader_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_displacementshader_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_displacementshader_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_displacementshader_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_displacementshader_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_displacementshader_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_filename_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_filename_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_filename_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_filename_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_filename_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_filename_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_float_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_float_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_float_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_float_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_integer_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_integer_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_integer_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_integer_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_integer_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_integer_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_lightshader_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_lightshader_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_lightshader_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_lightshader_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_lightshader_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_lightshader_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_matrix33_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_matrix33_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_matrix33_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_matrix44_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_matrix44_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_matrix44_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_string_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_string_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_string_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_string_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_string_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_string_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_surfaceshader_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_surfaceshader_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_surfaceshader_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_surfaceshader_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_surfaceshader_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_surfaceshader_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector2_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_vector2_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector2_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector3_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_vector3_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector3_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector4_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_vector4_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_vector4_genosl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_volumeshader_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dot.inline` ) removed from: `IM_dot_volumeshader_genglsl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_volumeshader_genglsl`<br>|
| <a href="../documents/definitions/dot.html">IM_dot_volumeshader_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dot.inline` ) removed from: `IM_dot_volumeshader_genosl`<br>- attribute: `sourcecode`=( `{{in}}` ) added to: `IM_dot_volumeshader_genosl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector2_genglsl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector2_genglsl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector2_genosl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector2_genosl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector3_genglsl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector3_genglsl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector3_genosl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector3_genosl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector4_genglsl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector4_genglsl`<br>|
| <a href="../documents/definitions/dotproduct.html">IM_dotproduct_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_dotproduct.inline` ) removed from: `IM_dotproduct_vector4_genosl`<br>- attribute: `sourcecode`=( `dot({{in1}}, {{in2}})` ) added to: `IM_dotproduct_vector4_genosl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_exp.inline` ) removed from: `IM_exp_float_genglsl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_float_genglsl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_exp.inline` ) removed from: `IM_exp_float_genosl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_float_genosl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_exp.inline` ) removed from: `IM_exp_vector2_genglsl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector2_genglsl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_exp.inline` ) removed from: `IM_exp_vector2_genosl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector2_genosl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_exp.inline` ) removed from: `IM_exp_vector3_genglsl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector3_genglsl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_exp.inline` ) removed from: `IM_exp_vector3_genosl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector3_genosl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_exp.inline` ) removed from: `IM_exp_vector4_genglsl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector4_genglsl`<br>|
| <a href="../documents/definitions/exp.html">IM_exp_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_exp.inline` ) removed from: `IM_exp_vector4_genosl`<br>- attribute: `sourcecode`=( `exp({{in}})` ) added to: `IM_exp_vector4_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_color3_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_color3_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_color3_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_color3_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_color4_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_color4_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_color4_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_color4_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_float_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_float_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_float_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_float_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_vector2_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector2_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_vector2_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector2_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_vector3_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector3_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_vector3_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector3_genosl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_floor.inline` ) removed from: `IM_floor_vector4_genglsl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector4_genglsl`<br>|
| <a href="../documents/definitions/floor.html">IM_floor_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_floor.inline` ) removed from: `IM_floor_vector4_genosl`<br>- attribute: `sourcecode`=( `floor({{in}})` ) added to: `IM_floor_vector4_genosl`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color3FA_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_color3FA_genglsl` from `( stdlib/genglsl/mx_fractal3d_fa_vector3.glsl )` to `( mx_fractal3d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color3FA_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_color3FA_genosl` from `( stdlib/genosl/mx_fractal3d_fa_color3.osl )` to `( mx_fractal3d_fa_color3.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color3_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_color3_genglsl` from `( stdlib/genglsl/mx_fractal3d_vector3.glsl )` to `( mx_fractal3d_vector3.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color3_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_color3_genosl` from `( stdlib/genosl/mx_fractal3d_color3.osl )` to `( mx_fractal3d_color3.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color4FA_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_color4FA_genglsl` from `( stdlib/genglsl/mx_fractal3d_fa_vector4.glsl )` to `( mx_fractal3d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color4FA_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_color4FA_genosl` from `( stdlib/genosl/mx_fractal3d_fa_color4.osl )` to `( mx_fractal3d_fa_color4.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color4_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_color4_genglsl` from `( stdlib/genglsl/mx_fractal3d_vector4.glsl )` to `( mx_fractal3d_vector4.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_color4_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_color4_genosl` from `( stdlib/genosl/mx_fractal3d_color4.osl )` to `( mx_fractal3d_color4.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_float_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_float_genglsl` from `( stdlib/genglsl/mx_fractal3d_float.glsl )` to `( mx_fractal3d_float.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_float_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_float_genosl` from `( stdlib/genosl/mx_fractal3d_float.osl )` to `( mx_fractal3d_float.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector2FA_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector2FA_genglsl` from `( stdlib/genglsl/mx_fractal3d_fa_vector2.glsl )` to `( mx_fractal3d_fa_vector2.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector2FA_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector2FA_genosl` from `( stdlib/genosl/mx_fractal3d_fa_vector2.osl )` to `( mx_fractal3d_fa_vector2.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector2_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector2_genglsl` from `( stdlib/genglsl/mx_fractal3d_vector2.glsl )` to `( mx_fractal3d_vector2.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector2_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector2_genosl` from `( stdlib/genosl/mx_fractal3d_vector2.osl )` to `( mx_fractal3d_vector2.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector3FA_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector3FA_genglsl` from `( stdlib/genglsl/mx_fractal3d_fa_vector3.glsl )` to `( mx_fractal3d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector3FA_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector3FA_genosl` from `( stdlib/genosl/mx_fractal3d_fa_vector3.osl )` to `( mx_fractal3d_fa_vector3.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector3_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector3_genglsl` from `( stdlib/genglsl/mx_fractal3d_vector3.glsl )` to `( mx_fractal3d_vector3.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector3_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector3_genosl` from `( stdlib/genosl/mx_fractal3d_vector3.osl )` to `( mx_fractal3d_vector3.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector4FA_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector4FA_genglsl` from `( stdlib/genglsl/mx_fractal3d_fa_vector4.glsl )` to `( mx_fractal3d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector4FA_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector4FA_genosl` from `( stdlib/genosl/mx_fractal3d_fa_vector4.osl )` to `( mx_fractal3d_fa_vector4.osl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector4_genglsl</a> |   - attribute: `file` changed on `IM_fractal3d_vector4_genglsl` from `( stdlib/genglsl/mx_fractal3d_vector4.glsl )` to `( mx_fractal3d_vector4.glsl )`<br>|
| <a href="../documents/definitions/fractal3d.html">IM_fractal3d_vector4_genosl</a> |   - attribute: `file` changed on `IM_fractal3d_vector4_genosl` from `( stdlib/genosl/mx_fractal3d_vector4.osl )` to `( mx_fractal3d_vector4.osl )`<br>|
| <a href="../documents/definitions/frame.html">IM_frame_float_genosl</a> |   - attribute: `file` changed on `IM_frame_float_genosl` from `( stdlib/genosl/mx_frame_float.osl )` to `( mx_frame_float.osl )`<br>|
| <a href="../documents/definitions/generalized_schlick_bsdf.html">IM_generalized_schlick_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_generalized_schlick_bsdf_genglsl` from `( pbrlib/genglsl/mx_generalized_schlick_bsdf.glsl )` to `( mx_generalized_schlick_bsdf.glsl )`<br>|
| <a href="../documents/definitions/generalized_schlick_bsdf.html">IM_generalized_schlick_bsdf_genmdl</a> |   - attribute: `sourcecode` changed on `IM_generalized_schlick_bsdf_genmdl` from `( mx::pbrlib::mx_generalized_schlick_bsdf(mxp_weight:{{weight}}, mxp_color0:{{color0}}, mxp_color90:{{color90}}, mxp_exponent:{{exponent}},mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}, mxp_scatter_mode:{{scatter_mode}}, mxp_base:{{base}}) )` to `( mx::pbrlib::mx_generalized_schlick_bsdf(mxp_weight:{{weight}}, mxp_color0:{{color0}}, mxp_color90:{{color90}}, mxp_exponent:{{exponent}},mxp_roughness:{{roughness}}, mxp_normal:{{normal}}, mxp_tangent:{{tangent}}, mxp_distribution:{{distribution}}, mxp_scatter_mode:{{scatter_mode}}, mxp_base:{{base}}, mxp_thinfilm_thickness:{{thinfilm_thickness}}, mxp_thinfilm_ior:{{thinfilm_ior}}) )`<br>|
| <a href="../documents/definitions/generalized_schlick_bsdf.html">IM_generalized_schlick_bsdf_genosl</a> |   - attribute: `file` changed on `IM_generalized_schlick_bsdf_genosl` from `( pbrlib/genosl/mx_generalized_schlick_bsdf.osl )` to `( legacy/mx_generalized_schlick_bsdf.osl )`<br>|
| <a href="../documents/definitions/geomcolor.html">IM_geomcolor_color3_genosl</a> |   - attribute: `file` changed on `IM_geomcolor_color3_genosl` from `( stdlib/genosl/mx_geomcolor_color3.osl )` to `( mx_geomcolor_color3.osl )`<br>|
| <a href="../documents/definitions/geomcolor.html">IM_geomcolor_color4_genosl</a> |   - attribute: `file` changed on `IM_geomcolor_color4_genosl` from `( stdlib/genosl/mx_geomcolor_color4.osl )` to `( mx_geomcolor_color4.osl )`<br>|
| <a href="../documents/definitions/geomcolor.html">IM_geomcolor_float_genosl</a> |   - attribute: `file` changed on `IM_geomcolor_float_genosl` from `( stdlib/genosl/mx_geomcolor_float.osl )` to `( mx_geomcolor_float.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_boolean_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_boolean_genosl` from `( stdlib/genosl/mx_geompropvalue_boolean.osl )` to `( mx_geompropvalue_boolean.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_color3_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_color3_genosl` from `( stdlib/genosl/mx_geompropvalue_color3.osl )` to `( mx_geompropvalue_color3.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_color4_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_color4_genosl` from `( stdlib/genosl/mx_geompropvalue_color4.osl )` to `( mx_geompropvalue_color4.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_float_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_float_genosl` from `( stdlib/genosl/mx_geompropvalue_float.osl )` to `( mx_geompropvalue_float.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_integer_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_integer_genosl` from `( stdlib/genosl/mx_geompropvalue_integer.osl )` to `( mx_geompropvalue_integer.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_string_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_string_genosl` from `( stdlib/genosl/mx_geompropvalue_string.osl )` to `( mx_geompropvalue_string.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_vector2_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_vector2_genosl` from `( stdlib/genosl/mx_geompropvalue_vector2.osl )` to `( mx_geompropvalue_vector2.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_vector3_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_vector3_genosl` from `( stdlib/genosl/mx_geompropvalue_vector3.osl )` to `( mx_geompropvalue_vector3.osl )`<br>|
| <a href="../documents/definitions/geompropvalue.html">IM_geompropvalue_vector4_genosl</a> |   - attribute: `file` changed on `IM_geompropvalue_vector4_genosl` from `( stdlib/genosl/mx_geompropvalue_vector4.osl )` to `( mx_geompropvalue_vector4.osl )`<br>|
| <a href="../documents/definitions/heighttonormal.html">IM_heighttonormal_vector3_genosl</a> |   - attribute: `file` changed on `IM_heighttonormal_vector3_genosl` from `( stdlib/genosl/mx_heighttonormal_vector3.osl )` to `( mx_heighttonormal_vector3.osl )`<br>|
| <a href="../documents/definitions/hsvtorgb.html">IM_hsvtorgb_color3_genglsl</a> |   - attribute: `file` changed on `IM_hsvtorgb_color3_genglsl` from `( stdlib/genglsl/mx_hsvtorgb_color3.glsl )` to `( mx_hsvtorgb_color3.glsl )`<br>|
| <a href="../documents/definitions/hsvtorgb.html">IM_hsvtorgb_color3_genosl</a> |   - attribute: `file` changed on `IM_hsvtorgb_color3_genosl` from `( stdlib/genosl/mx_hsvtorgb_color3.osl )` to `( mx_hsvtorgb_color3.osl )`<br>|
| <a href="../documents/definitions/hsvtorgb.html">IM_hsvtorgb_color4_genglsl</a> |   - attribute: `file` changed on `IM_hsvtorgb_color4_genglsl` from `( stdlib/genglsl/mx_hsvtorgb_color4.glsl )` to `( mx_hsvtorgb_color4.glsl )`<br>|
| <a href="../documents/definitions/hsvtorgb.html">IM_hsvtorgb_color4_genosl</a> |   - attribute: `file` changed on `IM_hsvtorgb_color4_genosl` from `( stdlib/genosl/mx_hsvtorgb_color4.osl )` to `( mx_hsvtorgb_color4.osl )`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3B_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color3B_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3B_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color3B_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color3I_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color3I_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color3_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color3_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4B_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color4B_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4B_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color4B_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color4I_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color4I_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_color4_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_color4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_color4_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_floatB_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_floatB_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_floatB_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_floatB_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_floatI_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_floatI_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_floatI_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_floatI_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_float_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_float_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_float_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_float_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2B_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector2B_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2B_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector2B_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector2I_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector2I_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector2_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector2_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector2_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3B_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector3B_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3B_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector3B_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector3I_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector3I_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector3_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector3_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4B_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector4B_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4B_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector4B_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector4I_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector4I_genosl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} == {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifequal_vector4_genglsl`<br>|
| <a href="../documents/definitions/ifequal.html">IM_ifequal_vector4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} == {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifequal_vector4_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_color3I_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_color3I_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_color3_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_color3_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_color4I_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_color4I_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_color4_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_color4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_color4_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_floatI_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_floatI_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_floatI_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_floatI_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_float_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_float_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_float_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_float_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector2I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector2I_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector2I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector2I_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector2_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector2_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector2_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector2_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector3I_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector3I_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector3_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector3_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector4I_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector4I_genosl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} > {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreater_vector4_genglsl`<br>|
| <a href="../documents/definitions/ifgreater.html">IM_ifgreater_vector4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} > {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreater_vector4_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_color3I_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_color3I_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_color3_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_color3_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_color4I_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_color4I_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_color4_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_color4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_color4_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_floatI_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_floatI_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_floatI_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_floatI_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_float_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_float_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_float_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_float_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector2I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector2I_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector2I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector2I_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector2_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector2_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector2_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector2_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector3I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector3I_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector3I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector3I_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector3_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector3_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector3_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector3_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector4I_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector4I_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector4I_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector4I_genosl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector4_genglsl</a> | - attribute: `sourcecode`=( `({{value1}} >= {{value2}}) ? {{in1}} : {{in2}}` ) added to: `IM_ifgreatereq_vector4_genglsl`<br>|
| <a href="../documents/definitions/ifgreatereq.html">IM_ifgreatereq_vector4_genosl</a> | - attribute: `sourcecode`=( `mx_ternary({{value1}} >= {{value2}}, {{in1}}, {{in2}})` ) added to: `IM_ifgreatereq_vector4_genosl`<br>|
| <a href="../documents/definitions/image.html">IM_image_color3_genglsl</a> |   - attribute: `file` changed on `IM_image_color3_genglsl` from `( stdlib/genglsl/mx_image_color3.glsl )` to `( mx_image_color3.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_color3_genosl</a> |   - attribute: `file` changed on `IM_image_color3_genosl` from `( stdlib/genosl/mx_image_color3.osl )` to `( mx_image_color3.osl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_color4_genglsl</a> |   - attribute: `file` changed on `IM_image_color4_genglsl` from `( stdlib/genglsl/mx_image_color4.glsl )` to `( mx_image_color4.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_color4_genosl</a> |   - attribute: `file` changed on `IM_image_color4_genosl` from `( stdlib/genosl/mx_image_color4.osl )` to `( mx_image_color4.osl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_float_genglsl</a> |   - attribute: `file` changed on `IM_image_float_genglsl` from `( stdlib/genglsl/mx_image_float.glsl )` to `( mx_image_float.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_float_genosl</a> |   - attribute: `file` changed on `IM_image_float_genosl` from `( stdlib/genosl/mx_image_float.osl )` to `( mx_image_float.osl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector2_genglsl</a> |   - attribute: `file` changed on `IM_image_vector2_genglsl` from `( stdlib/genglsl/mx_image_vector2.glsl )` to `( mx_image_vector2.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector2_genosl</a> |   - attribute: `file` changed on `IM_image_vector2_genosl` from `( stdlib/genosl/mx_image_vector2.osl )` to `( mx_image_vector2.osl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector3_genglsl</a> |   - attribute: `file` changed on `IM_image_vector3_genglsl` from `( stdlib/genglsl/mx_image_vector3.glsl )` to `( mx_image_vector3.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector3_genosl</a> |   - attribute: `file` changed on `IM_image_vector3_genosl` from `( stdlib/genosl/mx_image_vector3.osl )` to `( mx_image_vector3.osl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector4_genglsl</a> |   - attribute: `file` changed on `IM_image_vector4_genglsl` from `( stdlib/genglsl/mx_image_vector4.glsl )` to `( mx_image_vector4.glsl )`<br>|
| <a href="../documents/definitions/image.html">IM_image_vector4_genosl</a> |   - attribute: `file` changed on `IM_image_vector4_genosl` from `( stdlib/genosl/mx_image_vector4.osl )` to `( mx_image_vector4.osl )`<br>|
| <a href="../documents/definitions/in.html">IM_in_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_in_color4.inline` ) removed from: `IM_in_color4_genglsl`<br>- attribute: `sourcecode`=( `({{fg}}*{{bg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));` ) added to: `IM_in_color4_genglsl`<br>|
| <a href="../documents/definitions/in.html">IM_in_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_in.inline` ) removed from: `IM_in_color4_genosl`<br>- attribute: `sourcecode`=( `({{fg}}*{{bg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}))` ) added to: `IM_in_color4_genosl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_inside.inline` ) removed from: `IM_inside_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_color3_genglsl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_inside.inline` ) removed from: `IM_inside_color3_genosl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_color3_genosl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_inside.inline` ) removed from: `IM_inside_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_color4_genglsl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_inside.inline` ) removed from: `IM_inside_color4_genosl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_color4_genosl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_inside.inline` ) removed from: `IM_inside_float_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_float_genglsl`<br>|
| <a href="../documents/definitions/inside.html">IM_inside_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_inside.inline` ) removed from: `IM_inside_float_genosl`<br>- attribute: `sourcecode`=( `{{in}} * {{mask}}` ) added to: `IM_inside_float_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color3FA_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_color3FA_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color3FA_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_color3_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color3_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_color3_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color3_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color4FA_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_color4FA_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color4FA_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_color4_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color4_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_color4_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_color4_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_float_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_float_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_float_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_float_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector2FA_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector2FA_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector2_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector2_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector2_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector3FA_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector3FA_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector3_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector3_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector3_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector4FA_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector4FA_genosl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invert.inline` ) removed from: `IM_invert_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector4_genglsl`<br>|
| <a href="../documents/definitions/invert.html">IM_invert_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invert.inline` ) removed from: `IM_invert_vector4_genosl`<br>- attribute: `sourcecode`=( `{{amount}} - {{in}}` ) added to: `IM_invert_vector4_genosl`<br>|
| <a href="../documents/definitions/invertmatrix.html">IM_invertmatrix_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invertM.inline` ) removed from: `IM_invertmatrix_matrix33_genglsl`<br>- attribute: `sourcecode`=( `inverse({{in}})` ) added to: `IM_invertmatrix_matrix33_genglsl`<br>|
| <a href="../documents/definitions/invertmatrix.html">IM_invertmatrix_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invertM.inline` ) removed from: `IM_invertmatrix_matrix33_genosl`<br>- attribute: `sourcecode`=( `1 / {{in}}` ) added to: `IM_invertmatrix_matrix33_genosl`<br>|
| <a href="../documents/definitions/invertmatrix.html">IM_invertmatrix_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_invertM.inline` ) removed from: `IM_invertmatrix_matrix44_genglsl`<br>- attribute: `sourcecode`=( `inverse({{in}})` ) added to: `IM_invertmatrix_matrix44_genglsl`<br>|
| <a href="../documents/definitions/invertmatrix.html">IM_invertmatrix_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_invertM.inline` ) removed from: `IM_invertmatrix_matrix44_genosl`<br>- attribute: `sourcecode`=( `1 / {{in}}` ) added to: `IM_invertmatrix_matrix44_genosl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ln.inline` ) removed from: `IM_ln_float_genglsl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_float_genglsl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ln.inline` ) removed from: `IM_ln_float_genosl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_float_genosl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ln.inline` ) removed from: `IM_ln_vector2_genglsl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector2_genglsl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ln.inline` ) removed from: `IM_ln_vector2_genosl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector2_genosl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ln.inline` ) removed from: `IM_ln_vector3_genglsl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector3_genglsl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ln.inline` ) removed from: `IM_ln_vector3_genosl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector3_genosl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_ln.inline` ) removed from: `IM_ln_vector4_genglsl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector4_genglsl`<br>|
| <a href="../documents/definitions/ln.html">IM_ln_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ln.inline` ) removed from: `IM_ln_vector4_genosl`<br>- attribute: `sourcecode`=( `log({{in}})` ) added to: `IM_ln_vector4_genosl`<br>|
| <a href="../documents/definitions/luminance.html">IM_luminance_color3_genglsl</a> |   - attribute: `file` changed on `IM_luminance_color3_genglsl` from `( stdlib/genglsl/mx_luminance_color3.glsl )` to `( mx_luminance_color3.glsl )`<br>|
| <a href="../documents/definitions/luminance.html">IM_luminance_color3_genosl</a> |   - attribute: `file` changed on `IM_luminance_color3_genosl` from `( stdlib/genosl/mx_luminance_color3.osl )` to `( mx_luminance_color3.osl )`<br>|
| <a href="../documents/definitions/luminance.html">IM_luminance_color4_genglsl</a> |   - attribute: `file` changed on `IM_luminance_color4_genglsl` from `( stdlib/genglsl/mx_luminance_color4.glsl )` to `( mx_luminance_color4.glsl )`<br>|
| <a href="../documents/definitions/luminance.html">IM_luminance_color4_genosl</a> |   - attribute: `file` changed on `IM_luminance_color4_genosl` from `( stdlib/genosl/mx_luminance_color4.osl )` to `( mx_luminance_color4.osl )`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector2_genglsl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector2_genglsl`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector2_genosl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector2_genosl`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector3_genglsl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector3_genglsl`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector3_genosl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector3_genosl`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector4_genglsl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector4_genglsl`<br>|
| <a href="../documents/definitions/magnitude.html">IM_magnitude_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_magnitude.inline` ) removed from: `IM_magnitude_vector4_genosl`<br>- attribute: `sourcecode`=( `length({{in}})` ) added to: `IM_magnitude_vector4_genosl`<br>|
| <a href="../documents/definitions/mask.html">IM_mask_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mask_color4.inline` ) removed from: `IM_mask_color4_genglsl`<br>- attribute: `sourcecode`=( `({{bg}}*{{fg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));` ) added to: `IM_mask_color4_genglsl`<br>|
| <a href="../documents/definitions/mask.html">IM_mask_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mask.inline` ) removed from: `IM_mask_color4_genosl`<br>- attribute: `sourcecode`=( `({{bg}}*{{fg}}.a  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));` ) added to: `IM_mask_color4_genosl`<br>|
| <a href="../documents/definitions/matte.html">IM_matte_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_matte_color4.inline` ) removed from: `IM_matte_color4_genglsl`<br>- attribute: `sourcecode`=( `vec4( {{fg}}.xyz*{{fg}}.w + {{bg}}.xyz*(1.0-{{fg}}.w), {{fg}}.w + ({{bg}}.w*(1.0-{{fg}}.w)) ) * {{mix}} + ({{bg}} * (1.0-{{mix}}));` ) added to: `IM_matte_color4_genglsl`<br>|
| <a href="../documents/definitions/matte.html">IM_matte_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_matte_color4.inline` ) removed from: `IM_matte_color4_genosl`<br>- attribute: `sourcecode`=( `color4({{fg}}.rgb*{{fg}}.a + {{bg}}.rgb*(1.0-{{fg}}.a), {{fg}}.a + ({{bg}}.a*(1.0-{{fg}}.a)) ) * {{mix}} + ({{bg}} * (1.0-{{mix}}))` ) added to: `IM_matte_color4_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_color3FA_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color3FA_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_color3FA_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color3FA_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_color3_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color3_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_color3_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color3_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_color4FA_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color4FA_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_color4FA_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color4FA_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_color4_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color4_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_color4_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_color4_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_float_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_float_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_float_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_float_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector2FA_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector2FA_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector2_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector2_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector2_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector2_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector3FA_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector3FA_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector3_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector3_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector3_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector3_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector4FA_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector4FA_genosl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_max.inline` ) removed from: `IM_max_vector4_genglsl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector4_genglsl`<br>|
| <a href="../documents/definitions/max.html">IM_max_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_max.inline` ) removed from: `IM_max_vector4_genosl`<br>- attribute: `sourcecode`=( `max({{in1}}, {{in2}})` ) added to: `IM_max_vector4_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_color3FA_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color3FA_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_color3FA_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color3FA_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_color3_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color3_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_color3_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color3_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_color4FA_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color4FA_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_color4FA_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color4FA_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_color4_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color4_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_color4_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_color4_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_float_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_float_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_float_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_float_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector2FA_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector2FA_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector2_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector2_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector2_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector2_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector3FA_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector3FA_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector3_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector3_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector3_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector3_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector4FA_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector4FA_genosl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_min.inline` ) removed from: `IM_min_vector4_genglsl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector4_genglsl`<br>|
| <a href="../documents/definitions/min.html">IM_min_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_min.inline` ) removed from: `IM_min_vector4_genosl`<br>- attribute: `sourcecode`=( `min({{in1}}, {{in2}})` ) added to: `IM_min_vector4_genosl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_minus.inline` ) removed from: `IM_minus_color3_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_color3_genglsl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_minus.inline` ) removed from: `IM_minus_color3_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_color3_genosl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_minus.inline` ) removed from: `IM_minus_color4_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_color4_genglsl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_minus.inline` ) removed from: `IM_minus_color4_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_color4_genosl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_minus.inline` ) removed from: `IM_minus_float_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_float_genglsl`<br>|
| <a href="../documents/definitions/minus.html">IM_minus_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_minus.inline` ) removed from: `IM_minus_float_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} - {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_minus_float_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_bsdf_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_mix_bsdf.glsl` ) removed from: `IM_mix_bsdf_genglsl`<br>- attribute: `function`=( `mx_mix_bsdf` ) removed from: `IM_mix_bsdf_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_bsdf_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_mix.inline` ) removed from: `IM_mix_bsdf_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_color3_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_color3_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_color3_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_color3_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_color4_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_color4_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_color4_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_color4_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_edf_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_mix_edf.glsl` ) removed from: `IM_mix_edf_genglsl`<br>- attribute: `function`=( `mx_mix_edf` ) removed from: `IM_mix_edf_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_edf_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_mix.inline` ) removed from: `IM_mix_edf_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_float_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_float_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_float_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_float_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_surfaceshader_genglsl</a> |   - attribute: `file` changed on `IM_mix_surfaceshader_genglsl` from `( stdlib/genglsl/mx_mix_surfaceshader.glsl )` to `( mx_mix_surfaceshader.glsl )`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_surfaceshader_genmdl</a> |   - attribute: `sourcecode` changed on `IM_mix_surfaceshader_genmdl` from `( mx::pbrlib::mx_mix_bsdf({{fg}}, {{bg}}, {{mix}}) )` to `( mx::stdlib::mx_mix_surfaceshader({{fg}}, {{bg}}, {{mix}}) )`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_surfaceshader_genosl</a> | - attribute: `function`=( `mx_mix_surfaceshader` ) added to: `IM_mix_surfaceshader_genosl`<br>  - attribute: `file` changed on `IM_mix_surfaceshader_genosl` from `( stdlib/genosl/mx_mix.inline )` to `( mx_mix_surfaceshader.osl )`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_vector2_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector2_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_vector2_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector2_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_vector3_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector3_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_vector3_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector3_genosl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_mix.inline` ) removed from: `IM_mix_vector4_genglsl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector4_genglsl`<br>|
| <a href="../documents/definitions/mix.html">IM_mix_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_mix.inline` ) removed from: `IM_mix_vector4_genosl`<br>- attribute: `sourcecode`=( `mix({{bg}}, {{fg}}, {{mix}})` ) added to: `IM_mix_vector4_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_color3FA_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color3FA_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo_color3FA.inline` ) removed from: `IM_modulo_color3FA_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, color({{in2}},{{in2}},{{in2}}))` ) added to: `IM_modulo_color3FA_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_color3_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color3_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_color3_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color3_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_color4FA_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color4FA_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_color4FA_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color4FA_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_color4_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color4_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_color4_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_color4_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_float_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_float_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_float_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_float_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_vector2FA_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector2FA_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector2_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector2_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_vector2_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector2_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo_vector3FA.inline` ) removed from: `IM_modulo_vector3FA_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, vector({{in2}},{{in2}},{{in2}}))` ) added to: `IM_modulo_vector3FA_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector3_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector3_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_vector3_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector3_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_vector4FA_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector4FA_genosl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_modulo.inline` ) removed from: `IM_modulo_vector4_genglsl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector4_genglsl`<br>|
| <a href="../documents/definitions/modulo.html">IM_modulo_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_modulo.inline` ) removed from: `IM_modulo_vector4_genosl`<br>- attribute: `sourcecode`=( `mod({{in1}}, {{in2}})` ) added to: `IM_modulo_vector4_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_bsdfC_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_multiply_bsdf_color.glsl` ) removed from: `IM_multiply_bsdfC_genglsl`<br>- attribute: `function`=( `mx_multiply_bsdf_color` ) removed from: `IM_multiply_bsdfC_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_bsdfC_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_multiply_bsdf.inline` ) removed from: `IM_multiply_bsdfC_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_bsdfF_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_multiply_bsdf_float.glsl` ) removed from: `IM_multiply_bsdfF_genglsl`<br>- attribute: `function`=( `mx_multiply_bsdf_float` ) removed from: `IM_multiply_bsdfF_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_bsdfF_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_multiply_bsdf.inline` ) removed from: `IM_multiply_bsdfF_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color3FA_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_color3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color3FA_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color3_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_color3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color3_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color4FA_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_color4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color4FA_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color4_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_color4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_color4_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfC_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_multiply_edf_color.glsl` ) removed from: `IM_multiply_edfC_genglsl`<br>- attribute: `function`=( `mx_multiply_edf_color` ) removed from: `IM_multiply_edfC_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfC_genmdl</a> |   - attribute: `sourcecode` changed on `IM_multiply_edfC_genmdl` from `( {{in1}} )` to `( mx::pbrlib::mx_multiply_edf_color3(mxp_in1:{{in1}}, mxp_in2:{{in2}}) )`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfC_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_multiply_edf.inline` ) removed from: `IM_multiply_edfC_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfF_genglsl</a> | - attribute: `file`=( `pbrlib/genglsl/mx_multiply_edf_float.glsl` ) removed from: `IM_multiply_edfF_genglsl`<br>- attribute: `function`=( `mx_multiply_edf_float` ) removed from: `IM_multiply_edfF_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfF_genmdl</a> |   - attribute: `sourcecode` changed on `IM_multiply_edfF_genmdl` from `( {{in1}} )` to `( mx::pbrlib::mx_multiply_edf_float(mxp_in1:{{in1}}, mxp_in2:{{in2}}) )`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_edfF_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_multiply_edf.inline` ) removed from: `IM_multiply_edfF_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_float_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_float_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_float_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_float_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_matrix33_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_matrix33_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_matrix33_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_matrix44_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_matrix44_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_matrix44_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector2FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector2FA_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector2_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector2_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector2_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector3FA_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector3_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector3_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector4FA_genosl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_multiply.inline` ) removed from: `IM_multiply_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector4_genglsl`<br>|
| <a href="../documents/definitions/multiply.html">IM_multiply_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_multiply.inline` ) removed from: `IM_multiply_vector4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} * {{in2}}` ) added to: `IM_multiply_vector4_genosl`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color3FA_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_color3FA_genglsl` from `( stdlib/genglsl/mx_noise2d_fa_vector3.glsl )` to `( mx_noise2d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color3FA_genosl</a> |   - attribute: `file` changed on `IM_noise2d_color3FA_genosl` from `( stdlib/genosl/mx_noise2d_fa_color3.osl )` to `( mx_noise2d_fa_color3.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color3_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_color3_genglsl` from `( stdlib/genglsl/mx_noise2d_vector3.glsl )` to `( mx_noise2d_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color3_genosl</a> |   - attribute: `file` changed on `IM_noise2d_color3_genosl` from `( stdlib/genosl/mx_noise2d_color3.osl )` to `( mx_noise2d_color3.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color4FA_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_color4FA_genglsl` from `( stdlib/genglsl/mx_noise2d_fa_vector4.glsl )` to `( mx_noise2d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color4FA_genosl</a> |   - attribute: `file` changed on `IM_noise2d_color4FA_genosl` from `( stdlib/genosl/mx_noise2d_fa_color4.osl )` to `( mx_noise2d_fa_color4.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color4_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_color4_genglsl` from `( stdlib/genglsl/mx_noise2d_vector4.glsl )` to `( mx_noise2d_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_color4_genosl</a> |   - attribute: `file` changed on `IM_noise2d_color4_genosl` from `( stdlib/genosl/mx_noise2d_color4.osl )` to `( mx_noise2d_color4.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_float_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_float_genglsl` from `( stdlib/genglsl/mx_noise2d_float.glsl )` to `( mx_noise2d_float.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_float_genosl</a> |   - attribute: `file` changed on `IM_noise2d_float_genosl` from `( stdlib/genosl/mx_noise2d_float.osl )` to `( mx_noise2d_float.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector2FA_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector2FA_genglsl` from `( stdlib/genglsl/mx_noise2d_fa_vector2.glsl )` to `( mx_noise2d_fa_vector2.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector2FA_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector2FA_genosl` from `( stdlib/genosl/mx_noise2d_fa_vector2.osl )` to `( mx_noise2d_fa_vector2.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector2_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector2_genglsl` from `( stdlib/genglsl/mx_noise2d_vector2.glsl )` to `( mx_noise2d_vector2.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector2_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector2_genosl` from `( stdlib/genosl/mx_noise2d_vector2.osl )` to `( mx_noise2d_vector2.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector3FA_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector3FA_genglsl` from `( stdlib/genglsl/mx_noise2d_fa_vector3.glsl )` to `( mx_noise2d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector3FA_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector3FA_genosl` from `( stdlib/genosl/mx_noise2d_fa_vector3.osl )` to `( mx_noise2d_fa_vector3.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector3_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector3_genglsl` from `( stdlib/genglsl/mx_noise2d_vector3.glsl )` to `( mx_noise2d_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector3_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector3_genosl` from `( stdlib/genosl/mx_noise2d_vector3.osl )` to `( mx_noise2d_vector3.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector4FA_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector4FA_genglsl` from `( stdlib/genglsl/mx_noise2d_fa_vector4.glsl )` to `( mx_noise2d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector4FA_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector4FA_genosl` from `( stdlib/genosl/mx_noise2d_fa_vector4.osl )` to `( mx_noise2d_fa_vector4.osl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector4_genglsl</a> |   - attribute: `file` changed on `IM_noise2d_vector4_genglsl` from `( stdlib/genglsl/mx_noise2d_vector4.glsl )` to `( mx_noise2d_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise2d.html">IM_noise2d_vector4_genosl</a> |   - attribute: `file` changed on `IM_noise2d_vector4_genosl` from `( stdlib/genosl/mx_noise2d_vector4.osl )` to `( mx_noise2d_vector4.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color3FA_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_color3FA_genglsl` from `( stdlib/genglsl/mx_noise3d_fa_vector3.glsl )` to `( mx_noise3d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color3FA_genosl</a> |   - attribute: `file` changed on `IM_noise3d_color3FA_genosl` from `( stdlib/genosl/mx_noise3d_fa_color3.osl )` to `( mx_noise3d_fa_color3.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color3_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_color3_genglsl` from `( stdlib/genglsl/mx_noise3d_vector3.glsl )` to `( mx_noise3d_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color3_genosl</a> |   - attribute: `file` changed on `IM_noise3d_color3_genosl` from `( stdlib/genosl/mx_noise3d_color3.osl )` to `( mx_noise3d_color3.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color4FA_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_color4FA_genglsl` from `( stdlib/genglsl/mx_noise3d_fa_vector4.glsl )` to `( mx_noise3d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color4FA_genosl</a> |   - attribute: `file` changed on `IM_noise3d_color4FA_genosl` from `( stdlib/genosl/mx_noise3d_fa_color4.osl )` to `( mx_noise3d_fa_color4.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color4_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_color4_genglsl` from `( stdlib/genglsl/mx_noise3d_vector4.glsl )` to `( mx_noise3d_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_color4_genosl</a> |   - attribute: `file` changed on `IM_noise3d_color4_genosl` from `( stdlib/genosl/mx_noise3d_color4.osl )` to `( mx_noise3d_color4.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_float_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_float_genglsl` from `( stdlib/genglsl/mx_noise3d_float.glsl )` to `( mx_noise3d_float.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_float_genosl</a> |   - attribute: `file` changed on `IM_noise3d_float_genosl` from `( stdlib/genosl/mx_noise3d_float.osl )` to `( mx_noise3d_float.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector2FA_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector2FA_genglsl` from `( stdlib/genglsl/mx_noise3d_fa_vector2.glsl )` to `( mx_noise3d_fa_vector2.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector2FA_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector2FA_genosl` from `( stdlib/genosl/mx_noise3d_fa_vector2.osl )` to `( mx_noise3d_fa_vector2.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector2_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector2_genglsl` from `( stdlib/genglsl/mx_noise3d_vector2.glsl )` to `( mx_noise3d_vector2.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector2_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector2_genosl` from `( stdlib/genosl/mx_noise3d_vector2.osl )` to `( mx_noise3d_vector2.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector3FA_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector3FA_genglsl` from `( stdlib/genglsl/mx_noise3d_fa_vector3.glsl )` to `( mx_noise3d_fa_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector3FA_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector3FA_genosl` from `( stdlib/genosl/mx_noise3d_fa_vector3.osl )` to `( mx_noise3d_fa_vector3.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector3_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector3_genglsl` from `( stdlib/genglsl/mx_noise3d_vector3.glsl )` to `( mx_noise3d_vector3.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector3_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector3_genosl` from `( stdlib/genosl/mx_noise3d_vector3.osl )` to `( mx_noise3d_vector3.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector4FA_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector4FA_genglsl` from `( stdlib/genglsl/mx_noise3d_fa_vector4.glsl )` to `( mx_noise3d_fa_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector4FA_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector4FA_genosl` from `( stdlib/genosl/mx_noise3d_fa_vector4.osl )` to `( mx_noise3d_fa_vector4.osl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector4_genglsl</a> |   - attribute: `file` changed on `IM_noise3d_vector4_genglsl` from `( stdlib/genglsl/mx_noise3d_vector4.glsl )` to `( mx_noise3d_vector4.glsl )`<br>|
| <a href="../documents/definitions/noise3d.html">IM_noise3d_vector4_genosl</a> |   - attribute: `file` changed on `IM_noise3d_vector4_genosl` from `( stdlib/genosl/mx_noise3d_vector4.osl )` to `( mx_noise3d_vector4.osl )`<br>|
| <a href="../documents/definitions/normal.html">IM_normal_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_normal_vector3.inline` ) removed from: `IM_normal_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{space}}, N)` ) added to: `IM_normal_vector3_genosl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_normalize.inline` ) removed from: `IM_normalize_vector2_genglsl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector2_genglsl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_normalize.inline` ) removed from: `IM_normalize_vector2_genosl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector2_genosl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_normalize.inline` ) removed from: `IM_normalize_vector3_genglsl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector3_genglsl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_normalize.inline` ) removed from: `IM_normalize_vector3_genosl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector3_genosl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_normalize.inline` ) removed from: `IM_normalize_vector4_genglsl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector4_genglsl`<br>|
| <a href="../documents/definitions/normalize.html">IM_normalize_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_normalize.inline` ) removed from: `IM_normalize_vector4_genosl`<br>- attribute: `sourcecode`=( `normalize({{in}})` ) added to: `IM_normalize_vector4_genosl`<br>|
| <a href="../documents/definitions/normalmap.html">IM_normalmap_genglsl</a> |   - attribute: `file` changed on `IM_normalmap_genglsl` from `( stdlib/genglsl/mx_normalmap.glsl )` to `( mx_normalmap.glsl )`<br>|
| <a href="../documents/definitions/normalmap.html">IM_normalmap_genosl</a> |   - attribute: `file` changed on `IM_normalmap_genosl` from `( stdlib/genosl/mx_normalmap.osl )` to `( mx_normalmap.osl )`<br>|
| <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">IM_oren_nayar_diffuse_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_oren_nayar_diffuse_bsdf_genglsl` from `( pbrlib/genglsl/mx_oren_nayar_diffuse_bsdf.glsl )` to `( mx_oren_nayar_diffuse_bsdf.glsl )`<br>|
| <a href="../documents/definitions/oren_nayar_diffuse_bsdf.html">IM_oren_nayar_diffuse_bsdf_genosl</a> | - attribute: `function`=( `mx_oren_nayar_diffuse_bsdf` ) added to: `IM_oren_nayar_diffuse_bsdf_genosl`<br>  - attribute: `file` changed on `IM_oren_nayar_diffuse_bsdf_genosl` from `( pbrlib/genosl/mx_oren_nayar_diffuse_bsdf.inline )` to `( legacy/mx_oren_nayar_diffuse_bsdf.osl )`<br>|
| <a href="../documents/definitions/out.html">IM_out_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_out_color4.inline` ) removed from: `IM_out_color4_genglsl`<br>- attribute: `sourcecode`=( `({{fg}}*(1.0-{{bg}}.a)  * {{mix}}) + ({{bg}} * (1.0-{{mix}}));` ) added to: `IM_out_color4_genglsl`<br>|
| <a href="../documents/definitions/out.html">IM_out_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_out.inline` ) removed from: `IM_out_color4_genosl`<br>- attribute: `sourcecode`=( `({{fg}}*(1.0-{{bg}}.a)  * {{mix}}) + ({{bg}} * (1.0-{{mix}}))` ) added to: `IM_out_color4_genosl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_outside.inline` ) removed from: `IM_outside_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_color3_genglsl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_outside.inline` ) removed from: `IM_outside_color3_genosl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_color3_genosl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_outside.inline` ) removed from: `IM_outside_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_color4_genglsl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_outside.inline` ) removed from: `IM_outside_color4_genosl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_color4_genosl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_outside.inline` ) removed from: `IM_outside_float_genglsl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_float_genglsl`<br>|
| <a href="../documents/definitions/outside.html">IM_outside_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_outside.inline` ) removed from: `IM_outside_float_genosl`<br>- attribute: `sourcecode`=( `{{in}} * (1.0 - {{mask}})` ) added to: `IM_outside_float_genosl`<br>|
| <a href="../documents/definitions/over.html">IM_over_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_over_color4.inline` ) removed from: `IM_over_color4_genglsl`<br>- attribute: `sourcecode`=( `({{fg}} + ({{bg}}*(1.0-{{fg}}[3]))) * {{mix}} + {{bg}} * (1.0-{{mix}})` ) added to: `IM_over_color4_genglsl`<br>|
| <a href="../documents/definitions/over.html">IM_over_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_over.inline` ) removed from: `IM_over_color4_genosl`<br>- attribute: `sourcecode`=( `({{fg}} + ({{bg}}*(1.0-{{fg}}.a))) * {{mix}} + {{bg}} * (1.0-{{mix}})` ) added to: `IM_over_color4_genosl`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_color3_genglsl</a> |   - attribute: `file` changed on `IM_overlay_color3_genglsl` from `( stdlib/genglsl/mx_overlay_color3.glsl )` to `( mx_overlay_color3.glsl )`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_color3_genosl</a> |   - attribute: `file` changed on `IM_overlay_color3_genosl` from `( stdlib/genosl/mx_overlay_color3.osl )` to `( mx_overlay_color3.osl )`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_color4_genglsl</a> |   - attribute: `file` changed on `IM_overlay_color4_genglsl` from `( stdlib/genglsl/mx_overlay_color4.glsl )` to `( mx_overlay_color4.glsl )`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_color4_genosl</a> |   - attribute: `file` changed on `IM_overlay_color4_genosl` from `( stdlib/genosl/mx_overlay_color4.osl )` to `( mx_overlay_color4.osl )`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_overlay_float.inline` ) removed from: `IM_overlay_float_genglsl`<br>- attribute: `sourcecode`=( `({{fg}} < 0.5) ? ({{mix}}*2.0*{{fg}}*{{bg}}) + ((1.0-{{mix}})*{{bg}}) : ({{mix}}*(1.0-(1.0-{{fg}})*(1.0-{{bg}}))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_overlay_float_genglsl`<br>|
| <a href="../documents/definitions/overlay.html">IM_overlay_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_overlay.inline` ) removed from: `IM_overlay_float_genosl`<br>- attribute: `sourcecode`=( `({{fg}} < 0.5) ? ({{mix}}*2.0*{{fg}}*{{bg}}) + ((1.0-{{mix}})*{{bg}}) : ({{mix}}*(1.0-(1.0-{{fg}})*(1.0-{{bg}}))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_overlay_float_genosl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_plus.inline` ) removed from: `IM_plus_color3_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_color3_genglsl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_plus.inline` ) removed from: `IM_plus_color3_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_color3_genosl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_plus.inline` ) removed from: `IM_plus_color4_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_color4_genglsl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_plus.inline` ) removed from: `IM_plus_color4_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_color4_genosl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_plus.inline` ) removed from: `IM_plus_float_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_float_genglsl`<br>|
| <a href="../documents/definitions/plus.html">IM_plus_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_plus.inline` ) removed from: `IM_plus_float_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*({{bg}} + {{fg}})) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_plus_float_genosl`<br>|
| <a href="../documents/definitions/point_light.html">IM_point_light_genglsl</a> |   - attribute: `file` changed on `IM_point_light_genglsl` from `( lights/genglsl/mx_point_light.glsl )` to `( mx_point_light.glsl )`<br>|
| <a href="../documents/definitions/position.html">IM_position_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_position_vector3.inline` ) removed from: `IM_position_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{space}}, P)` ) added to: `IM_position_vector3_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power_color3_float.inline` ) removed from: `IM_power_color3FA_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, vec3({{in2}}))` ) added to: `IM_power_color3FA_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_color3FA_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color3FA_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_color3_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color3_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_color3_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color3_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power_color4_float.inline` ) removed from: `IM_power_color4FA_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, vec4({{in2}}))` ) added to: `IM_power_color4FA_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_color4FA_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color4FA_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_color4_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color4_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_color4_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_color4_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_float_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_float_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_float_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_float_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power_vector2_float.inline` ) removed from: `IM_power_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, vec2({{in2}}))` ) added to: `IM_power_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector2FA_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector2FA_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_vector2_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector2_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector2_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector2_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power_vector3_float.inline` ) removed from: `IM_power_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, vec3({{in2}}))` ) added to: `IM_power_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector3FA_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector3FA_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_vector3_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector3_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector3_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector3_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power_vector4_float.inline` ) removed from: `IM_power_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, vec4({{in2}}))` ) added to: `IM_power_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector4FA_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector4FA_genosl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_power.inline` ) removed from: `IM_power_vector4_genglsl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector4_genglsl`<br>|
| <a href="../documents/definitions/power.html">IM_power_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_power.inline` ) removed from: `IM_power_vector4_genosl`<br>- attribute: `sourcecode`=( `pow({{in1}}, {{in2}})` ) added to: `IM_power_vector4_genosl`<br>|
| <a href="../documents/definitions/premult.html">IM_premult_color4_genglsl</a> |   - attribute: `file` changed on `IM_premult_color4_genglsl` from `( stdlib/genglsl/mx_premult_color4.glsl )` to `( mx_premult_color4.glsl )`<br>|
| <a href="../documents/definitions/premult.html">IM_premult_color4_genosl</a> |   - attribute: `file` changed on `IM_premult_color4_genosl` from `( stdlib/genosl/mx_premult_color4.osl )` to `( mx_premult_color4.osl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_color3_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_color3_genglsl` from `( stdlib/genglsl/mx_ramplr_vector3.glsl )` to `( mx_ramplr_vector3.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_color3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_color3_genosl`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_color4_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_color4_genglsl` from `( stdlib/genglsl/mx_ramplr_vector4.glsl )` to `( mx_ramplr_vector4.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_color4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_color4_genosl`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_float_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_float_genglsl` from `( stdlib/genglsl/mx_ramplr_float.glsl )` to `( mx_ramplr_float.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_float_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_float_genosl`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector2_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_vector2_genglsl` from `( stdlib/genglsl/mx_ramplr_vector2.glsl )` to `( mx_ramplr_vector2.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_vector2_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_vector2_genosl`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector3_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_vector3_genglsl` from `( stdlib/genglsl/mx_ramplr_vector3.glsl )` to `( mx_ramplr_vector3.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_vector3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_vector3_genosl`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector4_genglsl</a> |   - attribute: `file` changed on `IM_ramplr_vector4_genglsl` from `( stdlib/genglsl/mx_ramplr_vector4.glsl )` to `( mx_ramplr_vector4.glsl )`<br>|
| <a href="../documents/definitions/ramplr.html">IM_ramplr_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramplr.inline` ) removed from: `IM_ramplr_vector4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, clamp({{texcoord}}.x, 0, 1))` ) added to: `IM_ramplr_vector4_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_color3_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_color3_genglsl` from `( stdlib/genglsl/mx_ramptb_vector3.glsl )` to `( mx_ramptb_vector3.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_color3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_color3_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_color4_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_color4_genglsl` from `( stdlib/genglsl/mx_ramptb_vector4.glsl )` to `( mx_ramptb_vector4.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_color4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_color4_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_float_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_float_genglsl` from `( stdlib/genglsl/mx_ramptb_float.glsl )` to `( mx_ramptb_float.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_float_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_float_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector2_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_vector2_genglsl` from `( stdlib/genglsl/mx_ramptb_vector2.glsl )` to `( mx_ramptb_vector2.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_vector2_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_vector2_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector3_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_vector3_genglsl` from `( stdlib/genglsl/mx_ramptb_vector3.glsl )` to `( mx_ramptb_vector3.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_vector3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_vector3_genosl`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector4_genglsl</a> |   - attribute: `file` changed on `IM_ramptb_vector4_genglsl` from `( stdlib/genglsl/mx_ramptb_vector4.glsl )` to `( mx_ramptb_vector4.glsl )`<br>|
| <a href="../documents/definitions/ramptb.html">IM_ramptb_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_ramptb.inline` ) removed from: `IM_ramptb_vector4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, clamp({{texcoord}}.y, 0, 1))` ) added to: `IM_ramptb_vector4_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_color3FA_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_color3FA_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_color3FA_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_color3_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_color3_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_color3_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_color3_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_color4FA_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_color4FA_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_color4FA_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_color4_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_color4_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_color4_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_color4_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_float_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_float_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_float_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_float_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector2FA_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector2FA_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector2_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector2_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector2_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector3FA_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector3FA_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector3_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector3_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector3_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector4FA_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector4FA_genosl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_remap.inline` ) removed from: `IM_remap_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{outlow}} + ({{in}} - {{inlow}}) * ({{outhigh}} - {{outlow}}) / ({{inhigh}} - {{inlow}})` ) added to: `IM_remap_vector4_genglsl`<br>|
| <a href="../documents/definitions/remap.html">IM_remap_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_remap.inline` ) removed from: `IM_remap_vector4_genosl`<br>- attribute: `sourcecode`=( `mx_remap({{in}}, {{inlow}}, {{inhigh}}, {{outlow}}, {{outhigh}}, 0)` ) added to: `IM_remap_vector4_genosl`<br>|
| <a href="../documents/definitions/rgbtohsv.html">IM_rgbtohsv_color3_genglsl</a> |   - attribute: `file` changed on `IM_rgbtohsv_color3_genglsl` from `( stdlib/genglsl/mx_rgbtohsv_color3.glsl )` to `( mx_rgbtohsv_color3.glsl )`<br>|
| <a href="../documents/definitions/rgbtohsv.html">IM_rgbtohsv_color3_genosl</a> |   - attribute: `file` changed on `IM_rgbtohsv_color3_genosl` from `( stdlib/genosl/mx_rgbtohsv_color3.osl )` to `( mx_rgbtohsv_color3.osl )`<br>|
| <a href="../documents/definitions/rgbtohsv.html">IM_rgbtohsv_color4_genglsl</a> |   - attribute: `file` changed on `IM_rgbtohsv_color4_genglsl` from `( stdlib/genglsl/mx_rgbtohsv_color4.glsl )` to `( mx_rgbtohsv_color4.glsl )`<br>|
| <a href="../documents/definitions/rgbtohsv.html">IM_rgbtohsv_color4_genosl</a> |   - attribute: `file` changed on `IM_rgbtohsv_color4_genosl` from `( stdlib/genosl/mx_rgbtohsv_color4.osl )` to `( mx_rgbtohsv_color4.osl )`<br>|
| <a href="../documents/definitions/rotate2d.html">IM_rotate2d_vector2_genglsl</a> |   - attribute: `file` changed on `IM_rotate2d_vector2_genglsl` from `( stdlib/genglsl/mx_rotate_vector2.glsl )` to `( mx_rotate_vector2.glsl )`<br>|
| <a href="../documents/definitions/rotate2d.html">IM_rotate2d_vector2_genosl</a> |   - attribute: `file` changed on `IM_rotate2d_vector2_genosl` from `( stdlib/genosl/mx_rotate_vector2.osl )` to `( mx_rotate_vector2.osl )`<br>|
| <a href="../documents/definitions/rotate3d.html">IM_rotate3d_vector3_genglsl</a> |   - attribute: `file` changed on `IM_rotate3d_vector3_genglsl` from `( stdlib/genglsl/mx_rotate_vector3.glsl )` to `( mx_rotate_vector3.glsl )`<br>|
| <a href="../documents/definitions/rotate3d.html">IM_rotate3d_vector3_genosl</a> |   - attribute: `file` changed on `IM_rotate3d_vector3_genosl` from `( stdlib/genosl/mx_rotate_vector3.osl )` to `( mx_rotate_vector3.osl )`<br>|
| <a href="../documents/definitions/roughness_anisotropy.html">IM_roughness_anisotropy_genglsl</a> |   - attribute: `file` changed on `IM_roughness_anisotropy_genglsl` from `( pbrlib/genglsl/mx_roughness_anisotropy.glsl )` to `( mx_roughness_anisotropy.glsl )`<br>|
| <a href="../documents/definitions/roughness_anisotropy.html">IM_roughness_anisotropy_genosl</a> |   - attribute: `file` changed on `IM_roughness_anisotropy_genosl` from `( pbrlib/genosl/mx_roughness_anisotropy.osl )` to `( mx_roughness_anisotropy.osl )`<br>|
| <a href="../documents/definitions/roughness_dual.html">IM_roughness_dual_genglsl</a> |   - attribute: `file` changed on `IM_roughness_dual_genglsl` from `( pbrlib/genglsl/mx_roughness_dual.glsl )` to `( mx_roughness_dual.glsl )`<br>|
| <a href="../documents/definitions/roughness_dual.html">IM_roughness_dual_genosl</a> |   - attribute: `file` changed on `IM_roughness_dual_genosl` from `( pbrlib/genosl/mx_roughness_dual.osl )` to `( mx_roughness_dual.osl )`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_screen.inline` ) removed from: `IM_screen_color3_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_color3_genglsl`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_color3_genmdl</a> |   - attribute: `sourcecode` changed on `IM_screen_color3_genmdl` from `( ({{mix}}*((1.0 - (1.0 - {{fg}})) * (1 - {{bg}}))) + ((1.0-{{mix}})*{{bg}}) )` to `( ({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}}) )`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_screen.inline` ) removed from: `IM_screen_color3_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_color3_genosl`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_screen.inline` ) removed from: `IM_screen_color4_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_color4_genglsl`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_screen.inline` ) removed from: `IM_screen_color4_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_color4_genosl`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_screen.inline` ) removed from: `IM_screen_float_genglsl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1.0 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_float_genglsl`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_float_genmdl</a> |   - attribute: `sourcecode` changed on `IM_screen_float_genmdl` from `( ({{mix}}*((1.0 - (1.0 - {{fg}})) * (1 - {{bg}}))) + ((1.0-{{mix}})*{{bg}}) )` to `( ({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}}) )`<br>|
| <a href="../documents/definitions/screen.html">IM_screen_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_screen.inline` ) removed from: `IM_screen_float_genosl`<br>- attribute: `sourcecode`=( `({{mix}}*((1.0 - (1.0 - {{fg}}) * (1 - {{bg}})))) + ((1.0-{{mix}})*{{bg}})` ) added to: `IM_screen_float_genosl`<br>|
| <a href="../documents/definitions/sheen_bsdf.html">IM_sheen_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_sheen_bsdf_genglsl` from `( pbrlib/genglsl/mx_sheen_bsdf.glsl )` to `( mx_sheen_bsdf.glsl )`<br>|
| <a href="../documents/definitions/sheen_bsdf.html">IM_sheen_bsdf_genosl</a> |   - attribute: `file` changed on `IM_sheen_bsdf_genosl` from `( pbrlib/genosl/mx_sheen_bsdf.osl )` to `( legacy/mx_sheen_bsdf.osl )`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_color3_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_color3_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_color3_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_color3_genosl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_color4_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_color4_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_color4_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_color4_genosl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_float_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_float_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_float_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_float_genosl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_vector2_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector2_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_vector2_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector2_genosl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_vector3_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector3_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_vector3_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector3_genosl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sign.inline` ) removed from: `IM_sign_vector4_genglsl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector4_genglsl`<br>|
| <a href="../documents/definitions/sign.html">IM_sign_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sign.inline` ) removed from: `IM_sign_vector4_genosl`<br>- attribute: `sourcecode`=( `sign({{in}})` ) added to: `IM_sign_vector4_genosl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sin.inline` ) removed from: `IM_sin_float_genglsl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_float_genglsl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sin.inline` ) removed from: `IM_sin_float_genosl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_float_genosl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sin.inline` ) removed from: `IM_sin_vector2_genglsl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector2_genglsl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sin.inline` ) removed from: `IM_sin_vector2_genosl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector2_genosl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sin.inline` ) removed from: `IM_sin_vector3_genglsl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector3_genglsl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sin.inline` ) removed from: `IM_sin_vector3_genosl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector3_genosl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sin.inline` ) removed from: `IM_sin_vector4_genglsl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector4_genglsl`<br>|
| <a href="../documents/definitions/sin.html">IM_sin_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sin.inline` ) removed from: `IM_sin_vector4_genosl`<br>- attribute: `sourcecode`=( `sin({{in}})` ) added to: `IM_sin_vector4_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color3FA_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_color3FA_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec3FA.glsl )` to `( mx_smoothstep_vec3FA.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_color3FA_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_color3FA_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color3_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_color3_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec3.glsl )` to `( mx_smoothstep_vec3.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_color3_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_color3_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color4FA_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_color4FA_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec4FA.glsl )` to `( mx_smoothstep_vec4FA.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_color4FA_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_color4FA_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color4_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_color4_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec4.glsl )` to `( mx_smoothstep_vec4.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_color4_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_color4_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_float_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_float_genglsl` from `( stdlib/genglsl/mx_smoothstep_float.glsl )` to `( mx_smoothstep_float.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_float_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_float_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector2FA_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector2FA_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec2FA.glsl )` to `( mx_smoothstep_vec2FA.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector2FA_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector2FA_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector2_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector2_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec2.glsl )` to `( mx_smoothstep_vec2.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector2_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector2_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector3FA_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector3FA_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec3FA.glsl )` to `( mx_smoothstep_vec3FA.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector3FA_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector3FA_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector3_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector3_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec3.glsl )` to `( mx_smoothstep_vec3.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector3_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector3_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector4FA_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector4FA_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec4FA.glsl )` to `( mx_smoothstep_vec4FA.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector4FA_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector4FA_genosl`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector4_genglsl</a> |   - attribute: `file` changed on `IM_smoothstep_vector4_genglsl` from `( stdlib/genglsl/mx_smoothstep_vec4.glsl )` to `( mx_smoothstep_vec4.glsl )`<br>|
| <a href="../documents/definitions/smoothstep.html">IM_smoothstep_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_smoothstep.inline` ) removed from: `IM_smoothstep_vector4_genosl`<br>- attribute: `sourcecode`=( `smoothstep({{low}}, {{high}}, {{in}})` ) added to: `IM_smoothstep_vector4_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_color3_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_color3_genglsl` from `( stdlib/genglsl/mx_splitlr_vector3.glsl )` to `( mx_splitlr_vector3.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_color3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_color3_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_color4_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_color4_genglsl` from `( stdlib/genglsl/mx_splitlr_vector4.glsl )` to `( mx_splitlr_vector4.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_color4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_color4_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_float_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_float_genglsl` from `( stdlib/genglsl/mx_splitlr_float.glsl )` to `( mx_splitlr_float.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_float_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_float_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector2_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_vector2_genglsl` from `( stdlib/genglsl/mx_splitlr_vector2.glsl )` to `( mx_splitlr_vector2.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_vector2_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_vector2_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector3_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_vector3_genglsl` from `( stdlib/genglsl/mx_splitlr_vector3.glsl )` to `( mx_splitlr_vector3.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_vector3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_vector3_genosl`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector4_genglsl</a> |   - attribute: `file` changed on `IM_splitlr_vector4_genglsl` from `( stdlib/genglsl/mx_splitlr_vector4.glsl )` to `( mx_splitlr_vector4.glsl )`<br>|
| <a href="../documents/definitions/splitlr.html">IM_splitlr_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splitlr.inline` ) removed from: `IM_splitlr_vector4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuel}}, {{valuer}}, aastep({{center}}, {{texcoord}}.x))` ) added to: `IM_splitlr_vector4_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_color3_genglsl</a> |   - attribute: `file` changed on `IM_splittb_color3_genglsl` from `( stdlib/genglsl/mx_splittb_vector3.glsl )` to `( mx_splittb_vector3.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_color3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_color3_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_color4_genglsl</a> |   - attribute: `file` changed on `IM_splittb_color4_genglsl` from `( stdlib/genglsl/mx_splittb_vector4.glsl )` to `( mx_splittb_vector4.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_color4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_color4_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_float_genglsl</a> |   - attribute: `file` changed on `IM_splittb_float_genglsl` from `( stdlib/genglsl/mx_splittb_float.glsl )` to `( mx_splittb_float.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_float_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_float_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector2_genglsl</a> |   - attribute: `file` changed on `IM_splittb_vector2_genglsl` from `( stdlib/genglsl/mx_splittb_vector2.glsl )` to `( mx_splittb_vector2.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_vector2_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_vector2_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector3_genglsl</a> |   - attribute: `file` changed on `IM_splittb_vector3_genglsl` from `( stdlib/genglsl/mx_splittb_vector3.glsl )` to `( mx_splittb_vector3.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_vector3_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_vector3_genosl`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector4_genglsl</a> |   - attribute: `file` changed on `IM_splittb_vector4_genglsl` from `( stdlib/genglsl/mx_splittb_vector4.glsl )` to `( mx_splittb_vector4.glsl )`<br>|
| <a href="../documents/definitions/splittb.html">IM_splittb_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_splittb.inline` ) removed from: `IM_splittb_vector4_genosl`<br>- attribute: `sourcecode`=( `mix({{valuet}}, {{valueb}}, aastep({{center}}, {{texcoord}}.y))` ) added to: `IM_splittb_vector4_genosl`<br>|
| <a href="../documents/definitions/spot_light.html">IM_spot_light_genglsl</a> |   - attribute: `file` changed on `IM_spot_light_genglsl` from `( lights/genglsl/mx_spot_light.glsl )` to `( mx_spot_light.glsl )`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sqrt.inline` ) removed from: `IM_sqrt_float_genglsl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_float_genglsl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sqrt.inline` ) removed from: `IM_sqrt_float_genosl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_float_genosl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector2_genglsl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector2_genglsl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector2_genosl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector2_genosl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector3_genglsl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector3_genglsl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector3_genosl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector3_genosl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector4_genglsl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector4_genglsl`<br>|
| <a href="../documents/definitions/sqrt.html">IM_sqrt_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_sqrt.inline` ) removed from: `IM_sqrt_vector4_genosl`<br>- attribute: `sourcecode`=( `sqrt({{in}})` ) added to: `IM_sqrt_vector4_genosl`<br>|
| <a href="../documents/definitions/subsurface_bsdf.html">IM_subsurface_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_subsurface_bsdf_genglsl` from `( pbrlib/genglsl/mx_subsurface_bsdf.glsl )` to `( mx_subsurface_bsdf.glsl )`<br>|
| <a href="../documents/definitions/subsurface_bsdf.html">IM_subsurface_bsdf_genosl</a> |   - attribute: `file` changed on `IM_subsurface_bsdf_genosl` from `( pbrlib/genosl/mx_subsurface_bsdf.osl )` to `( legacy/mx_subsurface_bsdf.osl )`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_color3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color3FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_color3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color3FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_color3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color3_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_color3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color3_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_color4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color4FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_color4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color4FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_color4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color4_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_color4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_color4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_color4_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_float_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_float_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_float_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_float_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix33FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_matrix33FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_matrix33FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix33FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_matrix33FA_genosl`<br>- attribute: `sourcecode`=( `mx_subtract({{in1}}, {{in2}})` ) added to: `IM_subtract_matrix33FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_matrix33_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_matrix33_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_matrix33_genosl`<br>- attribute: `sourcecode`=( `mx_subtract({{in1}}, {{in2}})` ) added to: `IM_subtract_matrix33_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix44FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_matrix44FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_matrix44FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix44FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_matrix44FA_genosl`<br>- attribute: `sourcecode`=( `mx_subtract({{in1}}, {{in2}})` ) added to: `IM_subtract_matrix44FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_matrix44_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_matrix44_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_matrix44_genosl`<br>- attribute: `sourcecode`=( `mx_subtract({{in1}}, {{in2}})` ) added to: `IM_subtract_matrix44_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector2FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector2FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector2FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector2FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector2FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector2FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector2_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector2_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector2_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector2_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector3FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector3FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector3FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector3FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector3FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector3FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector3_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector3_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector3_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector4FA_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector4FA_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector4FA_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector4FA_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector4FA_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector4FA_genosl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_subtract.inline` ) removed from: `IM_subtract_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector4_genglsl`<br>|
| <a href="../documents/definitions/subtract.html">IM_subtract_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_subtract.inline` ) removed from: `IM_subtract_vector4_genosl`<br>- attribute: `sourcecode`=( `{{in1}} - {{in2}}` ) added to: `IM_subtract_vector4_genosl`<br>|
| <a href="../documents/definitions/surface.html">IM_surface_genosl</a> |   - attribute: `file` changed on `IM_surface_genosl` from `( pbrlib/genosl/mx_surface.osl )` to `( legacy/mx_surface.osl )`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_float_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_tan.inline` ) removed from: `IM_tan_float_genglsl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_float_genglsl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_float_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_tan.inline` ) removed from: `IM_tan_float_genosl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_float_genosl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector2_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_tan.inline` ) removed from: `IM_tan_vector2_genglsl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector2_genglsl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_tan.inline` ) removed from: `IM_tan_vector2_genosl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector2_genosl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_tan.inline` ) removed from: `IM_tan_vector3_genglsl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector3_genglsl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_tan.inline` ) removed from: `IM_tan_vector3_genosl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector3_genosl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_tan.inline` ) removed from: `IM_tan_vector4_genglsl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector4_genglsl`<br>|
| <a href="../documents/definitions/tan.html">IM_tan_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_tan.inline` ) removed from: `IM_tan_vector4_genosl`<br>- attribute: `sourcecode`=( `tan({{in}})` ) added to: `IM_tan_vector4_genosl`<br>|
| <a href="../documents/definitions/tangent.html">IM_tangent_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_tangent_vector3.inline` ) removed from: `IM_tangent_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{space}}, normalize(dPdu))` ) added to: `IM_tangent_vector3_genosl`<br>|
| <a href="../documents/definitions/texcoord.html">IM_texcoord_vector2_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_texcoord_vector2.inline` ) removed from: `IM_texcoord_vector2_genosl`<br>- attribute: `sourcecode`=( `vector2(u,v)` ) added to: `IM_texcoord_vector2_genosl`<br>|
| <a href="../documents/definitions/texcoord.html">IM_texcoord_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_texcoord_vector3.inline` ) removed from: `IM_texcoord_vector3_genosl`<br>- attribute: `sourcecode`=( `vector(u,v,0)` ) added to: `IM_texcoord_vector3_genosl`<br>|
| <a href="../documents/definitions/thin_surface.html">IM_thin_surface_genmdl</a> |   - attribute: `sourcecode` changed on `IM_thin_surface_genmdl` from `( mx::pbrlib::mx_thin_surface(mxp_front_bsdf:{{front_bsdf}}, mxp_front_edf:{{front_edf}}, (mxp_back_bsdf:{{back_bsdf}}, mxp_back_edf:{{back_edf}}, mxp_opacity:{{opacity}}) )` to `( mx::pbrlib::mx_thin_surface(mxp_front_bsdf:{{front_bsdf}}, mxp_front_edf:{{front_edf}}, mxp_back_bsdf:{{back_bsdf}}, mxp_back_edf:{{back_edf}}, mxp_opacity:{{opacity}}) )`<br>|
| <a href="../documents/definitions/time.html">IM_time_float_genosl</a> |   - attribute: `file` changed on `IM_time_float_genosl` from `( stdlib/genosl/mx_time_float.osl )` to `( mx_time_float.osl )`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector2M3_genglsl</a> |   - attribute: `file` changed on `IM_transformmatrix_vector2M3_genglsl` from `( stdlib/genglsl/mx_transformmatrix_vector2M3.glsl )` to `( mx_transformmatrix_vector2M3.glsl )`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector2M3_genosl</a> |   - attribute: `file` changed on `IM_transformmatrix_vector2M3_genosl` from `( stdlib/genosl/mx_transformmatrix_vector2M3.osl )` to `( mx_transformmatrix_vector2M3.osl )`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector3M4_genglsl</a> |   - attribute: `file` changed on `IM_transformmatrix_vector3M4_genglsl` from `( stdlib/genglsl/mx_transformmatrix_vector3M4.glsl )` to `( mx_transformmatrix_vector3M4.glsl )`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector3M4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformmatrix.inline` ) removed from: `IM_transformmatrix_vector3M4_genosl`<br>- attribute: `sourcecode`=( `transform({{mat}}, {{in}})` ) added to: `IM_transformmatrix_vector3M4_genosl`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector3_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_transformmatrix.inline` ) removed from: `IM_transformmatrix_vector3_genglsl`<br>- attribute: `sourcecode`=( `{{mat}} * {{in}}` ) added to: `IM_transformmatrix_vector3_genglsl`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformmatrix.inline` ) removed from: `IM_transformmatrix_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{mat}}, {{in}})` ) added to: `IM_transformmatrix_vector3_genosl`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector4_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_transformmatrix.inline` ) removed from: `IM_transformmatrix_vector4_genglsl`<br>- attribute: `sourcecode`=( `{{mat}} * {{in}}` ) added to: `IM_transformmatrix_vector4_genglsl`<br>|
| <a href="../documents/definitions/transformmatrix.html">IM_transformmatrix_vector4_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformmatrix.inline` ) removed from: `IM_transformmatrix_vector4_genosl`<br>- attribute: `sourcecode`=( `transform({{mat}}, {{in}})` ) added to: `IM_transformmatrix_vector4_genosl`<br>|
| <a href="../documents/definitions/transformnormal.html">IM_transformnormal_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformnormal.inline` ) removed from: `IM_transformnormal_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{fromspace}}, {{tospace}}, normal({{in}}))` ) added to: `IM_transformnormal_vector3_genosl`<br>|
| <a href="../documents/definitions/transformpoint.html">IM_transformpoint_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformpoint.inline` ) removed from: `IM_transformpoint_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{fromspace}}, {{tospace}}, point({{in}}))` ) added to: `IM_transformpoint_vector3_genosl`<br>|
| <a href="../documents/definitions/transformvector.html">IM_transformvector_vector3_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transformvector.inline` ) removed from: `IM_transformvector_vector3_genosl`<br>- attribute: `sourcecode`=( `transform({{fromspace}}, {{tospace}}, {{in}})` ) added to: `IM_transformvector_vector3_genosl`<br>|
| <a href="../documents/definitions/translucent_bsdf.html">IM_translucent_bsdf_genglsl</a> |   - attribute: `file` changed on `IM_translucent_bsdf_genglsl` from `( pbrlib/genglsl/mx_translucent_bsdf.glsl )` to `( mx_translucent_bsdf.glsl )`<br>|
| <a href="../documents/definitions/translucent_bsdf.html">IM_translucent_bsdf_genosl</a> | - attribute: `function`=( `mx_translucent_bsdf` ) added to: `IM_translucent_bsdf_genosl`<br>  - attribute: `file` changed on `IM_translucent_bsdf_genosl` from `( pbrlib/genosl/mx_translucent_bsdf.inline )` to `( legacy/mx_translucent_bsdf.osl )`<br>|
| <a href="../documents/definitions/transpose.html">IM_transpose_matrix33_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_transpose.inline` ) removed from: `IM_transpose_matrix33_genglsl`<br>- attribute: `sourcecode`=( `transpose({{in}})` ) added to: `IM_transpose_matrix33_genglsl`<br>|
| <a href="../documents/definitions/transpose.html">IM_transpose_matrix33_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transpose.inline` ) removed from: `IM_transpose_matrix33_genosl`<br>- attribute: `sourcecode`=( `transpose({{in}})` ) added to: `IM_transpose_matrix33_genosl`<br>|
| <a href="../documents/definitions/transpose.html">IM_transpose_matrix44_genglsl</a> | - attribute: `file`=( `stdlib/genglsl/mx_transpose.inline` ) removed from: `IM_transpose_matrix44_genglsl`<br>- attribute: `sourcecode`=( `transpose({{in}})` ) added to: `IM_transpose_matrix44_genglsl`<br>|
| <a href="../documents/definitions/transpose.html">IM_transpose_matrix44_genosl</a> | - attribute: `file`=( `stdlib/genosl/mx_transpose.inline` ) removed from: `IM_transpose_matrix44_genosl`<br>- attribute: `sourcecode`=( `transpose({{in}})` ) added to: `IM_transpose_matrix44_genosl`<br>|
| <a href="../documents/definitions/uniform_edf.html">IM_uniform_edf_genglsl</a> |   - attribute: `file` changed on `IM_uniform_edf_genglsl` from `( pbrlib/genglsl/mx_uniform_edf.glsl )` to `( mx_uniform_edf.glsl )`<br>|
| <a href="../documents/definitions/uniform_edf.html">IM_uniform_edf_genosl</a> | - attribute: `file`=( `pbrlib/genosl/mx_uniform_edf.inline` ) removed from: `IM_uniform_edf_genosl`<br>- attribute: `sourcecode`=( `{{color}} * emission()` ) added to: `IM_uniform_edf_genosl`<br>|
| <a href="../documents/definitions/unpremult.html">IM_unpremult_color4_genglsl</a> |   - attribute: `file` changed on `IM_unpremult_color4_genglsl` from `( stdlib/genglsl/mx_unpremult_color4.glsl )` to `( mx_unpremult_color4.glsl )`<br>|
| <a href="../documents/definitions/unpremult.html">IM_unpremult_color4_genosl</a> |   - attribute: `file` changed on `IM_unpremult_color4_genosl` from `( stdlib/genosl/mx_unpremult_color4.osl )` to `( mx_unpremult_color4.osl )`<br>|
| <a href="../documents/definitions/place2d.html">NG_place2d_vector2</a> | - Number of children on: `NG_place2d_vector2` changed from ( 6 ) to ( 11 )<br>- subtract `/N_applyoffset2` add to `NG_place2d_vector2`<br>- rotate2d `/N_applyrot2` add to `NG_place2d_vector2`<br>- divide `/N_applyscale2` add to `NG_place2d_vector2`<br>- add `/N_addpivot2` add to `NG_place2d_vector2`<br>- switch `/N_switch_operationorder` add to `NG_place2d_vector2`<br>  - attribute: `nodename` changed on `out` from `( N_addpivot )` to `( N_switch_operationorder )`<br>|
| <a href="../documents/definitions/range.html">NG_range_color3</a> | - Number of children on: `N_clamp_color3` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_color3/N_clamp_color3`. Value=(  )<br>- input `/high` added to `NG_range_color3/N_clamp_color3`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_color3FA</a> | - Number of children on: `N_clamp_color3FA` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_color3FA/N_clamp_color3FA`. Value=(  )<br>- input `/high` added to `NG_range_color3FA/N_clamp_color3FA`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_color4</a> | - Number of children on: `NG_range_color4` changed from ( 11 ) to ( 10 )<br>- input `/value1` removed from `NG_range_color4`. Value=(  )<br>- Number of children on: `N_clamp_color4` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_color4/N_clamp_color4`. Value=(  )<br>- input `/high` added to `NG_range_color4/N_clamp_color4`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_color4FA</a> | - Number of children on: `N_clamp_color4FA` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_color4FA/N_clamp_color4FA`. Value=(  )<br>- input `/high` added to `NG_range_color4FA/N_clamp_color4FA`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_float</a> | - Number of children on: `N_clamp_float` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_float/N_clamp_float`. Value=(  )<br>- input `/high` added to `NG_range_float/N_clamp_float`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector2</a> | - Number of children on: `N_clamp_vector2` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector2/N_clamp_vector2`. Value=(  )<br>- input `/high` added to `NG_range_vector2/N_clamp_vector2`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector2FA</a> | - Number of children on: `N_clamp_vector2FA` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector2FA/N_clamp_vector2FA`. Value=(  )<br>- input `/high` added to `NG_range_vector2FA/N_clamp_vector2FA`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector3</a> | - Number of children on: `N_clamp_vector3` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector3/N_clamp_vector3`. Value=(  )<br>- input `/high` added to `NG_range_vector3/N_clamp_vector3`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector3FA</a> | - Number of children on: `N_clamp_vector3FA` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector3FA/N_clamp_vector3FA`. Value=(  )<br>- input `/high` added to `NG_range_vector3FA/N_clamp_vector3FA`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector4</a> | - Number of children on: `N_clamp_vector4` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector4/N_clamp_vector4`. Value=(  )<br>- input `/high` added to `NG_range_vector4/N_clamp_vector4`. Value=(  )<br>|
| <a href="../documents/definitions/range.html">NG_range_vector4FA</a> | - Number of children on: `N_clamp_vector4FA` changed from ( 1 ) to ( 3 )<br>- input `/low` added to `NG_range_vector4FA/N_clamp_vector4FA`. Value=(  )<br>- input `/high` added to `NG_range_vector4FA/N_clamp_vector4FA`. Value=(  )<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_color3</a> | - Number of children on: `NG_triplanarprojection_color3` changed from ( 22 ) to ( 25 )<br>- normalize `/N_norm_color3` removed from `NG_triplanarprojection_color3`<br>- dotproduct `/N_dotX_color3` removed from `NG_triplanarprojection_color3`<br>- dotproduct `/N_dotY_color3` removed from `NG_triplanarprojection_color3`<br>- dotproduct `/N_dotZ_color3` removed from `NG_triplanarprojection_color3`<br>- absval `/N_blendX_color3` removed from `NG_triplanarprojection_color3`<br>- absval `/N_blendY_color3` removed from `NG_triplanarprojection_color3`<br>- absval `/N_blendZ_color3` removed from `NG_triplanarprojection_color3`<br>- normalize `/N_norm_vector3` add to `NG_triplanarprojection_color3`<br>- absval `/N_absN` add to `NG_triplanarprojection_color3`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_color3`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_color3`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_color3`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_color3`<br>- power `/N_blendPower` add to `NG_triplanarprojection_color3`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_color3`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_color3`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_color3`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_color3 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_color3 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_color3 )` to `( N_separateWeights )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_color4</a> | - Number of children on: `NG_triplanarprojection_color4` changed from ( 22 ) to ( 25 )<br>- normalize `/N_norm_color4` removed from `NG_triplanarprojection_color4`<br>- dotproduct `/N_dotX_color4` removed from `NG_triplanarprojection_color4`<br>- dotproduct `/N_dotY_color4` removed from `NG_triplanarprojection_color4`<br>- dotproduct `/N_dotZ_color4` removed from `NG_triplanarprojection_color4`<br>- absval `/N_blendX_color4` removed from `NG_triplanarprojection_color4`<br>- absval `/N_blendY_color4` removed from `NG_triplanarprojection_color4`<br>- absval `/N_blendZ_color4` removed from `NG_triplanarprojection_color4`<br>- normalize `/N_norm_vector3` add to `NG_triplanarprojection_color4`<br>- absval `/N_absN` add to `NG_triplanarprojection_color4`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_color4`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_color4`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_color4`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_color4`<br>- power `/N_blendPower` add to `NG_triplanarprojection_color4`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_color4`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_color4`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_color4`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_color4 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_color4 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_color4 )` to `( N_separateWeights )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_float</a> | - Number of children on: `NG_triplanarprojection_float` changed from ( 22 ) to ( 25 )<br>- normalize `/N_norm_float` removed from `NG_triplanarprojection_float`<br>- dotproduct `/N_dotX_float` removed from `NG_triplanarprojection_float`<br>- dotproduct `/N_dotY_float` removed from `NG_triplanarprojection_float`<br>- dotproduct `/N_dotZ_float` removed from `NG_triplanarprojection_float`<br>- absval `/N_blendX_float` removed from `NG_triplanarprojection_float`<br>- absval `/N_blendY_float` removed from `NG_triplanarprojection_float`<br>- absval `/N_blendZ_float` removed from `NG_triplanarprojection_float`<br>- normalize `/N_norm_vector3` add to `NG_triplanarprojection_float`<br>- absval `/N_absN` add to `NG_triplanarprojection_float`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_float`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_float`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_float`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_float`<br>- power `/N_blendPower` add to `NG_triplanarprojection_float`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_float`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_float`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_float`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_float )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_float )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_float )` to `( N_separateWeights )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_vector2</a> | - Number of children on: `NG_triplanarprojection_vector2` changed from ( 22 ) to ( 25 )<br>- normalize `/N_norm_vector2` removed from `NG_triplanarprojection_vector2`<br>- dotproduct `/N_dotX_vector2` removed from `NG_triplanarprojection_vector2`<br>- dotproduct `/N_dotY_vector2` removed from `NG_triplanarprojection_vector2`<br>- dotproduct `/N_dotZ_vector2` removed from `NG_triplanarprojection_vector2`<br>- absval `/N_blendX_vector2` removed from `NG_triplanarprojection_vector2`<br>- absval `/N_blendY_vector2` removed from `NG_triplanarprojection_vector2`<br>- absval `/N_blendZ_vector2` removed from `NG_triplanarprojection_vector2`<br>- normalize `/N_norm_vector3` add to `NG_triplanarprojection_vector2`<br>- absval `/N_absN` add to `NG_triplanarprojection_vector2`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_vector2`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_vector2`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_vector2`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_vector2`<br>- power `/N_blendPower` add to `NG_triplanarprojection_vector2`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_vector2`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_vector2`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_vector2`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_vector2 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_vector2 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_vector2 )` to `( N_separateWeights )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_vector3</a> | - Number of children on: `NG_triplanarprojection_vector3` changed from ( 22 ) to ( 25 )<br>- dotproduct `/N_dotX_vector3` removed from `NG_triplanarprojection_vector3`<br>- dotproduct `/N_dotY_vector3` removed from `NG_triplanarprojection_vector3`<br>- dotproduct `/N_dotZ_vector3` removed from `NG_triplanarprojection_vector3`<br>- absval `/N_blendX_vector3` removed from `NG_triplanarprojection_vector3`<br>- absval `/N_blendY_vector3` removed from `NG_triplanarprojection_vector3`<br>- absval `/N_blendZ_vector3` removed from `NG_triplanarprojection_vector3`<br>- absval `/N_absN` add to `NG_triplanarprojection_vector3`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_vector3`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_vector3`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_vector3`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_vector3`<br>- power `/N_blendPower` add to `NG_triplanarprojection_vector3`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_vector3`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_vector3`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_vector3`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_vector3 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_vector3 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_vector3 )` to `( N_separateWeights )`<br>|
| <a href="../documents/definitions/triplanarprojection.html">NG_triplanarprojection_vector4</a> | - Number of children on: `NG_triplanarprojection_vector4` changed from ( 22 ) to ( 25 )<br>- normalize `/N_norm_vector4` removed from `NG_triplanarprojection_vector4`<br>- dotproduct `/N_dotX_vector4` removed from `NG_triplanarprojection_vector4`<br>- dotproduct `/N_dotY_vector4` removed from `NG_triplanarprojection_vector4`<br>- dotproduct `/N_dotZ_vector4` removed from `NG_triplanarprojection_vector4`<br>- absval `/N_blendX_vector4` removed from `NG_triplanarprojection_vector4`<br>- absval `/N_blendY_vector4` removed from `NG_triplanarprojection_vector4`<br>- absval `/N_blendZ_vector4` removed from `NG_triplanarprojection_vector4`<br>- normalize `/N_norm_vector3` add to `NG_triplanarprojection_vector4`<br>- absval `/N_absN` add to `NG_triplanarprojection_vector4`<br>- dotproduct `/N_dotN` add to `NG_triplanarprojection_vector4`<br>- divide `/N_normalizeWeights` add to `NG_triplanarprojection_vector4`<br>- clamp `/N_clampForPrecision` add to `NG_triplanarprojection_vector4`<br>- divide `/N_oneOverBlend` add to `NG_triplanarprojection_vector4`<br>- power `/N_blendPower` add to `NG_triplanarprojection_vector4`<br>- dotproduct `/N_dotBlendedN` add to `NG_triplanarprojection_vector4`<br>- divide `/N_normalizeBlendedWeights` add to `NG_triplanarprojection_vector4`<br>- separate3 `/N_separateWeights` add to `NG_triplanarprojection_vector4`<br>- attribute: `output`=( `outx` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendX_vector4 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outy` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendY_vector4 )` to `( N_separateWeights )`<br>- attribute: `output`=( `outz` ) added to: `in2`<br>  - attribute: `nodename` changed on `in2` from `( N_blendZ_vector4 )` to `( N_separateWeights )`<br>|

</details>


### Higher Level Comparisons

As can be seen, the comparisons only provide low level details of count and value changes but not any higher level
semantic meaning. Some examples include:

- The change to inline source code was done for consistency and performance reasons but shows up as a a few hundred attribute changes. Some additional logic could be added to detect a `file` attribute to `sourcecode` attribute but
this could be "brittle" long-term if these attribute names ever changed. Additional information such as how to
perform this update would be useful. For example the script used to perform this can be found in 
<a href="./inlineImplementationFiles.py" target="_blank">inlineImplementationFiles.py</a> file included here.

- Changes for node graph implementations are very hard to understand as only textual comparison is performed
with possible new values and connections reflected when comparing the definitions.

It would probably be more useful to traverse the graphs via it's connections to give a better sense of whether there have been any topological changes (added / removed nodes, changes in connections). Better visual feedback could be given by drawing out the graphs as diagrams. This is beyond the scope of this notebook.

- Changes due to inheritance, whether added, removed or changed, are very hard determine though the change in the `inherit`
attribute or a version change is a first indicator. The utility inheritance functions added here would be useful if added to the core distribution.

- There is a very large number of changes between the previous and current patch release tested (at time of writing).
The same comparison could be performed between patches to see incremental changes.
